<font size="6">Building Fast Queries on a CSV</font>

We will imagine that we are researchers monitoring the discourse about the climate changes on reddit, to find out what people think and discuss on this topic.

The goal of this project is to create a class that represent the comments about the climate changes on reddit. The methods in that class will implement the queries that we want to answer about our inventory. We will also preprocess that data to make those queries run faster.

In [ ]:
from csv import reader

opened_file = open('the-reddit-climate-change-dataset-comments.csv', encoding='UTF-8')
read_file = reader(opened_file)
rows = list(read_file)
header = rows[0]
rows = rows[1:]
print(header)
for i in range(2):
    print(rows[i])

['type', 'id', 'subreddit.id', 'subreddit.name', 'subreddit.nsfw', 'created_utc', 'permalink', 'body', 'sentiment', 'score']
['comment', 'imlddn9', '2qh3l', 'news', 'false', '1661990368', 'https://old.reddit.com/r/news/comments/x2cszk/us_life_expectancy_down_for_secondstraight_year/imlddn9/', 'Yeah but what the above commenter is saying is their base doesn’t want any of that. They detest all of those things, even the small gradual changes. Investing in nuclear energy is a tacit acknowledgement of man made climate change. Any acknowledgement or concession and they will be primaried out in a minute', '0.5719', '2']
['comment', 'imldbeh', '2qn7b', 'ohio', 'false', '1661990340', 'https://old.reddit.com/r/Ohio/comments/x2awnp/state_government_may_soon_kill_a_solar_project_in/imldbeh/', "Any comparison of efficiency between solar and fossil fuels is nonsensical at best and intentionally misleading at worst. In no universe is light -&gt; photovoltaic cell -&gt; electricity less efficient than

<font size="5">Reddit Class</font>

Let's start by implementing the constructor. It will take the name of the CSV file as argument and then read the rows contained in self.header and self.rows.

In [ ]:
from csv import reader
class Reddit():
    def __init__(self, csv_filename):
        opened_file = open(csv_filename, encoding='UTF-8')
        read_file = reader(opened_file)
        rows = list(read_file)
        self.header = rows[0]
        self.rows = rows[1:]
            
reddit = Reddit('the-reddit-climate-change-dataset-comments.csv')
print(reddit.header)
print(len(reddit.rows))

['type', 'id', 'subreddit.id', 'subreddit.name', 'subreddit.nsfw', 'created_utc', 'permalink', 'body', 'sentiment', 'score']
4600698


<font size="5">Finding a Comment Form the Id</font>

Throughout this project, we will make several improvements to the Comment class.<br>
The first thing that we will implement is a way to look up a comment from a given identifier. In this way, we can register and analyze the comment that we think is relevant.

The idea is proceprocess the data into a dictionary where the keys are the IDs and the values the rows.

In [ ]:
from csv import reader
class Reddit():
    def __init__(self, csv_filename):
        opened_file = open(csv_filename, encoding='UTF-8')
        read_file = reader(opened_file)
        rows = list(read_file)
        self.header = rows[0]
        self.rows = rows[1:]
        self.id_to_row = {}
        for row in self.rows:
            self.id_to_row[row[1]] = row
            
    def get_comment_from_id(self, comment_id):
        if comment_id in self.id_to_row:
            return self.id_to_row[comment_id]
        return None

<font size="5">Analized Sentiment</font>

On our database, we have the analyzed sentiment of each comment, with the upper limit being defined as 1 and the lower limit as -1. This sentiments can be used to see how the comments were written, in a negative, neutral or positive way, which can inform us how this person sees the climate changes.

We will write a function that, given an interval between -1 and 1, return the messagens. The idea is proceprocess the data into a dictionary where the keys are the Sentiments and the values the rows.

In [1]:
from csv import reader
class Reddit():
    def __init__(self, csv_filename):
        opened_file = open(csv_filename, encoding='UTF-8')
        read_file = reader(opened_file)
        rows = list(read_file)
        self.header = rows[0]
        self.rows = rows[1:]
        self.id_to_row = {}
        for row in self.rows:
            self.id_to_row[row[1]] = row
            
    def get_comment_from_id(self, comment_id):
        if comment_id in self.id_to_row:
            return self.id_to_row[comment_id]
        return None
    
    def analized_sentiment(self, lim_inf, lim_sur):
        list_sentiment = []
        if lim_inf < -1 or lim_sur > 1:
            return None
        for rows in self.rows:
          if rows[-2] == '':
              rows[-2] = '0.0'
          sent = float(rows[-2])
          mess = rows[-3]
          if sent > lim_inf and sent < lim_sur:
              list_sentiment.append(mess)
        return list_sentiment

<font size="5">Comment Score</font>

With the score atribute, we can determine which comments are the most highly rated and, consequently, the most important one.

We will write a function that, given an score, return two messagens whose sum of the "score return" column value is equal to the score.

In [ ]:
from csv import reader
class Reddit():
    def __init__(self, csv_filename):
        opened_file = open(csv_filename, encoding='UTF-8')
        read_file = reader(opened_file)
        rows = list(read_file)
        self.header = rows[0]
        self.rows = rows[1:]
        self.id_to_row = {}
        for row in self.rows:
            self.id_to_row[row[1]] = row
            
    def get_comment_from_id(self, comment_id):
        if comment_id in self.id_to_row:
            return self.id_to_row[comment_id]
        return None
    
    def analized_sentiment(self, lim_inf, lim_sur):
        list_sentiment = []
        if lim_inf < -1 or lim_sur > 1:
            return None
        for rows in self.rows:
          if rows[-2] == '':
              rows[-2] = '0.0'
          sent = float(rows[-2])
          mess = rows[-3]
          if sent > lim_inf and sent < lim_sur:
              list_sentiment.append(mess)
        return list_sentiment
    
    def score(self, score):
        score_messagens = []
        for row1 in self.rows:
            for row2 in self.rows:
                if float(row1[-1]) + float(row2[-1]) == score:
                  score_messagens.append(row1[-3])
                  score_messagens.append(row2[-3])
                  return score_messagens
        return -1

In [ ]:
!pip install pytest pytest-sugar

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
%%file test_data.py
import pytest

from csv import reader
class Reddit():
    def __init__(self, csv_filename):
        opened_file = open(csv_filename, encoding='UTF-8')
        read_file = reader(opened_file)
        rows = list(read_file)
        self.header = rows[0]
        self.rows = rows[1:]
        self.id_to_row = {}
        for row in self.rows:
            self.id_to_row[row[1]] = row
            
    def get_comment_from_id(self, comment_id):
        if comment_id in self.id_to_row:
            return self.id_to_row[comment_id]
        return None
    
    def analized_sentiment(self, lim_inf, lim_sur):
        list_sentiment = []
        if lim_inf < -1 or lim_sur > 1:
            return None
        for rows in self.rows:
          if rows[-2] == '':
              rows[-2] = '0.0'
          sent = float(rows[-2])
          mess = rows[-3]
          if sent > lim_inf and sent < lim_sur:
              list_sentiment.append(mess)
        return list_sentiment
    
    def score(self, score):
        score_messagens = []
        #for row in self.rows:
        #    if row[-1] == score:
        #        score_messagens.append(row[-1])
        for row1 in self.rows:
            for row2 in self.rows:
                if float(row1[-1]) + float(row2[-1]) == score:
                  score_messagens.append(row1[-3])
                  score_messagens.append(row2[-3])
                  return score_messagens
        return -1

def test_1():
    """
    Test evaluation for id 'imlddn9'
    """
    reddit = Reddit('the-reddit-climate-change-dataset-comments.csv')
    assert reddit.get_comment_from_id('imlddn9') == ['comment', 'imlddn9', '2qh3l', 'news', 'false', '1661990368', 'https://old.reddit.com/r/news/comments/x2cszk/us_life_expectancy_down_for_secondstraight_year/imlddn9/', 'Yeah but what the above commenter is saying is their base doesn’t want any of that. They detest all of those things, even the small gradual changes. Investing in nuclear energy is a tacit acknowledgement of man made climate change. Any acknowledgement or concession and they will be primaried out in a minute', '0.5719', '2']

def test_2():
    """
    Test evaluation for id imldbeh
    """
    reddit = Reddit('the-reddit-climate-change-dataset-comments.csv')
    assert reddit.get_comment_from_id('imldbeh') == ['comment', 'imldbeh', '2qn7b', 'ohio', 'false', '1661990340', 'https://old.reddit.com/r/Ohio/comments/x2awnp/state_government_may_soon_kill_a_solar_project_in/imldbeh/', "Any comparison of efficiency between solar and fossil fuels is nonsensical at best and intentionally misleading at worst. In no universe is light -&gt; photovoltaic cell -&gt; electricity less efficient than light -&gt; entire food chain -&gt; biomass -&gt; decomposition -&gt; millions of years of geothermal heat and pressure -&gt; extraction -&gt; refining and transport -&gt; burning -&gt; turbine generator -&gt; electricity.\n\nUgly? More ugly than power lines, roads, apartment buildings, tractors, fences, etc etc etc? You're simply not used to it. Also, who cares? Sure, millions of people died in the displacement from sea rise, famines cause by climate change, and wars from resource scarcity, but thank God we didn't have to look at those solar panels occasionally.\n\nOffshore fracking? Fucks sake. Have you already forgotten the deepwater horizon disaster? Think about the damage that did to the gulf coast, add a few million barrels of highly toxic fracking fluid to the mix, and there you have our next major environmental disaster courtesy of offshore fracking. But oh man, those solar panels sure are ugly.\n\nStop repeating the lies fed to you by the people getting rich from fossil fuels, who couldn't care less about you or your descendants as long as they get theirs. We can deal with the less environmentally friendly aspects of solar power. We cannot deal with the consequences of continuing to pump carbon into the atmosphere. Y'all need to grow the fuck up and stop whining about the view.", '-0.9877', '2']

def test_3():
    """
    Test evaluation for sentiment between -0.5 and 0.5
    """
    reddit = Reddit('the-reddit-climate-change-dataset-comments.csv')
    assert reddit.analized_sentiment(-0.5,0.5) == ["I'm honestly waiting for climate change and the impacts of that to kick some fucking sense into people. But who am I kidding itll still just be more of the poor suffering while the rich claim victim hood for handouts while letting us all starve. Its honestly hard some days to not just give up, and I truly wonder if and when anything will ever actually be done.",
 "Not just Sacramento. It's actually happening all over the world. Climate change is real, believe it or not.",
 'Naaa how could anyone be mad at a face like that... Must definitely be climate change',
 "They need to change laws so it's more worth selling agriculture products in the US rather than export it.  They also need to change laws so there are monetary penalties for growing crops that are not particularly viable to an area's natural climate.  As it stands right now, my neighbor makes double the price per head of cattle by exporting out of country than he would selling right here.  All the people complaining about climate change on here should probably be complaining about this too.",
 "Blaming environmentalists for the failure of nuclear energy world wide is just such a weird take, when did governments/companies ever give a fuck about their opinion? Never as seen by the small problem of climate change that we have now. But what do they care about? MONEY! \n\nNuclear energy was never cheap like the industry promised in the 70s. In the past it wasn't competitive against fossil fuels and now it isn't against renewables.",
 "&gt;Not climate change mind you\n\nHi. I have a degree in Climate Science. While California has been wetter in the past 150 years compared to the state's recent paleoclimatological history, the current aridification of the American west is driven by human-caused climate change and excessive water usage.",
 'Climate change is not "staring" you in the face, it has been in your face from the day you were born and will stay there \'till you\'re 6 feet under. You won\'t be there to face "the end of times", neither will the children of your children. Fixing thd climate will take as long as we\'ve been destroying it.\n\nWe will never call this good times. Comparing economy with climate makes no sense.',
 'The overturning of Roe V Wade, children are literally forced to keep a fetus in their body that cannot live after being raped because it still has a heart beat with a significant chance of permanently harming or killing the child mother (10 years old 10 YEARS OLD!!! in the last example I heard about). This is beyond disgusting, and if something like this can pass (and still find approval), then the sky is the limit. Members of the supreme court have openly said they are going after inter racial marriage, and same sex marriage. \n\nGerrymandering is so pervasive now its destroyed the ability of congress to actually represent its constituents. \n\nWe have members of the government openly spouting blatant hate speech and pushing easily debunked conspiracy theories. Yet the majority of their party still supports them. \n\nHigher education is absurdly expensive, in the 1960\'s you could go to a university for less than 1k a year, adjusted for inflation. Before that believe it or not university was actually free all over the place. \n\nCorporations continue to gain more and more power, the wealth gap in the US is like that before the French revolution. Its literally multiplied in the last couple decades. \n\nStaple foundational books are being taken out of school library\'s. Laws are going up saying that school library\'s have to show "both sides" ...the example of the holocaust was brought up, and yes there needs to be a pro-holocaust perspective. \n\nTo pass the "biggest green climate change helping" legislation in ages they lumped in TONS of regressive fossil fuel industry help. \n\nI don\'t remember the exact specifics (its been a long day), but basically the court is going to be possibly allowing those that normally transfer the election results to decide that the election was rigged and to cast the vote for someone that did not win the election. (I wish I was making this stuff up)\n\n...the list goes on and on, I\'m tired. Not saying, there have not been positive changes, however there are some huge shifts going on that honestly could bring the US to its knee\'s its not forgone, but it ABSOTLUTELY should not be ignored.',
 "You should check out Paul Nicklen's (the guy in the video) Instagram account. The man basically lives in Antarctica and runs a non profit called Sea Legacy that is working to stem the tide of climate change and ocean pollution",
 'Damn that climate change',
 'Correct, "climate change" is not the same thing as the air quality index in Shanghai.  No, I don\'t think you get it.',
 "I'm sure it's climate change.  Probably has nothing to do with the vaccine.",
 'Climate Change',
 "Federal protection was introduced to protect them (and many other birds) from the feather and egg trades, so anything else that negatively effects them like habitat loss or loss of food (such as through development or climate change) isn't prohibited.\n\nedit:changed is to isn't. woops.",
 'Sheesh, these folks would have you believing snow is black and climate change makes it white.\n\nNothing makes sense',
 "Doesn't change the fact that this is short sighted, because the effect of climate change are going to be many many many times costlier than any amount of nuclear plant we could build.",
 'I can’t speak for your exact scenario but the SYGAV unit I have ties into the CANBUS system and I can change the climate control settings through the touchscreen AND via the climate control buttons below.',
 'The first part is what they should say.  The second part makes no sense at all.  It would cost more to move everyone into space then it would to fix climate change.',
 'Yeah I remember an article somewhere suggesting climate change was causing an increase in this and other heart-related issues in young people. Not sure if any formal study was done',
 'I’m sure their first suggestion was a MASSIVE reduction in annual military spending an audit of existing waste and a pledge to not topple democratically elected leaders in the areas most likely to be destabilized by climate change right?',
 'I think we should develop underwater cities, aside from protection from climate change extremes, there may be planets where we have to live under water or under ice, or under liquid methane etc.',
 'Ambient particular matter pollution is not climate change. It’s pollution. The actual temperature change or weather extremities do not meaningfully contribute to the risk factor at all.\n\nTo equate particulate pollution to climate change is very disingenuous imo.',
 '&gt;This and this period of time doesn\'t seem to be any different.\n\nThe difference is that in the past machines replaced human physical labour and pushed humans to leverage their mental intelligence to justify their labour value instead. Now artificial intelligence replaces human mental intelligence. What do humans have to leverage next?\n\nThe main point is that artificial intelligence is a fundamentally unique type of automation.\n\nBut often people fall into the fallacy of believing that "human society always eventually advances in a positive direction" is an unfallable law of nature based on an insignificant 100k year record which biases them towards over-valuing historical trends and prevents them from correctly appraising new things.\n\nSimilar reason for why people have difficulty recognizing how human intervention can drive climate change and mass extinction of millions of species. It is difficult to recognize how advancement enables human actions to negatively impact the environment on a planetary scale. Especially so when historical biases drive people to assume that humans are, and have always been mostly insignificant when discussing planetary scale events.',
 "Oh we're absolutely fucked for climate change. \n\nEven if there were the political will to turn down industry money and do anything meaningfully positive there, every thing we attempt to do to improve things going forward will be called tyranny by the fucking mush brains who have propagated in the age of social media and disinformation",
 'Ironically these are the same people who pretend to be so concerned with climate change/ etc.',
 'And Climate Change.  \n\n\nNeed more people to take the bus, bike or walk.   Only way we will survive is if there are less cars and more bikes.',
 "they're not wrong... people try to debunk climate change and the fossil record with the bible. *And they're serious about it*. ***And they teach their kids about it.***",
 'Greatest country on earth strikes again. Major cities without water? Whole states power grid buckling under self-imposed climate change? A million plus dead from a pandemic? Sorry, unless it can produce immediate, infinitely-growing profit for someone who doesn’t need money, the American plan is to just do nothing. We’re told that government serving the quality of life of the citizens would be socialism, which they tell us is an unforgivable evil. \n\nBut we need to funnel another 1.5 billion to our arms manufacturers, betcha that’ll have no difficulty immediately passing through congress.',
 "Forget this milquetoast center-left rainbow capitalism. If this won't stop climate change, wealth inequality, consumerism, wage theft, private prisons, excessive working hours, food shortages, expensive rent, expensive healthcare, or cities designed around cars, then I'm not in.",
 'Racist! Semi-Fascist! Climate change! Reeeeeeeeeee',
 'I think you need to deal with this asap and consider that the near future holds even more aggressive rain than we get now\n\n&amp;#x200B;\n\nyay climate change.',
 'Is anybody else paying attention to the insane weather events happening in china? Is climate change the black swan that takes us all out?',
 'Climate Change and too many electric cars',
 'Europe, Pakistan, China, the western US, and more are all dealing with unprecedented accelerating climate change driven droughts, floods, and energy crises. I have a difficult time seeing why so much ink is spilled on AI-gone-wrong extinction scenarios given the massive ongoing Anthropocene extinction event.',
 'What do you think the definition of climate change is?  lol',
 'This has been a strong ecumenical view since forever, yet individual Christians keep doing stupid stuff like not believing in climate change.',
 'The actual transformation necessary for us to stop climate change within our lifetimes',
 "Was alive for 58 minutes. The original [post](https://np.reddit.com/r/worldnews/comments/x2qgek/claims_of_torture_of_china_uyghurs_credible_un/).\n\nThe deleted submission has been flagged with the flair **Already Submitted**\n\nI tried to find other articles: \n\n| Post | Confidence | Old | Score |\n|:- |-: |:-: |:-: |\n| [Claims of torture of China Uyghurs are credible - UN](https://np.reddit.com/r/worldnews/comments/x2q3z5/claims_of_torture_of_china_uyghurs_are_credible_un/)^(hot) | 103.0% | an hour | [+639/c98] |\n| [Claims of torture of China Uyghurs are credible - UN](https://np.reddit.com/r/worldnews/comments/x2q76t/claims_of_torture_of_china_uyghurs_are_credible_un/)^(hot) | 103.0% | an hour | [+30/c4] |\n| [UN rights chief 'under tremendous pressure' over report on China's Uyghurs](https://np.reddit.com/r/worldnews/comments/wxcpop/un_rights_chief_under_tremendous_pressure_over/)^(removed) | 45.984% | 6 days | [#83/+23/c8] |\n| [Point of no return: crunch time as China tries to fend off property crash](https://np.reddit.com/r/worldnews/comments/wzwelq/point_of_no_return_crunch_time_as_china_tries_to/)^(removed) | 43.017% | 3 days | [#14/+258/c58] |\n| [Defiant Taiwan says it has 'resolve' to stand up to China on battle anniversary](https://np.reddit.com/r/worldnews/comments/wvkn35/defiant_taiwan_says_it_has_resolve_to_stand_up_to/) | 42.795% | 8 days | [+76/c1] |\n| [Its largest lake is so dry, China digs deep to water crops](https://np.reddit.com/r/worldnews/comments/wvymhs/its_largest_lake_is_so_dry_china_digs_deep_to/)^(removed) | 42.736% | 8 days | [#44/+48/c32] |\n| [Taiwan vows to 'counter-attack' if China's forces enter its territory](https://np.reddit.com/r/worldnews/comments/x26eks/taiwan_vows_to_counterattack_if_chinas_forces/)^(removed) | 42.607% | 16 hours | [#42/+31/c4] |\n| ['Man of the Hole': Last of his tribe dies in Brazil](https://np.reddit.com/r/worldnews/comments/x0qid1/man_of_the_hole_last_of_his_tribe_dies_in_brazil/) | 42.424% | 2 days | [+18/c3] |\n| ['Man of the Hole': Last of his tribe dies in Brazil](https://np.reddit.com/r/worldnews/comments/x0lsnf/man_of_the_hole_last_of_his_tribe_dies_in_brazil/) | 42.424% | 2 days | [+70/c8] |\n| ['Man of the Hole': Last of his tribe dies in Brazil](https://np.reddit.com/r/worldnews/comments/x0yvur/man_of_the_hole_last_of_his_tribe_dies_in_brazil/)^(removed) | 42.424% | 2 days | [#70/+31/c6] |\n| ['Man of the Hole': Last of his tribe dies in Brazil](https://np.reddit.com/r/worldnews/comments/x0r50i/man_of_the_hole_last_of_his_tribe_dies_in_brazil/) | 42.424% | 2 days | [+59/c4] |\n| [Solomon Islands Bans U.S. Navy Ships From Ports After China Security Deal](https://np.reddit.com/r/worldnews/comments/x1f8et/solomon_islands_bans_us_navy_ships_from_ports/) | 41.364% | a day | [+179/c35] |\n| [Zelenskiy vows to restore Ukrainian rule in Crimea](https://np.reddit.com/r/worldnews/comments/wvmk83/zelenskiy_vows_to_restore_ukrainian_rule_in_crimea/) | 40.816% | 8 days | [+7978/c482] |\n| [Tuvalu minister says both China, U.S. needed for climate change action](https://np.reddit.com/r/worldnews/comments/wy72hh/tuvalu_minister_says_both_china_us_needed_for/) | 40.593% | 5 days | [+4/c0] |\n\n\n---\n\n*^(This info could also be completely unrelated or unhelpful in which case I apologize. I'm still learning.)*\n\n^^Show ^^me ^^what ^^you ^^got ^^/u/coverageanalysisbot",
 "Hear that! I'd cry a river, but they're all dried up due to climate change",
 'climate change?',
 "Here's some fun. Take a look at snowfalls and green Christmases in Canada. Can you spot a trend? Almost every city is experiencing less snow on average and more Green Christmases. It's especially worse in some areas.\n\n[https://www.canada.ca/en/environment-climate-change/services/weather-general-tools-resources/historical-christmas-snowfall-data.html](https://www.canada.ca/en/environment-climate-change/services/weather-general-tools-resources/historical-christmas-snowfall-data.html)\n\nCheck out change of frequency, snow depth in the past, and green Christmases.\n\nSarnia is nice, 6 in 30 years, then 21 green Christmases in the past 30.\n\nHow many cities on there are having more snow now?",
 'Probably not too much. Sindh Province has received 784% higher than average rain while Baluchistan received 500%. In Monsoon season. So yeah, that’s gonna do damage no matter what. Climate change almost certainly caused that.',
 'And I’m sure nothing you do contributes to climate change',
 'I can\'t say how bad over the next year. Year by year you can have pretty drastic differences, but it\'s the long-term trend you have to look at. But 100 years from now, I don\'t think you\'re going to find many humans on this planet, if you find any... We\'re very rapidly approaching a point where a virologist with the motivation and modest means can produce a handful of deadly, contagious viruses and wipe out most of mankind. This isn\'t crazy talk. You can order DNA on the internet. You can buy the equipment to manufacture viruses for less than the cost of a house. We have complete genomes (publicly available info) for thousands of viruses.\n\nSome might ask, "why would someone do that?" I\'d say, "Ask Stephen Paddock, Jim Jones, David Koresh, and Jin Ruchao, among others." You can\'t tell me Stephen Paddock wouldn\'t have wiped out mankind given the opportunity. So it\'s not a matter of IF it will happen. It\'s simply a matter of when a virologist gets angry enough.\n\nAnd if by some chance that just doesn\'t happen (maybe civilization collapses first), then we\'re screwed in a number of other ways. Every day there are stories about how climate change is worse than they originally predicted. Today from Scientific American, "This Hot Summer Is One of the Coolest of the Rest of Our Lives" An article on CBS today about lakes in the Arctic disappearing, "Scientists long expected that Arctic lakes would expand with climate change as ground ice continued to melt and climate models showed that drying would not be seen until at least 2060 or 2150." Oops, guess they botched that prediction pretty badly. All the glaciers are disappearing way faster than they ever predicted. Nobody predicted the massive methane emissions from thawing permafrost, and thawing permafrost wasn\'t anything we were expecting to see in the 2000s, if you asked 20 years ago.\n\nA lot of people find this really depressing. I mean, I\'ve done a good bit of research in this and I don\'t really see a silver lining. Our best case scenario is that there will be a massive loss of life and maybe some people will survive long-term, but I think it\'s unlikely. I\'m way past the emotional side of this. Until someone can show me something super-promising, I see this as more or less, a fact waiting to happen.\n\nDid I mention the plummeting fertility rates for men? Between 1930 and 1992, sperm counts in men in America dropped about 50%. They continue to fall and those rates are now dropping worldwide. Within a few decades, most men will be effectively infertile. So you tag that on to all the other stuff going wrong and it\'s going to be hard to survive.',
 "The failure of Yucca Mountain, Fukushima, the leaks found at San Onofre, Chernobyl and the fact that nuclear sites can apparently be used to store your troops and ammo does not make me feel reassured.     \nTell me where you're going to store the waste, tell me how you're going to remove the fissile material under extreme circumstances, how to stop a meltdown, and how to protect it from being used as enemy strongholds and you've got my support.  Specially with sea level rise and extreme climate change your can't guarantee that the location you pick today to build a plant will be safe tomorrow.",
 "Unfortunately it's likely to get colder there. The last time the northern hemisphere warmed up it tossed Europe into an ice age.\n\nClimate change is weird like that.",
 'Well, according to this [source](https://news.stanford.edu/2016/09/01/extreme-weather-winters-becoming-common/), this [source](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5928063/), this [source](https://www.science.org/doi/10.1126/science.abi9167), and this [source](https://journals.ametsoc.org/configurable/content/journals$002fclim$002f33$002f18$002fjcliD190994.xml?t:ac=journals%24002fclim%24002f33%24002f18%24002fjcliD190994.xml&amp;tab_body=fulltext-display), (these are all academic sources mind you) it’s happening everywhere. Global climate change will mean more extreme winters.\n\nThat frozen stuff also won’t necessarily be arable for crops, and you also have to factor in the effects of [soil degradation](https://link.springer.com/article/10.1007/s10661-020-08429-5), [water resources](https://www.researchgate.net/profile/J-Tarafdar/post/How_climate_change_impact_on_water_situation_in_India/attachment/5f4643f36a5a0300017b1b09/AS%3A928740653682689%401598440435831/download/JEEpaper-ImpactofClimateChangeonWaterResourcesinIndia.pdf), and [other issues](https://www.mdpi.com/2071-1050/13/3/1318/htm) that will affect our current amount of arable land thanks to climate change. Events such as flooding as this video shows end up stripping the soil of nutrients needed to grow crops, and these events are much more frequent under climate stress.',
 'Climate change is at its core a naturally occurring phenomenon.',
 "So the one I have is the UK one. The fan speed control and temperature are all on the screen. You press the climate button and it takes you to a separate page on the screen. That's why I'm unsure if I can change it.",
 'Because of projected future scarcity as climate change ravages the worlds ecosystems?',
 'The irony here is- a majority of these people probably claim to be concerned about climate change.',
 'The Intergovernmental Panel on Climate Change (IPCC) provide executive summaries of all their reports, with the idea of making the language simple and understandable. \n\nThey are completely objective, and the only people questioning their objectivity are those with vested interests themselves. \n\nhttps://www.ipcc.ch/report/ar6/wg2/downloads/report/IPCC_AR6_WGII_SummaryForPolicymakers.pdf',
 'We should not be flying on fossil fuel commercially, considering climate change.',
 'Unfortunately you can’t exactly slay climate change.',
 '“1,200 scientists out of hundreds of thousands disagree with the verbiage of “emergency”, when speaking on climate change, but still very much believe in climate change and global warming” ….is what the headline should have said.\n\nI found and read the article. In a nutshell-  the scientists believe in climate change, but not the non-uniform measures for assessing what is or isn’t considered a climate “emergency”. And it will be important moving forward to distinguish different climate changes based on a trusted method.\n\nI fucking hate headlines and the way news is presented + consumed nowadays. He posted this with NO context 😂 and it doesn’t even defend the fact that global warming isn’t real sooo.. (I’m assuming that’s what his rectangular self was getting at)',
 'https://www.bbc.com/future/article/20210816-how-water-shortages-are-brewing-wars\n\nhttps://www.bloomberg.com/news/articles/2021-11-19/global-warming-is-destroying-crops-can-it-be-stopped-in-time-kw65js74\n\nhttps://press.un.org/en/2018/gaef3499.doc.htm\n\nhttps://www.usnews.com/news/best-countries/articles/2021-10-29/how-climate-change-may-increase-global-conflicts\n\nhttps://www.pnas.org/doi/10.1073/pnas.0703073104\n\nhttps://www.theatlantic.com/science/archive/2022/03/nuclear-war-would-ravage-the-planets-climate/627005/\n\nhttps://www.theguardian.com/environment/2021/jan/13/top-scientists-warn-of-ghastly-future-of-mass-extinction-and-climate-disruption-aoe\n\nhttps://www.smithsonianmag.com/science-nature/without-action-on-climate-another-mass-extinction-event-will-likely-happen-in-the-worlds-oceans-180979991/\n\nhttps://www.bbc.com/news/science-environment-62378157',
 'So here’s me thinking about so many 20 somethings complaining about climate change (FTR I’m 32 and yes believe in climate change) and here is a festival (more than likely) attended by mostly 20 somethings and look at the WASTE. The post pilfering will take care of maybe a 4th, and the rest will end up in a land fill .. just absolutely gross.',
 'Man not even Washington state requires smog testing and they support climate change measures more then anyone. It’s just another tax…',
 "Let's say I own part of a solar panel company but am under the cap.\n\nI vote through climate change measures which increases the companies worth and I'm now above the cap. Do I have to resign immediately? What if the price later goes down?",
 "See that would've been a GOOD character arc but he would have to support fighting climate change though",
 "No, it's pretty simple really. You don't want there to be any actual action on climate change, you'd prefer to just post about it and feel progressive. For there to be any actual action you would need to do what needs to be done to actually make change and you aren't interested in that, 43% isn't good enough so you'll happily take Morrison and 25% reduction as that allows you to continue making pathetic attempts at using rape to feel superior.",
 'Over population is probably the single biggest contributor to Climate change.',
 "And her state is not about to become uninhabitable from drought, because climate change isn't real!",
 'Climate change started well over 40 years ago.',
 "Purely for midterm messaging. \n\nIt's ultimately a climate change and healthcare bill that will have only modest, long-term impact on inflation, mostly via the medicare price negotiation feature and diversification of the energy grid, but no real short term impact. \n\nThat said, I'm supportive of about 80% of the bill so I don't really care what they call it.",
 'Climate change being our fault, that we need to change the whole system.',
 "How many more years of oil do you think are available in the quantities that we are accustomed to?\n\nOne silver lining of petroleum-induced climate change is that we can drill in the Arctic.  Even so, how much longer can we burn this finite resource until there is simply no more oil to be had?\n\nDo we care about the long-term viability of modern civilization?  If so, perhaps we should give serious thought to  this question and it's implications.",
 'Would be nice if African and South American were doing more to fight climate change',
 'Online misinformation about climate change is a super category I guess',
 'There’s room for more than them. But Nixon also created the EPA and did a lot for climate change and the environment. Gotta take the good with the bad.',
 'We deserve climate change.',
 'mf literally romanticizing climate change',
 'Climate change',
 'Serious question for these troubling times: how is the best way to profit from redditors who think that there are like ten years before climate change kills us all',
 'Corporations that invest millions of dollars into campaigns focused on making people feel individually responsible for trash, pollution, and other climate change contributors in an effort to avoid real change and legislation that would actually hold them rightfully accountable.',
 'The US had an exodus of natural gas prior, its almost as if we could… start them back up and sell them to the EU. But ya know… “climate change”',
 'Climate change. Our emissions. Our waste',
 'If humans give up meat for climate change, we deserve extinction.',
 'Some of these people are the same ones screaming about climate change.',
 "I mean, there's evidence the Earth is flat, too. Just look at the horizon. Does it look curved? \n\nThe evidence against man-made climate change is similar - it requires a myopic view that either ignores or distorts the vast bulk of the scientific evidence. \n\nScientists spent much of the 1900s hashing this issue out, and finally got good enough data in the 1950s-1970s to finally come to a conclusion. And its not like another 50 years of even *more* data has changed that. On the contrary, it's firmed up even more.",
 'Although the film crew swore up and down they didn\'t do *anything* to upset the walruses? The facts are different:  \n(This is aside from the fact they spliced footage from TWO different areas and pretended it was one area)  \n- there were polar bears at the top.  \n- they buzzed the walruses with drones frequently.  \n- they set up cameras FAR too close to the ramp the walruses use to get down from there  \n- walruses have been using that area (up at the top) for as long as we know, it isn\'t a "new thing"  \n- walrus numbers shrink and grow quite a bit, and numbers have been duly noted by every explorer of the area. Sometimes huge \'haul outs\' like what was filmed, other times hardly any, long before climate change started.  \n- early explorers noted how polar bears liked to scare walruses over cliffs because in huge herds they\'re quite dangerous, so it\'s safe and easy :&gt;\n\nAttenborough refused to discuss the matter, ever. BUT he did use footage (taken at that time, not new stuff) in a later documentary showing the bears &amp; drones that the film crew swore weren\'t there...',
 'When climate change shuts down the electricity grid due to flooding and unbearable heat… maybe we’ll have our jobs back.',
 'Michael Crichton (the Jurassic Park guy) was a trained scientist. His last book was a footnoted novel "State of Fear" which covered climate change.  I remember it talking about the difference in temperature rise in the last 100 years in a city that has been cemented/ asphalted in verse a rural area 100 miles away which hadn\'t been cemented in, for example.',
 'The best of times, the worst of times - Paul Behrens. It sketches an outlook for the coming period through a pessimistic and an optimistic lense. Good read on the various subjects within climate change',
 "You might want to look outside, because most people do not know this.  Most people think climate change is a thing that happens in like 100 years, and that we can stop it by then.  They're too blind to see it's happening now.",
 'Coastal lowland tiles are ones that can flood with climate change. This thread has some pictures that show examples: https://forums.civfanatics.com/threads/climate-change-quirks.653220/',
 'That’s when you tune them out, though they are more civilized. Their problem is they are the educators. Then, when you say education should be free or open to all classes (including the ones they themselves build, although they are no terrorists, big oil (big pharma, all the bigs), you will find resistance. The most powerful have the most money, the least debt and that is of importance over the others in their class. Either that, or it threatens their business and future, which is always ironic considering some would die for climate change yet never consider those who lose jobs from coal plants. \n\n\nHas anyone figured out who to listen to/ recommend anything once you are past the Conservatives, the Democrats, and the liberals? I know there are good people in all political groups, but it does get pretty ugly once you see through liberals. Ugly and lonely… I try to say that it’s my problem for expecting or searching for an ideology without contractions, but admitting that I actually do believe we shouldn’t have these barriers has lost me many friends over the years. \n\n\nSuggestions welcome. Please no trolling.',
 "J'ai mit le Congo (sous entendu RDC) par préjugé, comme j'aurai pu mettre presque n'importe quel pays d'Afrique sub-saharienne.\n\nSi on s'en tient à des sondages réalisés sur l'Afrique, il y a bien la moitié de la population qui ne sait pas ce qu'est le réchauffement climatique (c'est pas tant du déni mais plutôt qu'ils sont juste pas au courant). [https://theconversation.com/africas-first-continent-wide-survey-of-climate-change-literacy-finds-education-is-key-169426](https://theconversation.com/africas-first-continent-wide-survey-of-climate-change-literacy-finds-education-is-key-169426)\n\nAlors la RDC n'est pas le pire dans le groupe vu qu'ils ont 80% d'alphabétisme, mais si on prend le Niger avec ses 20% d'alphabétisation, c'est pas étonnant que peu de gens soient au courant. Tous les pays de l'étude de ce post ont un taux d'alphabétisation supérieur à 90%, ça fait une grosse différence.\n\nEdit : ce que je voulais dire c'est que si on doit se comparer à des pays qui ont 60% d'alphabétisation ou des dictatures pétrolières alors c'est déjà placer la barre beaucoup trop bas, dans tous les cas c'est la honte.",
 'Turns out when you ignore environmentalists, scientists and evidence about climate change and refuse to invest in renewables on a National scale, when the oil starts to run out and the air is nasty to breathe, the only option you have left is nuclear or societal breakdown. \n\nAnd I’d still pick societal breakdown. \n\nEnvironmentalists are not choosing nuclear because its clean or safe. Lobbyists have made sure it’s the only option on the table.',
 "You have a real problem with reading comprehension and putting words in people's mouth. You haven't given me a shred of proof either. You linked me a graph about CO2 levels, which I never questioned.  I also never said man made climate change doesn't exist. In fact, I said the opposite. Try again Karen.",
 'Global warming/climate change and pollution in general. This should not be a political issue.',
 'If you ever hear about "1,200 scientists agree climate change is a hoax," most of those guys are engineers, too.',
 'Could a permanent lake created by such floods bring humidity and change the climate in the region for a certain period of time?',
 "Any support for him is completely fabricated. The Democratic Party is worthless. They have nothing to offer. Their biggest claim is they hate Trump still, they impeached Trump and Black Lives matter, climate change. It's a dystopia.",
 'Kinda hard to stabilize the home insurance market for a future underwater land mass.\n\nSometimes wealthy people pretend they\'re "skeptical" of climate change in public to get the rubes to keep voting red, but when it\'s time to really get down to it they know much of Florida\'s days above water are numbered.',
 'Pakistan foreign minister:\n&gt;\t"And while we understand that the new reality of climate change means more extreme weather, or monsoons, more extreme heat waves like we saw earlier this year, the scale of the current flood is of apocalyptic proportions. We certainly hope it\'s not a new climate reality."',
 "You are correct. I have no concern for climate change. I leave that in God's hands to not man's.\n\nHow would you express Christ to the world?",
 'Climate change has stopped literally nobody from exploiting their oil. If it was supposed to be a ploy, it failed pretty fucking miserably.\n\nIncidentally, the advent of renewable energy has meant many developing nations with rich mineral deposits have benefited financially (when the US isn’t brazenly overthrowing their governments to get at it, of course). So your argument that it’s a plot to cripple developing nations is again baseless.\n\nCO2, methane, and other byproducts of human industry are targeted because they are the ones we are producing at a colossal industrial scale. Water vapour, unlike these gases, is not an issue because it turns back into water very easily and quickly: That’s where rain comes from.\n\nIt is estimated that humans currently produce 3% of all carbon emissions on Earth. That’s 3% more per annum than what nature was accomodating for before human industrialisation. If you think that doesn’t sound like a lot, run it through an interest calculator for a hundred years.',
 'Can YOU fix climate change?',
 'That is sad. Wondering how climate change affected it, thx for update',
 'This FutureForward ads gives me chills.\n\nBecause climate no change.',
 'Or maybe the climate change IS happening and we are starting to see visible symptoms through world wide drought?',
 "I'm leftish socially and politically. Have been all my life. Not so much so with the climate change catastrophe issue though. Canceling is definitely a thing. I'm surprised you haven't noticed it.",
 "Of course not. Okay, I should've been more specific: the air pollution (displayed as smog) shown in pic 1, is caused by airborne particles (visible), and climate change is caused by greenhouse gas (invisible). There are industries that do both at the same time, however, greenhouse gas is not airborne particles and does not cause smog, nor the airborne particles cause climate change, therefore these two parameters are conventionally counted separately when discussing the environmental impact. And overuse is not pollution by definition since it does not input any substance into the environment.",
 "Makes sense, Europe is turning off all it's nuclear plants.\n\nGiven the whole Russia thing, is insane.\n\nClimate change is real, but not caused by us at all, it is and always had been, the sun.",
 "Now you've gone from Pagan Earth worship to science worship. Just so you understand, science is just the process of testing and disproving. So because I disagree, makes me more scientifically minded than people like you who look at a chart and think it's the Bible. \n\nNothing you've provided proves me wrong.  You're looking at a scale of 10,000 years. A blip in geological time scale. Even a million years is. I also *never claimed that climate change isn't real.* I simply disagree that it's going to be catastrophic for the earth or the human species. You're also completely disregarding the fact that our weather is largely determined by the sun, which goes through cycles. \n\nCoral reefs are actually growing in many places. We have more trees now than 100 years ago. Plant yields will grow. Oceans are going to slowly, very slowly, rise. It will be so slow that it won't be a problem for people to relocate if necessary, but even still, plenty of countries (like the Netherlands) already exist well below sea level. \n\nNone of these climate models even consider human ingenuity or mitigation efforts. I'm much more concerned about over fishing, than I am about climate change. I'll be long dead and the temperature will have risen a whopping 2-3 degrees centigrade. \n\nFurthermore, these models are notoriously inaccurate. 50 years ago, we were all told that all of the evidence points to the Earth cooling and drying. They were telling people in GB to start planting succulents because it'll be so dry.",
 "Climate change is the fascism of weather. Anything I don't like can be attributed to it.",
 'Beautiful, no climate change? I could meditate for hours',
 'Fucking global climate change!',
 "&gt;it's workers that cause climate change. Nonworkers don't expand consumption because they don't expand production.",
 'We have responded to COVID far more effectively than we have ever responded to climate change or are likely to (ozone layer is one counter example).',
 'I mean, there have been a ton. \n\nI’d like you to explain what has happened to all the glaciers and permafrost if climate change isn’t happening?',
 'I wonder which transportation methods are better for climate change than walking if you factor in food emissions.',
 "Va voir par toi-même: https://www.carbonbrief.org/analysis-which-countries-are-historically-responsible-for-climate-change/\n\nDescends jusqu'à Cumulative per-capita emissions et dans la partie gauche du tableau on est numéro 1 producteur de carbone divisé par la population.",
 'For the most part, people stick to the same state but I’ve heard of people moving around state a lot. \n\nWith that being said, the 2 most common reasons I’d hear people moving for are work or change of climate',
 'We have people who love bicycles, eventually change to cars. With our climate and distances, the bicycle is absolutely not popular. The average distance to work is about 15 km, the average travel time by public transport is about 1 hour. You can go once or twice a season. The rental of electric scooters is much more popular. But... with the general conflict of the very idea of 30,000 scooters (and a comparable number of cyclists who use a bicycle for trips) for 8 million cars, they do not make the weather.',
 'yeah his commercial’s are very cringe.  but even aside from his cringeworthiness, this guy should not be anywhere near a public office *at all*.  he’s an outspoken christian nationalist since before the trump era (i think budd attended/graduated from a biblical seminary), he denies that human-caused climate change exists, he obviously opposes any sensible gun law reforms, he’s consistently voted against every single bill that would protect lgbtq+ rights (i think he’s said even in recent times that he still opposes marriage equality), he doesn’t want weed decriminalized, and he was/is in league with deranged traitors on january 6th.  \n\nvote this pig out of office.',
 "The miserable future that climate change will give us if you are don't change society or at least looking after for each other while facing the big drought and famines of the future",
 "Even if climate change was real, us eating bugs and using electric vehicles would do NOTHING as long as china continues to go on like that. But it was never about climate anyway. It's all about control.",
 "I wonder when Gavin Newsom is just going to send out packets of California Climate Change Kool-aid and tell everyone to drink it at the same time. It'll probably be on New Years.",
 "As a species, we made our choice regarding climate change decades ago.\n\nIt wasn't the right one.",
 'Climate change',
 'Die Summe wird sich wohl auf den CO2-Preis bzw. Carbon Offsets beziehen, sollte also als praktisch unerreichbares "Minimum" gesehen werden.\n\nHatte gerade noch [diesen](https://www.reddit.com/r/dezwo/comments/x2p1yq/jetzt_hilft_nur_noch_heiterkeit/) Artikel gelesen, welcher [diesen](https://www.reuters.com/article/climate-change-stimulus/tenth-of-pandemic-stimulus-spend-could-help-world-reach-climate-goals-study-idUSKBN271098) Artikel über eine Studie des Imperial College London zitiert, welche als Kosten zum Erreichen des Zwei-Grad-Ziel mindestens eine Größenordnung mehr nennt - aber immer noch nur ein Zehntel der Hilfsgelder in der Pandemie.\n\nEdit: Dummen Vergleich ausgebessert.',
 'Number one driver of inequality and climate change.',
 "Your species isn't going to die. Well they might, but not from climate change.",
 "It looks like you shared an AMP link. These should load faster, but AMP is controversial because of [concerns over privacy and the Open Web](https://www.reddit.com/r/AmputatorBot/comments/ehrq3z/why_did_i_build_amputatorbot).\n\nMaybe check out **the canonical page** instead: **[https://www.texastribune.org/2022/05/18/climate-change-texas/](https://www.texastribune.org/2022/05/18/climate-change-texas/)**\n\n*****\n\n ^(I'm a bot | )[^(Why &amp; About)](https://www.reddit.com/r/AmputatorBot/comments/ehrq3z/why_did_i_build_amputatorbot)^( | )[^(Summon: u/AmputatorBot)](https://www.reddit.com/r/AmputatorBot/comments/cchly3/you_can_now_summon_amputatorbot/)",
 "I'm quite familiar with the topics of China, Russia, corporations, climate change, arable land (and land use more broadly), and the challenges of producing and distributing food globally. I'd say I have moderate knowledge of chemicals used in agriculture. I'd be happy to discuss this with you, but if you have no interest in discussing nobody is forcing you.",
 'Pretty much every metric except for those related to pollution and climate change',
 '&gt; Klimakrise: Jetzt hilft nur noch Heiterkeit\n&gt; ---\n&gt;\n&gt; **Die Klimabewegung verbohrt sich in der Angst. Aber für Weltuntergangsstimmung haben wir keine Zeit.**\n&gt;\n&gt; *Von [Philipp Bovermann](https://www.sueddeutsche.de/autoren/autor-1.3421045)*\n&gt;\n&gt; 30\\. August 2022 - 6 Min. Lesezeit\n&gt;\n&gt; Das Beste und zugleich das Schlechteste, das man über die Aktionen der Protestbewegung „Letzte Generation“ sagen kann, ist, dass sie traurig machen. Die zumeist jungen Klimaaktivisten scheinen nicht mehr daran zu glauben, dass die Staaten die notwendigen Maßnahmen gegen eine katastrophale Erderhitzung ergreifen werden. Im vergangenen Sommer adressierten sie noch mit einem Hungerstreik im Berliner Regierungsviertel die Bundespolitik. Inzwischen ist ihr Gegner diffus geworden. Sie kleben sich an Autobahnauffahrten fest, an Gemälden. Am Wochenende versuchte eine Gruppe der Aktivisten, sich beim Bundesligaspiel des FC Bayern gegen Borussia Mönchengladbach mit Kabelbindern an die Torpfosten zu ketten. Bayern-Stürmer Thomas Müller, schon qua Name der Normalodeutsche schlechthin, rempelte einen der beiden an und brüllte ihm unhörbar etwas entgegen.\n&gt;\n&gt; **Herrje, können die sich nicht mal zusammenreißen?**\n&gt;\n&gt; Es wäre ein Fehler, die Klimakrise zu psychologisieren – sie ist keine Neurose, die Gefahr ist erdrückend real. Die Psychologie hilft aber, die Sackgasse zu erkennen, in die sich die Klimabewegung derzeit manövriert.\n&gt;\n&gt; In den vergangenen Jahren, seit Greta Thunberg ihre Wut und Verzweiflung in die höchsten politischen Gremien der Welt trug, begann eine Diskussion über die psychologischen Aspekte der anrollenden Klimakatastrophe, darüber, wie es sich anfühlt, auf einem angezählten Planeten zu leben. Meist war da von der „Klimaangst“ die Rede. Selten von der Trauer.\n&gt;\n&gt; Gegen die Angst empfehlen Psychologen das Prinzip der Hoffnung: Die „kognitive Dissonanz“ zwischen dem persönlichen Handeln und der immensen Bedrohung lasse sich nicht mit der Brechstange schließen, sagen sie, durch immer drastischere Warnungen. Damit die Angst nicht lähmend wird, sei es ebenso wichtig, sich zu vergegenwärtigen, was kollektiv und individuell noch alles möglich ist.\n&gt;\n&gt; **Das leuchtet ein: Wer nur Dystopien, aber keine Utopien anzubieten hat, überlässt anderen die Initiative und beschränkt sich aufs Verhindern. Man geht in die innere Sitzblockade**\n&gt;\n&gt; Die findet in den allermeisten Fällen aber nicht auf den Gleisen einer Kohletransport-Trasse statt, sondern in einem beheizten Wohnzimmer, auf dem eigenen, dicker werdenden Hintern.\n&gt;\n&gt; Der Verzweiflung zu entsagen, mutet skeptisch veranlagten Menschen allerdings auch einiges zu. Auf Tiktok geben sich viele junge Klimaaktivisten inzwischen betont optimistisch, handlungsfähig und „resilient“. Zum Beispiel der junge Mann aus Florida, der sich [dabei filmt](https://www.tiktok.com/@trashcaulin/video/7135814842878840110), wie er jeden Tag an den Strand geht und dort mit ganz viel Spaß Plastik sammelt. Das ist besser, als nichts zu tun, klar. Aber es schwingt darin auch der Verdacht protestantischen Kitsches mit. Luther wird das Zitat zugeschrieben, er würde, wenn er wüsste, dass morgen die Welt unterginge, „heute noch ein Apfelbäumchen pflanzen“.\n&gt;\n&gt; **Sollten wir, auch wenn morgen die Welt unterginge, heute noch unseren Müll trennen?**\n&gt;\n&gt; Der Treibhauseffekt würde ja sogar dann für lange Zeit weitergehen, wenn schon morgen kein Gramm Kohlendioxid mehr in die Atmosphäre gelangte. Die Katastrophe entfaltet sich bereits. Schrecklicher als die Angst ist nach Jahrzehnten verpasster Chancen womöglich die Gewissheit, was bereits unaufhaltsam verloren ist. Klimaaktivisten können heute nur noch versuchen, das Unausweichliche zu begrenzen.\n&gt;\n&gt; Vielleicht ist das, was da lähmt und zynisch werden lässt, daher gar nicht so sehr die Angst vor dem Kommenden, sondern es sind unbewältigte Gefühle: Trauer um die eigene jahrelange Unbeschwertheit, die den Kindern und Enkeln nicht mehr offensteht. Trauer, die sich wie ein Vanitas-Zeichen in die noch intakten Landschaften einschreibt. *All das wird irgendwann nicht mehr sein.* Trauer um die Menschen in anderen Teilen der Welt, die viel stärker unter den sich häufenden Dürren und Naturkatastrophen zu leiden haben. So viele von ihnen werden hungern, fliehen müssen und sterben. Wie soll man sich das als Teil der industrialisierten Verursachergemeinschaft verzeihen, gar optimistisch in die Zukunft blicken, während der eigene Lebensstil diese Probleme beschleunigt?\n&gt;\n&gt; **Am diffusesten bei all dem ist die Trauer um die Natur.**\n&gt;\n&gt; **Das Verhältnis zu ihr lässt sich deshalb so schwer fassen, weil große Teile der westlichen Geistesgeschichte aus nichts anderem bestehen als dem Bemühen, den Menschen aus ihr abzusondern.**\n&gt;\n&gt; Die verdrängte Verbundenheit mir ihr bricht nun, da sie zu verarmen und veröden droht, allenthalben als Wehmut wieder auf. Die Bücher und die Künste sind seit Jahren voll davon. In Bäumen, Pilzen, Moosen oder Kraken, überall erkennt der Mensch sich wieder, wanzt sich an die Natur heran und streicht ihr, wie zum Abschied, über die flauschige Wange. Oder aber er lässt sich auf ihr Gewimmel, ihre Amorphie ein, um auf feierliche Art sprachlos zu werden. „(...) und wo auch nur Pflanzen rauschen, klingt immer eine Klage mit“, schrieb der Kulturphilosoph Walter Benjamin.\n&gt;\n&gt; Sich von der Sprachlosigkeit überwältigen zu lassen, durch das Dickicht hindurchzugehen, darin liegt die Chance auf Heilung. Die irreparablen Schäden anzuerkennen, um die Zukunft wieder zu öffnen.\n&gt;\n&gt; **Dann, endlich, gibt es Hoffnung.**\n&gt;\n&gt; Politisch wäre es dringend geboten, diesen Schritt zu tun. Die Klimabewegung hat sich in den Bildern, die sie produziert, zuletzt immer tiefer in der Verzweiflung vergraben. „Fridays for Future“ nährte sich einen ekstatischen Sommer lang vom Glauben, man müsse den Leuten die Klimakrise nur nachdrücklich genug erklären („Listen to the science!“), dann würden sie schon losrennen und das „brennende Haus“, die Erde, löschen. Diese naive Hoffnung erzeugte einen ungeheuren gesellschaftlichen Schwung, überlebte aber die vielen unsichtbaren Widerstände in der Realpolitik nicht lang.\n&gt;\n&gt; Die Gruppe Extinction Rebellion machte kurz darauf auf sich aufmerksam als so etwas wie der dunkelromantische Wiedergänger dieser Träume. Die Aktivisten hielten „Trauermärsche“ ab, schminkten sich die Gesichter totenblass und lasen bei Sitzblockaden traurige Gedichte vor.\n&gt;\n&gt; **Auf ein Banner, das sie an der Westminster Bridge in London entrollten, schrieben sie: *"We\'re fucked".***\n&gt;\n&gt; Die „Letzte Generation“ erzählt nun ebenfalls vom Untergang, dieselbe Geschichte noch mal. Was soll nach ihr noch kommen? Die diesmal wirklich allerletzte Generation?\n&gt;\n&gt; Wo bleibt die Gruppe, die den Menschen die frohe Botschaft der ökologischen Befreiung von den Fesseln des fossilen Zeitalters bringt?\n&gt;\n&gt; **Wo bleibt die erste Generation einer neuen Welt? Es wäre eine bunte, laute Bewegung von ansteckender Heiterkeit, die sich nicht zu schade wäre, auch einmal taktisch fröhlich zu sein, wenn ihr eigentlich nach Weinen zumute wäre.**\n&gt;\n&gt; Die von der Krise direkt Betroffenen können es sich ja auch nicht gestatten, der Verzweiflung nachzugeben.\n&gt;\n&gt; Anstatt die Krise in immer noch und noch dunkleren Farben zu malen, den Berg des Unheils, den es zu erklimmen gäbe, immer weiter aufzutürmen, würden diese Aktivisten die planetare Krise vielmehr kleiner und beherrschbarer erscheinen lassen. Sie würden den Menschen sagen, dass es nicht nur, wohin man auch blickt, kapitalistisches Wachstum gibt, das zwangsläufig in den kollektiven Suizid führt, sondern schon heute Wege in eine bessere Zukunft, auf denen sie mitkommen können. Sie würden ihre Erfolge zu feiern wissen. Auch das Gute wächst ja, langsam und mit Rückschlägen, aber stetig. Grüne Technologien werden immer günstiger. Das Versprechen ist einfach zu gut: kostenlose Energie, die buchstäblich vom Himmel fällt.\n&gt;\n&gt; Noch vor einem Jahrzehnt ergaben Hochrechnungen, dass der damalige Kurs der Klimapolitik zu einem Temperaturanstieg von vier bis fünf Grad geführt hätte – die Erde würde gebacken wie eine Kartoffel. [Aktuell](https://climateactiontracker.org/global/temperatures/) steuert die Welt auf ein Plus von 2,5 bis 2,9 Grad zu. Die Auswirkungen wären katastrophal, aber doch etliche Höllenkreise weniger schlimm.\n&gt;\n&gt; Den Rest zu schaffen und die Erderhitzung auf „deutlich unter zwei Grad“ zu begrenzen, wie im Pariser Klimaabkommen festgehalten, ist keineswegs unrealistisch. Im Oktober 2020 [errechnete](https://www.reuters.com/article/climate-change-stimulus/tenth-of-pandemic-stimulus-spend-could-help-world-reach-climate-goals-study-idUSKBN271098) ein Team von Wissenschaftlern des Imperial College London, dass dafür schon ein Zehntel der zwölf Billionen US-Dollar, die damals als Corona-Rettungspakete versprochen waren, reichen würde. Möglich, dass die Annahmen etwas optimistisch waren. Aber die Rettung der Welt wäre jedenfalls nicht unerschwinglich.\n&gt;\n&gt; Dass es nicht passiert, liegt an ökonomischen und politischen Interessen. Der Logik der Macht begegnet man ebenbürtig nicht in der Logik der Verzweiflung, indem man aus gefühlter Ohnmacht, die Mächtigen treffen zu können, wahllos Konsumenten-Fußvolk gegen sich aufbringt und die Gesellschaft in der Klimafrage weiter spaltet. Den Planeten und seine Bewohner zu schützen, ist unterm Strich auch nur ein Interesse, das es strategisch durchzusetzen gilt. Anstatt Verzicht zu predigen und ständig zu betonen, warum das alles sowieso nicht reicht, sollte man den Menschen etwas versprechen – *fake it till you make it*. Die Gegenseite tut ja auch nichts anderes, nur viel ungenierter.\n&gt;\n&gt; Vielleicht ist es, so besehen, gar nicht schlecht, beim melancholischen Grübeln zum Geräusch der tickenden *Doomsday Clock* etwas zynisch geworden zu sein. Den Weltuntergang zu verhindern, einfach so als Selbstzweck, zieht nicht? Na, dann bindet man in drei Teufels Namen eben auch noch eine rosa Schleife darum.',
 'This is happening *because of climate change* and those regulations on farmers are trying to *slow climate change*. \n\nWhat about that is hard to understand?',
 'No :)\n\nChina, Russia, Corporations, Climate Change, useable land, feeding 7 billion people, getting food to 7 billion people, the chemicals we farm with production, use, and sale.\n\nI could keep going on but i have no further inclination to speak with you. Have a great day.',
 'It\'s amazing how much wrong someone can pack into one paragraph.\n\nThere is a huge amount of uranium available from multiple suppliers, it can also be filtered economically from seawater because it\'s so energy-dense, and there are other materials which can be used, including thorium. There is effectively zero "scarcity" of uranium and there won\'t be for many decades, if ever. It is *not* running out.\n\nClimate change hasn\'t shutdown "most" of the reactors in Europe : it\'s reduced the output of 4-5 reactors in France which are waiting for the rivers to fill up again, and the remaining reactors in France are in maintenance and will be back online by the end of the year. [Here\'s the graph](https://i.imgur.com/QvnWdgp.png) of their return to service.\n\nI\'ve just counted : literally every sentence you uttered was wrong. Extraordinary.',
 "Its not science to me. For me, its you that is the Flat-Earther. Nature responds to change. I doubt science can predict this. For example, it may rain more due to warming.\n\nThe above said and returning to the OP, Buddhism teaches the social world arises from ignorance &amp; craving (SN 12.44); therefore, as the population of ignorant &amp; craving people grow, we should prepare for the worst. In other words, just let it go.\n\nTo conclude, I correctly advised if you believe in climate change, you should discipline yourself by living a sustainable life. This disciples is a step on the spiritual path to freedom from anxiety.\n\nBut if you are a hysterical untrained undisciplined young person, addicted to entertainment, addicted to TV, addicted to NetFlix, addicted to your I--Phone and Internet, addicted to wearing make-up, etc, then you really won't have any coping skills when your beloved mass-media &amp; Greta keep telling you there is a climate emergency.\n\nThe proof is in the pudding buddy. Are you another upper-middle class Climate Warrior on large income, with 4WD, flying around the world on nature holidays, etc,?\n\nIn Buddhism, we overcome anxiety by practicing the Path, which includes Renunciation of sensual/material indulgences as its second factor.\n\nThat's it. Bye bye.",
 "We poor and middle class members are the ones voting to ignore the climate change, so kinda deserved. The rich are a tiny minority in a democracy, and lobbyism/corruption doesn't change that.",
 "Antinatalism - Explosive population growth in the last century. Humanity in ever-burgeoning numbers are destroying the globe's ecosystems and the trajectory of human-induced climate change is ultimate annihilation.\n\nHowever, it seems to be morally corrupt to even raise a question about what unrestricted procreation is doing to our planet. Antinatalism as a philosophy is off the table, a non-starter.\n\nBut what would our world be like now if the population were 1.6B as it was in 1900, instead of almost 8B as it is now?",
 "Even assuming climate change is real and 100% human caused that's not reason to grind civilization to a hault and implement all of this crazy statist taxation schemes.",
 "&gt;I've heard suggestions that nuclear winter would be a way to shut down global warming.\n\nThe problem is, CO2 lasts for at least 1200 years, not to mention all the other gasses, and the burning of hundreds of urban targets and billions of bodies will only add more gasses into the atmosphere. The thing about nuclear or volcanic winters that causes the cooling effect is the soot or ash particulates, and these drop back down to the ground pretty quickly, say within 10 years.\n\nSo the actual effect of nuclear winter will be a massive and sharp increase in emissions with a very short term hiatus and then the newly added emissions will send climate change in overdrive!",
 'See, it\'s like this (or so I\'ve learned from the electric car and climate change shills):\n\nElectric cars are "green" and so are lithium batteries (bec they\'re used in "green" electric cars); therefore, carbon emission during "green" lithium mining is, by default "green".\n\nTa da!\n\nEdit: grammar',
 'Is it weird that even though south park is supposed to be the more political witty show....freaking family guy of all shows may have the better takes?\n\nSouth park spent a decade denying climate change, family guy was all in on pro choice episodes and such',
 "&gt; Couldn’t action on climate change be how god prevents us from destroying the planet?\n\nThen he probably shouldn't have opposed action on climate change.",
 "The following submission statement was provided by /u/-AMARYANA-:\n\n---\n\nSS: Citizens around the world can sense that something is very wrong, and it looks like the economy may be headed for a serious recession in the near term. The operation of our economy requires energy of the correct type and the right quantity. The International Energy Agency and politicians around the world have recommended a transition to the use of renewables to try to prevent climate change for quite a few years.\n\n---\n\n Please reply to OP's comment here: https://old.reddit.com/r/Futurology/comments/x2nyfg/the_worlds_energy_problem_is_far_worse_than_were/imkjz23/",
 'Esta personagem - o Dr. S. Matthew Liao - também acha boa ideia transformar os nossos olhos em olhos de gato para consumirmos menos eletricidade à noite, e tomar comprimidos que nos fazem ter pena do planeta. [Fonte](https://www.marketplace.org/2016/12/15/five-ways-bioethicist-wants-change-our-bodies-fight-climate-change/)',
 'Climate policy changes have reduced projected warming from &gt;4c to ~3c by the end of century\n\n\nhttps://nitter.kavin.rocks/hausfath/status/1511018638735601671#m\n\n\nhttps://climateactiontracker.org/\n\n\n&gt;ending 99.9% of humanity\n\n\nNo reputable scientist predicts that even under worst-case scenario(rcp8.5), not to mention the current one(rcp4.5)\n\n\nhttps://climatefeedback.org/claimreview/prediction-extinction-rebellion-climate-change-will-kill-6-billion-people-unsupported-roger-hallam-bbc/\n\n\nhttps://www.carbonbrief.org/in-depth-qa-the-ipccs-sixth-assessment-on-how-climate-change-impacts-the-world/',
 "I always suspected this. When the meteorite hit that wiped out the dinosaurs, the climate didn't change 'slowly'.\n\nOn a geological timescale, we are a literal meteorite.",
 'Do you think you need to see pollution for it to be pollution? What causes climate change?',
 "Which lies are you referring to on the left?  I can't think of any prevalent ones.  Please don't make me dig out a link to the 2020 GOP-chaired SIC report on Russian election interference.\n\nClimate change hoax, plandemic, stolen election, etc.  I know not every GOPer thinks these things, but a frighteningly large percentage do, and the mainstream right is careful not to alienate those people.  You literally have to garner support of stupid conspiracy theorists to win a GOP primary these days.  If I were a reasonable conservative I'd probably switch parties and vote for center right dems in primaries.  Better that than morons like Boebert.",
 'Climate change.   Oops...too late.',
 'I actually can see this\n\nSeth mcfarlane is very clear which sides he is on, he is a registered democrat who has made pro abortion chapters and such\n\nMatt stone and trey parker are more lolbertarians who denied climate change as far as 2006 and literally  made the meme that became emblematic of enlightened centrism for the 2004 election',
 "Elon is right. But it'll be due to climate change and starvation.",
 "Hopefully be taken care of by living in an actual community. I suspect that community is bound to re-emerge at some point in the near future, if only in response to drastic climate change, the collapse of globalization, and the escalating cost of everything. \n\nI mean, or just die having upheld our current notions of complete atomization, alienation, and separation from human community and the larger community of life. That's pretty possible too.",
 "Bruh. BRICS ain't going to do shit. Climate change is going to SERIOUSLY start to bite pretty soon and there's not enough cheap energy left in the world to project power across the globe. Can't gun for world domination when you have a revolution at home.",
 "&gt;I'm not particularly concerned about government employees actively grooming children to convince them to mutilate themselves, no\n\nOk, then I guess I don't really feel the need to convince you. \n\n&gt;climate change and greenhouse gas emissions\n\nthat's literally just a smokescreen for the malthusians.",
 "I always love seeing folks' passion projects like this! And this one is certainly in a league all its own when it comes to effort and detail.\n\nThere are some routings I wouldn't necessarily have chosen, though I see why they were, and some I would add, but overall it's a solid pitch. I haven't read through the report yet, so no idea on great/good/bad/ugly there, but a quick scroll through looks very well done (as is to be expected from a professional).\n\nOne thing I'll say that I don't like, and this is mostly angled at some of the comments on the Urbanize page, is that I always *hate* pitching this kind of proposal for wider regional rail *against* other modes of transit. In this case heavy and light rail.\n\nThere's more than enough pent up demand for transit-oriented (and walkable and bikeable and dense and etc.) development, and necessity to meet the challenge of climate change, to justify a wide, combined network, something that the paper seems to recognize with infill stations, (modest) service expansions for existing heavy rail, and the integration of light rail / streetcar. I've always preferred a more expansive vision (See: https://www.getonboardgeorgia.org/vision), but I don't disparage someone from keeping their efforts focused. What I do disparage are people who say 'do THIS instead of THIS OTHER THING!'. Please. Just stop.",
 'https://www.weforum.org/agenda/2021/07/why-we-need-to-give-insects-the-role-they-deserve-in-our-food-systems/\n\nhttps://www.weforum.org/agenda/2022/02/how-insects-positively-impact-climate-change/\n\nhttps://www.weforum.org/agenda/2018/07/good-grub-why-we-might-be-eating-insects-soon/\n\nhttps://www.weforum.org/videos/21336-europe-has-approved-insects-for-human-food-uplink',
 'Someone said there was a climate change sign somewhere',
 "Fly all over for useless conferences. It's certainly making climate change better",
 'At least California cares about doing something about climate change and protecting the planet unlike other doltish science-denying states',
 "As a wise sage said: They're doing their best to collect as much data as possible, but it's really difficult at the moment because all the mice seem to keep dying from climate change before they can make definitive observations.",
 '&gt;most of that money went to climate change projects\n\nNo it didnt.  Go ACTUALLY look at what\'s in it. Here\'s to help get you started.\n\n[https://www.whitehouse.gov/briefing-room/statements-releases/2022/08/19/fact-sheet-the-inflation-reduction-act-supports-workers-and-families/#:\\~:text=The%20Inflation%20Reduction%20Act%20will%20protect%20Medicare%20recipients%20from%20catastrophic,for%20the%20first%20time%20ever](https://www.whitehouse.gov/briefing-room/statements-releases/2022/08/19/fact-sheet-the-inflation-reduction-act-supports-workers-and-families/#:~:text=The%20Inflation%20Reduction%20Act%20will%20protect%20Medicare%20recipients%20from%20catastrophic,for%20the%20first%20time%20ever).\n\nWe live in a country where businesses and oligarchs have more power and influence over policy than the people or politicians do.  The government is restricted from what they are able to do to businesses.  They\'re in control.  Nothing about that is "moderate".  Compare our capitalism to any other country, and you\'ll see that our capitalism is far more broken than any other country. \n\nI\'d love to hear your explanation why record inflation is happening at the exact same time that companies are posting the highest profits they\'ve had in many decades',
 'Wonder why……………….. is the current explanstion climate change?',
 "The Daily Wire doesn't accept reality.\n\nParticularly with regards to climate change.",
 'Ethiopia’s Corbetti geothermal plant have plans for 1000MWe. 100 trillion in stranded assets if we get off fossil fuel and address climate change. \nInfinite growth paradigm, finite resource replenishment. No regard for biosphere inhabitability, soley infinite cyclical consumption and accruement of capital/profit. More debt in existence than actual currency.',
 'Well, in the context of the story that scene is definitely a commentary on the role of women in medieval societies and their lack of autonomy, choice and subjugation by the patriarchy. Since Martin’s work often time contains allegories to modern issues like climate change, it wouldn’t be far fetched to connect this scene of a woman’s reproductive choices being violently taken from her to the modern issues of abortion and feminism.',
 "Hot Summer = Proof of Climate change \nCold Winter  = Local weather doesn't matter",
 'The billionaires need to give up their lavish lifestyle that’s what’s really affecting climate change.  Done believe any propaganda about the common man needing to change anything it’s all a bunch of BS',
 "Not having kids is not a solution in any ecologists plan to tackle world hunger or climate change. It's global policies and investment in renewable energy sources as well as environmental stewardship education.\n\nYou know what those need to work? Future generations.",
 'Future where the US receives all of the climate change and Australia gets scrambled.',
 'It started with the vaccine and it’s massive. The only explanation is either climate change, which is the story the media is going with, or the vaccine. Anyone with any neural synapses knows what’s up. \n\nSeriously try and keep up.',
 'But at least they probably would join protests for awareness about climate change , right…effing pigs',
 "Because it's forking natural, look at history, there's been atleast three ice ages then everything melts then it gets hot as fork and most species die and a few survive then everything starts freezing and ice comes back new ice age then it melts and repeat\n\nEveryone doesn't do research before posting or saying things 40 years ago in the 80's they were saying prepare for the ice age and now they're preaching climate change, it's real and natural and inevitable because you can't beat nature it always finds a way and it's just a way for people to make money",
 'you can read a bit more about [jellyfish blooms](https://www.businessinsider.com/jellyfish-thriving-climate-change-warm-oceans-2019-10?amp) here!',
 'How do you feel about the impacts of climate change on future generations?',
 'No it’s still an issue but you can’t attribute all ice melt or shearing to climate change. It’s a normal process. Orcas breaking ice while hunting is not a new phenomenon.',
 'Climate change is "political" in the same way that covid or gay people existing is "political"',
 'Exactly. The profit motive rarely incentivises the right thing, for example climate change.',
 "There was a taboo about Nuclear maybe in no small part because of misinformation for a long time because of the potential for fallout and such. Reading into it though the fact that there's been a significant amount of improvements means there really needs to be a reevaluation of the issue expecially since Climate Change has become a much more serious issue in recent years and because while Nuclear has it's risks it doesnt contribute to Climate Change since it doesnt produce CO2.",
 'We’ve all been staying indoors for decades here in Phoenix in the summer. Just like people in North east do when it’s winter time. We love the summer. I’m all about the pool season and camping is 1.5 hours north in the cool pines. People that don’t live here always talk about how terrible and doomed we all are, but you need to experience it to know. I’ve lived here almost 40 years. I’ve seen the climate change. It’s there, but if the southwest is truly doomed in 30 years, so is most the rest of this country (and world). I just don’t play into this forecasting for the future stuff.',
 'It is a climate change',
 "„ Concerns about climate change or earthquakes, when mentioned, were framed as positives rather than negatives.[16] Sörgel's 1938 book Die Drei Grossen A has a quote from Hitler on the flyleaf to demonstrate that the concept was consistent with Nazi ideology.“\n\nBig surprise /s",
 'With Climate Change staring us in the face, I\'m worried that we may one day be calling these the "good times".',
 'Praying is stupid, we need to stop climate change.',
 'I think it’s an attempt to make climate change scarier to the layperson by blowing an insignificant statistic out of proportion. All the while increase in the heart attack rates are obviously caused by the obesity epidemic.',
 'It can’t be a prequel anymore since *FAM* has solved climate change.',
 'And people are gobsmacked that we can’t manage meaningful climate change goals, even moderate ones.',
 'There is one for climate change. It reduces the number of humans.',
 'This thread conflates what I see as two different questions: Why I personally didn\'t have children vs. why I now think no one should have children.\n\nI\'m 68 years old, so my reasons for not having children some 40 years ago were far more personal than ideological. I just didn\'t want them enough to be worth the incredible effort and expense of raising them. \n\nOver the years, however, as population continues to climb and climate change accelerates, I\'m convinced that having children is just not a good idea for anybody in the developed world. Every individual in the "first world" puts too much pressure on resource and energy use, not to mention that the world they\'re going to inherit will be collapsing around them.\n\nWhen I was born there were 2.5 billion people on Earth. We\'re approaching 8 billion people now. Enough.',
 "&gt; in the big picture of 100's of millions of years.\n\nI don't know how many 100's you're talking about, but modern humans have only been around for roughly 1/3 of a million years.  \n\nYes the climate has changed greatly over the lifecycle of the earth, and just about 65 million years ago it was uninhabitable to almost all life.  This is irrelevant in any conversation about the modern day effects of climate change.\n\n\nIn other words, wtf are you talking about?",
 'Can confirm the lemon juice trick works. I did it myself last year when it was going on. \n\nSide note: I wouldn’t be surprised if this was a reoccurring thing each fall in Btown going forward. Algal blooms are a likely source of both compounds, and with those increasing in frequency due to climate change, I doubt we’re out of the woods after this year.',
 "The following submission statement was provided by /u/IHathReturned:\n\n---\n\n&gt;In October, the Intergovernmental Panel on Climate Change (IPCC) released a report setting out why we must stop global warming at 1.5\u2009°C above pre-industrial levels, and how to do so1. It is grim reading. If the planet warms by 2\u2009°C — the widely touted temperature limit in the 2015 Paris climate agreement — twice as many people will face water scarcity than if warming is limited to 1.5\u2009°C. That extra warming will also expose more than 1.5 billion people to deadly heat extremes, and hundreds of millions of individuals to vector-borne diseases such as malaria, among other harms.\n\n&gt;But the latest IPCC special report underplays another alarming fact: global warming is accelerating. Three trends — rising emissions, declining air pollution and natural climate cycles — will combine over the next 20 years to make climate change faster and more furious than anticipated. In our view, there’s a good chance that we could breach the 1.5\u2009°C level by 2030, not by 2040 as projected in the special report (see ‘Accelerated warming’). The climate-modelling community has not grappled enough with the rapid changes that policymakers care most about, preferring to focus on longer-term trends and equilibria.\n\nThe IPCC is conservative in its predictions. This paper, published in 2018, details the reasons to expect global warming to occur faster than expected.\n\nThis is related to collapse because global warming on its own can destroy the food system through floods, droughts impacting food production.\n\n---\n\n Please reply to OP's comment here: https://old.reddit.com/r/collapse/comments/x2n19l/global_warming_will_happen_faster_than_we_think/imkeimo/",
 "In the South, schools refuse to teach things like CRT and climate change. For the sole purpose of dumbing students down, in hopes they'll grow up Republican.",
 '&gt; Democrats actually consult with experts in the field before they craft legislation.\n\nSo what expert determines that a rifle with the ability to attach a knife is magically more deadly than the exact same rifle without that ability to attach a knife because a bayonet lug is a feature that creates an assault weapon.  But to my knowledge there is not a problem of people being stabbed by rifle mounted knives.\n\nAlso I would love to find the expert that said "A heat shield exists purely to prevent the shooter from burning their hand on a hot barrel.  Obviously by banning this safety feature we will save lives."\n\nAlso where are these experts that say a rifle with a permanently collapsed stock is ok and one with the same stock permanently extended is ok but the same rifle with the same stock that can simply move between fully collapsed and fully extended, when the rifle in either configuration is still legal, somehow now is a weapon of war only suitable for armies and local police...\n\nGo ahead find me these experts.  They probably came from the same schools that create the "experts" republicans use to deny climate change\n\n&gt;Also, Presidents don\'t write laws.\n\nOh ok so then every time Trump said something dumb and everyone pointed out his ignorance all Republicans had to do was say he doesn\'t write the laws and then magically his ignorance is entirely unimportant?',
 "I would be incredibly surprised if any vehicle company that does business in the US and doesn't exclusively make EV's hasn't thrown plenty of lobby money against climate change laws.",
 'it’s causing environmental factors that can cause heart attacks, such as wildfires, droughts, and heatwaves. Makes sense to me 🤷\u200d♂️ doesn’t mean it’s the main cause, but it’s just more reason to be concerned about climate change. P',
 "Unexpected blips will happen more and more with climate change. Are we just going to have a un ending homeless population? That's not the kind of country i want to live in",
 'All climate change stuff.',
 "&gt;How could we not think that this was unsustainable?\n\nThe same way we all though climate change wouldn't get us, or that the economy would just keep going. When things are good, people don't *want* to think",
 'The same SF who have no policy at all on climate change? Its almost as if they are short term populists with no idea how to actually govern. All the young folk dying for a SF government should take a good look at the actual implications of their policy positions (or lack thereof).',
 'We are beyond the point of a competency test for voting,  Look all I\'m saying is the people who believe the Earth is flat and the people who look at climate change and they don\'t think "oh yeah that\'s just grade 10  Newtonian physics"  Not only do they get a vote but their vote is equal everybody else\'s.\n\nEdit addition : we have literally watched the federal education budget fall flat on it\'s face since the early 70\'s and subsequently the military\'s discretionary budget increase and no ome bats an eye',
 "Climate change? That is code for UN Commissars telling us Americans what temperature it is going to be in our outdoors. I say let the world warm up. See what Boutros Boutros Ghali Ghali thinks about that. We'll grow oranges in Alaska.",
 '*oof*… that chart on different countries’ political ideologies on climate change. The GOP does laps around other right wing parties. Literally every other countries’ conservatives (besides Israel, who still are much higher) are over twice as likely to believe in climate change',
 'Do you even know what climate change is',
 "It's very telling how you link to the appendix, instead of the fulltext. \n\nhttps://www.thelancet.com/journals/lanplh/article/PIIS2542-5196(21)00251-5/fulltext\n\nOtherwise people might have a chance to read the authors' actual conclusions:\n\n&gt;Compared with the cost of current diets, the healthy and sustainable dietary patterns were, depending on the pattern, up to 22–34% lower in cost in upper-middle-income to high-income countries on average (when considering statistical means), but at least 18–29% more expensive in lower-middle-income to low-income countries. Reductions in food waste, a favourable socioeconomic development scenario, and a fuller cost accounting that included the diet-related costs of climate change and health care in the cost of diets increased the affordability of the dietary patterns in our future projections. When these measures were combined, the healthy and sustainable dietary patterns were up to 25–29% lower in cost in low-income to lower-middle-income countries, and up to 37% lower in cost on average, for the year 2050. Variants of vegetarian and **vegan dietary patterns were generally most affordable**, and pescatarian diets were least affordable.",
 'You might wanna check your legal obligation if you are renting your flat. \nBelow 16 degrees, molds and other stuff could happen and damage the appartement. Which I believe you could be accountable for. \nBest idea would be to try not going below 16.\nBut then… will we have less than that this winter seeing the climate change',
 'But yeah, you should listen to the people telling you climate change is faked by China',
 "In 2010 the City budget for all property taxes was $1B.  Now is it just over $2B - that's a increase of over 100%.  Works out to compound increase of 6% per year.  That's way more than inflation over that period of time.  The MVA system hides it well.\n\nOperating costs of the City have more than doubled and the pace of increase is on track to double the operating costs again by at least that much in the next decade.  Throw in climate change spending and the 19+ new communities being built out and the picture for property taxes is very ugly.",
 'all of our opinions probably don’t even really matter, because we’re in a mass extinction event and soon enough we’ll be fighting over water thanks to climate change, sooner than we think!',
 'Climate change should not be a partisan issue',
 "I don't have anger at politicians because of climate change, I'm angry that politicians inflict harsh economic conditions on their own people in the name of climate change.  Save the planet, freeze and starve the people.  Save the planet by charging us twice as much for gas.  Save the planet by making sure farmers can't grow food.  Save the planet by committing to irrational co2 emission standards while allowing China to run rampant with them - hello, we all share the same planet dumbass! Save the planet by blithely telling us to get electric vehicles when many can't even afford rent and groceries. Save the planet by telling us to eat bugs. Speaking for myself I'm not angry because of climate change anxiety, I'm angry over what's being done to us under the guise of climate change.  And you're one of the worst, Trudeau.",
 'But that Jim Inhofe held a snowball in the chambers of congress! Climate change is NOT real. /S.',
 'And you can bet most of them are middle class lovvies living off mummy and daddy who protest for climate change.',
 'Bernie considers climate crisis the most essential change to national security. He opposed the establishment of the Dakota Access Pipeline on the grounds that it will have a huge effect on the climate.\n___\nBernie Bot -Bernie_Bot_02',
 "People still are not believing that climate change is real.  I hope you examine scientific evidence as to what climate change is and how it will impact you.  You're serving the disinformation campaigns from Oil industry.",
 'More than likely the river backed into the spring due to flooding. I was at Wakulla recently and it was extremely clear. That being said, climate change leads to more frequent brown outs of springs and can alter spring chemistry (through an increase of storm frequency and intensity). Source: been there, and I’m a hydrologist\n\nEdited to add when the water table is low, water from other conduits will be pulled in the direction (high to low hydraulic head) of the spring and can cause more tannic water to come out of the spring, but this will vary depending on aquifer levels. We see that with some of our springs here near Gainesville that sit at a higher potentiometric elevation.',
 "And also it's making it seem like electric vehicles are the problem when the fucking heat waves and climate change are the problems.",
 "did you see the 4 or 5 pages of imgur l linked.. \n\n individual links \n\n https://i.imgur.com/5PUpk6p.jpg\n\n https://i.imgur.com/eSnAPR0.jpg\n\n https://i.imgur.com/Ap8qG67.jpg\n\n https://i.imgur.com/NdBbVvF.jpg\n\n  the EGR solenoid is a bleed type... when not engaged it bleeds air.. so the vacuum supplied is always leaking..  the computer pulses the EGR solenoid to change the amount of opening of the EGR valve..  so tee'd into the red hose to the EGR solenoid you probably will get less vacuum..\n\n  and switching to defrost when driving uphill is SCREAMING at me VACUUM LEAK some place..  i have been at this for 43 years as a professional... i can't tell you where its leaking.. but i tell you you have an issue.. could be between the HVAC vacuum reservoir and the climate control head... \n\n save and print those diagrams...",
 "The inflection point. Energy is at a focal point thanks to sanctions on Russia, and historic global droughts impacting energy production while highlighting climate change. There's been so many crises throughout my short life, but this one's a kick on the arse if there's ever been one.",
 "Yes, we should be altering the path and deploying more clean energy. (I'm not claiming JK is interesting in that, although I believe EM is.) But there's no scenario where a great deal of short-term human suffering is avoided without fossil fuels.\n\nI'm expecting this winter to be really quite bad for much of the world. Maybe the winter after that too? But certainly this winter. Entire industries are shutting down in Germany because they lack affordable electricity and chemical inputs.\n\nAnd that's WITH Germany buying up all the world's LNG. Developing nations can't afford to import it any more. Very destabilizing. I don't think the 2022 impact of climate change is going to stack up to the impact of 2022 energy-crisis / fertilizer-crisis.\n\nMaybe JK doesn't really care about GW, I don't care. Broken-clock being right twice a day. On this point JK regurgitating EM is regurgitating true statements.",
 'Ooh I\'ve got some:\n\n* Letting motorcycles filter between cars saves lives.\n* Daily bathing is a scam concocted by Big Soap. (Seriously, look it up).\n* Women shaving their legs is a scam concocted by Gillette Razors. (Again, look it up).\n* The USA is not a capitalist, democratic state. It is at best an oligopoly, if not a straight up oligarchy.\n* "Socialism" the way Westerners use it is a misnomer. (They inevitably mean "authoritarianism").\n* Climate change is real and humans are accelerating its course. We\'re all about to have some serious egg on our faces in the next couple of decades.\n* News in the US is horseshit, period. Not "left side" or "right side" (although IMO the right is more blatant about not caring that it\'s horseshit). But compared to other countries, US citizens don\'t know fuck all about what\'s going on in the world.',
 '&gt;In October, the Intergovernmental Panel on Climate Change (IPCC) released a report setting out why we must stop global warming at 1.5\u2009°C above pre-industrial levels, and how to do so1. It is grim reading. If the planet warms by 2\u2009°C — the widely touted temperature limit in the 2015 Paris climate agreement — twice as many people will face water scarcity than if warming is limited to 1.5\u2009°C. That extra warming will also expose more than 1.5 billion people to deadly heat extremes, and hundreds of millions of individuals to vector-borne diseases such as malaria, among other harms.\n\n&gt;But the latest IPCC special report underplays another alarming fact: global warming is accelerating. Three trends — rising emissions, declining air pollution and natural climate cycles — will combine over the next 20 years to make climate change faster and more furious than anticipated. In our view, there’s a good chance that we could breach the 1.5\u2009°C level by 2030, not by 2040 as projected in the special report (see ‘Accelerated warming’). The climate-modelling community has not grappled enough with the rapid changes that policymakers care most about, preferring to focus on longer-term trends and equilibria.\n\nThe IPCC is conservative in its predictions. This paper, published in 2018, details the reasons to expect global warming to occur faster than expected.\n\nThis is related to collapse because global warming on its own can destroy the food system through floods, droughts impacting food production.',
 'Climate change',
 "Depending on the sciences you have differents name like the people that don't belive in climate change are climato septic the people that don't belive the earth is round are flat earthers the ones that don't like needles but are too afraid to say it are anti vaxxers etc people that don't belive in evolution and the concepts that go with it like baze are creationist",
 "Has there ever existed a time in the history of the world that one place wasn't extremely hot, anther place wasn't flooded, another wasn't experiencing a drought, when ice wasn't melting somewhere and where there were no fires?   \nThe climate has always been changing and that will never change. However your ability to see different parts of the world in real time has certainly changed over the last 20 years. And that's what's got you confused about what's normal and what isn't. What was distal is now proximal, and confusing the heck out of your brain when it comes existential risk. Your world is perfectly ok. Yes weather is changing, always has, and that has always been a threat to humans living in those areas where it changes inhospitably. \n\nWhile you may be scared of the weather, is it not scarier to believe that some people think they can control it? Can you control the weather?",
 'Nah, honestly as much as I don\'t want kids for reasons outside of my health issues (climate change, global issues, my mental health, etc) I\'d take being able bodied and having a child over having a progressive incurable genetic disorder that leaves me in constant pain and is making my life super shitty and will only make it shittier as time goes on. Albeit this is a better scenario than if I had let my exe\'s family bully me into a kid, as then I\'d be a disbled single parent who passed on a genetic disorder. But "best possible scenario"? Far from.',
 "wasn't that about climate change not covid?",
 "Put all stores on stilts, that way you can't drive a car into them. \n\nWould be great for avoiding flooding, you know, climate change and all.",
 '/u/PerBolund \n\nFossila bränslen kan inte helt ersättas av förnyelsebart förrän på 2040-talet enligt [Energimyndigheten](https://www.energimyndigheten.se/nyhetsarkiv/2019/sa-kan-100-procent-fornybar-elproduktion-se-ut/). Det betyder att vi har en övergångsperiod på minst 18 år. Vad gör vi under dessa 18 år?\n\nMP säger förnyelsebart, trots att det finns inte på bordet förrän på 2040-talet. Det svaret är alltså inte acceptabelt. Fram till dess har vi endast 2 alternativ:  \n\n* Använd minimal mängd kärnkraft, och maximal mängd fossil energi\n* Använd maximal mängd kärnkraft, och minimal mängd fossil energi\n\nNi tycker alltså att det är bättre för miljön att bränna gas och kol i 20 år, än att använda kärnkraft. Anmärkningsvärt dåligt påläst. [Som tur är så är medelsvensson bättre påläst](https://www.analys.se/opinion/) .\n\nJag uppmuntrar alla att titta på den här dokumentären, "[Do we need nuclear energy to stop climate change?](https://www.youtube.com/watch?v=EhAemz1v7dQ)" av Kurzgesagt. [Kan man lita på Kurzgesagt?](https://www.youtube.com/watch?v=JtUAAXe_0VI)',
 'Well my point is, it’s not that unbelievable if it were real. This seems very much on par with things.\n\nMake money in a way that sells you and your property as ad space instead of meaningful work✅\n\nThe ad is for a objectively evil company✅\n\nThe ad promotes selling water which said company argued that water isn’t a human right✅\n\nMaking it a bumper sticker for a car, my car, that did help to contribute in some small way to climate change making water scarcity a thing✅\n\nEtc, when I saw this it definitely felt extremely fucked up.',
 'Pollution is also in the air, which is causing the atmosphere to heat up along with weather patterns to change. Admittedly, a lot of that lower water probably has to do with over use, but climate change from pollution is a contributing factor',
 'They tell me its because of climate change.',
 'Is there a verse on climate change I’m unaware of?',
 "&gt; climate change makes food next to impossible to grow in most places\n\nuh... that's not a real climate change outcome at all.",
 'While I’m glad that hurricane season is quiet and hope that it remains that way, I’m concerned that if it remains quiet, republicans before the midterms can say that the climate scientist’s wrong prediction of an active season is “proof” that climate change is hysteria pushed by democrats.\n\nEdit: It also doesn’t help that the US has been relatively spared by severe climate disasters this year compared to the rest of the world.',
 'Quoting a conservative think tank that denies climate change seems about right for someone with opinions as deluded as yours',
 'Yes we have. Before climate change it was "global warming", before that it was "acid rain", before that it was "next ice age", before that it was "the hole in the ozone layer".\n\nWe\'ve been dealing with a forever collapsing climate for centuries. London had a mini ice age that lasted for a few years and then had a few years of extra hot weather. We have actual records of these things happening.\n\nClimate collapse is something we\'ve been flip flopping on for decades.',
 'SS: you can how Elon Musk is really scared of climate change. While he praises the number one emitter and flys private jets',
 "What about climate change and the eventual climate collapse? We haven't dealt with that before.",
 'Honestly, at this point, climate change deniers deserve hell. How could you? I wasn’t an avid climate change believer in that I knew it exist since I was like in Middle school learning about it in school. Until each year, I’d see the figures people claimed and warned about years ago, actually exceeding what was expected. Each year summer gets hotter to dangerous degrees. I live in the northeast and literally people can’t live. People are sweating 24/7 in dangerous extends, the subways are full of sweats, and i don’t think we can bear the uncomfortableness much more. This isn’t about ‘Liberals’, this is science. Even religious scholars, atleast Christians anywhere but America, Muslims, Jews all believe the value of taking care of our planet and the destruction society is causing it cus we are actually facing the reality of it. Antarctica is in shit conditions, wild fires keep getting more prominent, ice glaziers melting and other HORRIBLE realities of the consequences of not acknowledging the shit condition our planet is in. Do better.',
 'Bernie considers climate crisis the most timely change to national security. He opposed the building of the Dakota Access Pipeline on the grounds that it will have a huge effect on the climate.\n___\nBernie Bot -Bernie_Bot_03',
 'Woke capitalism is ESG.  It stands for environment, social, governance.  In short, they invest based on a firms stated values and beliefs (do they like puppies and rainbows), and not their financial performance\n\nESG funds are expensive and don’t outperform the SP index.  They also avoid fossil fuels because “climate change bad”\n\nIt’s wokeness injecting itself into the financial world',
 'Climate change has its effects everywhere',
 '[The E-bike Is a Monstrosity](https://www.theatlantic.com/technology/archive/2022/08/ebike-electric-bikes-climate-change-exercise/671305/)\n\n\nMost of this article is just complaining about the aesthetics of e-bikes, but the highlight of this whole thing is when the author says\n\n\n&gt;\tWalking the streets of New York City, it now feels just as likely that you might get mowed down by an e-bike as a taxicab. \n\n\nAs if there is no difference between getting hit by a 65 lb e-bike and a 2 ton taxicab.',
 'https://www.esquire.com/entertainment/tv/a23863674/george-rr-martin-game-of-thrones-politics-trump-climate-change/',
 'Thats good. Meat should be more of a luxury to relieve farmers and to countrr the (dunno english) mass animal industry, which is the biggest problem of climate change',
 "Here in CA it costs less than half to go 300 mi on electricity. You will find cherry picked posts, but generally it's about half. \n\nThis means you have to justify the EV price with something else, gas alone will not make up the difference. Climate change, performance, time savings, etc. TOC is generally better with an EV of the same performance and luxury level.",
 'our climate change conscious king 😍',
 'I guess this would accelerate climate change and exterminate the human race even sooner. 🤔',
 'Ah yes because the eleventh commandment was "thou shall not contribute to climate change"',
 "It's not economical. Uranium is a finite resource that's running out faster than oil. When it gets scarcer, the price will go up and it's already an expensive form of power. Because it's running out, it barely makes any difference worth mentioning to climate change but climate change already has most reactors in the EU shutdown, they can't be cooled properly because of drought.\n\nNuclear is not the answer lobbyists would have you believe.",
 'Climate Change. It’s either seen as a hoax or an inevitable issue that ”we need to address soon”. I’m pretty sure the point of no return for our planet has passed by already, and we need to come up with solutions fast or else the EARTH IS GOING TO BECOME INHOSPITABLE.',
 'Starts getting cooler in middle of October usually\n\nBut who knows, climate change and all now',
 'The actual impact humans have on climate change.',
 "Why do you think that?\n\nI've never personally met a Christian who doesn't believe in climate change. I only know that exist because they pop up on documentaries 😂",
 'The answer to climate change is space exploration, change my mind.',
 'Is this the same Telegraph that on the front page today has an opinion piece stating that Boris will never be forgiven for Net Zero.\n\nHowever, against inflation we are seeing similar positions coming out in the US too. 17 US states have adopted anti ESG legislation for instance. Climate change is getting strong "don\'t look up" vibes.',
 'One good source to read (The Xu paper I haven\'t read yet but will... really need to get to work ha) \n\nHowever, I\'d lose the institute for energy research piece in the future both for your own sake and trying to convince others. IER is a known fossil fuel and climate change denial lobby group initially founded by Koch but mostly funded by Exxon and the API (petriolium trade group). No matter if the piece is solid it will not be taken seriously by most environmentalist.  I\'ve found many of their pieces in the past have to "misinterpreted" the referenced data and sources or simply reference pieces from sister lobbying groups in a petroleum/climate denial misinformation self referential circle.',
 "Corporations are working hard to make common people believe climate change is a hoax because that's how they make the most money.\n\nFollow the money.",
 "c'est effectivement le choix de la décroissance plutôt que celui de la récession subie qui est le meilleur. mais ce n'est pas la direction qu'on prends.\n\nPar contre ce n'est pas le moyen qui est sûr en lui même, c'est la politique qu'on lui applique qui fait sa sûreté, or [cette sûreté ne pourra probablement plus être garantie dans un avenir proche en conséquence des choix politiques d'après le patron de l'ASN](https://www.lemonde.fr/economie/article/2022/01/19/la-poursuite-de-fonctionnement-des-reacteurs-nucleaires-d-edf-ne-doit-pas-etre-la-variable-d-ajustement-de-la-politique-energetique-francaise_6110102_3234.html)\n\nQuand à la stabilité c'est un point qui n'est pas garanti à l'avenir car on ne sait si le nucléaire sera adapté face au dérèglement climatique, comme mis en avant dans le papier publié dans energy policy: [Climate change, nuclear power, and the adaptation–mitigation dilemma](https://www.sciencedirect.com/science/article/abs/pii/S0301421510007329).\n\nIl y a d'autres choix qui pourraient (devraient?) être fait qui nous éviterait de construire plus de capacité production. Par exemple au lieu du non sens de la voiture électrique, on pourrait avoir des vélomobiles électriques. Si on remplacait la totalité du parc automobile par des vélomobiles électriques ça pourrait largement être alimenté par un fraction de l'éolien existant: https://www.lowtechmagazine.com/les-v%C3%A9lomobiles-electriques-aussi-rapides-et-confortables-que-les-automobiles-mais-80-fois-plus-efficaces.html\n\nEt il n'y a pas besoin de chercher bien loin pour trouver des postes d'économies similaires qui permettraient de réduire notre parc électrogène.\n\nOn peut facilement arguer qu'on consommera toujours tout ce qu'on produit et que face à l'urgence climatique il serait plus judicieux de réduire la capacité de production pour réduire la consommation énergétique.",
 'It\'s fascinating to sit back and observe how impressionable folks can be in the absence of rigorous scrutiny of the body of science available in regards to "man-made" climate (cooling, warming, and now change) impacts.  Further, the elitists that set out to manipulate your opinions employ class politics to divide out society; pitting one group against another.  Unfortunately, the gullible fall for it every time. A simple review of catastrophe predictions during the 20th century by leading scientists of the day, will display a dizzying array of misguided, unrealized, poorly researched, and even poorer conclusions about the impending doom of the world.  Good luck making any sense of it all; you would have better luck with the local shaman or palm reader.\n\nIf you take the time to consider all the data, conclusions, and the most obvious contributors, you will quickly come to the conclusion it\'s the biggest hoax of the 21st century and the public is taking it hook, line, and sinker.\n\nIn spite of the skewed characterizations of white conservatives the only real thing you can conclude from the survey, is the fact they tend to be more educated, skeptical, and less prone to hysteria.  The balance of the conservative wing of the Republican party will exhibit the same attributes no matter their sex, ethnicity or age, hence the "conservative" moniker.  In other words, it\'s not a "white devil" thing...',
 'Mm not necessarily but I have noticed an uptick in studies about climate change.',
 'Then what problems do you perceive the government capable of solving? Since apparently climate change, economic depression, or mental health crises are not within their preview? \n\nIm seriously asking, since i dont think you are being realistic, what other entity is well suited to solve these issues facing literally every other nation on earth?\n\nIf you cannot answer why the goverment should not solve these problems, then you are a taxpaying m0ron. They’re collecting taxes either way they might as well solve problems or *govern*',
 'Now it’s climate “death”. Did climate change just not motivate enough people anymore? \n\nWhen we’ve hit all the metrics that you said would instill “climate death” and nothing really happens are we going to get “climate mega death”?\n\nIs it getting hotter and the climate is shifting, yes. Has it been even slightly as bad as we’ve been told it was going to be for the last 30 years, lol no.',
 'Bernie considers climate crisis the most valuable change to national security. He opposed the establishment of the Dakota Access Pipeline on the grounds that it will have a big result on the climate.\n___\nBernie Bot -Bernie_Bot_03',
 "Human-activity-related climate change accelerates and distorts these otherwise normal phenomena. The natural world isn't static and is punctuated by a handful of catastrophic events on geologic scales.",
 'I fucking hate the "it\'s just a mask" justification for keeping mandates.\n\nSomeone could pass a law mandating that everybody clap their hands three times every morning at 10:10 in an effort to stop climate change. It might not stop climate change, but it won\'t make it worse and only takes a second or two, so it\'s not that onerous of a burden, right?',
 'This isn’t just a group with an unpopular/easily proven wrong theory, like climate change is a hoax or trickle down economics is the best way to stimulate an economy. These people are sick. They have serious mental health problems to be able to justify their positions. Honestly, I’d say all Trump supports have a mental health issue, but Qanon especially.',
 "Oceanic climate + particles from coal and wood burning + no global climate change (yet) etc. would mean it wasn't as warm as it is today.",
 'I can’t say with 100% certainty, but perhaps it’s because many American evangelicals lean politically right-wing/conservative, electric cars are probably seen as giving in to “the green climate change agenda,” and people nowadays mindlessly promote the brain dead memes and epithets of their political tribe.',
 "The following submission statement was provided by /u/radiomatrix:\n\n---\n\nSS: Pakistan's planning minister has said that the recent climate change induced floods have caused at least 10 billion dollars of damage. It was a monsoon on steroids. These floods have submerged a third of pakistan.\n\n---\n\n Please reply to OP's comment here: https://old.reddit.com/r/collapse/comments/x2l3i3/pakistan_floods_disaster_to_cost_more_than_10bn/imk1bsl/",
 "Climate changes naturally, but this climate change isn't natural so precedent doesn't apply",
 'Climate change haha. Do you know where Canada rates on climate change? Let’s tax us more, raise the cost of everything, meanwhile we’re one of the lowest countries polluting. Nothing will come of it, unless other countries follow suit. Which they also haven’t. It’s a pipe dream.',
 'What do you do about climate change, when the voters who keep you in office say "Let it burn"? \n\nThe problem is the people, which is why I am not optimistic about a solution.',
 'As long as he says climate change bad. He can treat women how he wants.',
 'lol...so you aren\'t smart enough to figure out why they have changed it.\n\nHurry up and go put on your mask snowflake.\n\nBTW...if you are so certain "global warming" is caused by humanity then why are you on Reddit? You accessed the site from a fossil fuel product produced in a fossil fuel consuming plant. If you think you\'re the cause then you shouldn\'t even be on here. Or are you just another "climate change" hypocrite who continues to bitch about a made up narrative designed to control you while you use your fossil fuel derivative products on a daily basis?\n\nlol...the cognitive dissonance in "climate change" snowflakes is next level',
 'I want to make sure you understand that the ice sheet is always changing. It’s not a monolith it’s creating new ice and breaking off all the time from prehistory to today. The rate that it’s adding has slowed and the rate that it melts/breaks has increased due to climate change but it’s not all attributed to climate change.',
 'Climate change isn’t caused by leaving trash on the ground.',
 "It's not even a plan, it's a natural progression for society. Notice how we don't live in tribal villages anymore. \n\nOf course climate change will be catastrophic, it already is, doesn't mean combatting it will require descent into dictatorship.",
 'Wonder if it was far right white supremacy or climate change?',
 "I mean, the other option is to buy an EV from a company that hasn't been against Climate Change like Hyundai, Ford, and Tesla.  (GM was also lobbying against Climate Change laws...)",
 'Even outside of climate change, the ice sheets are always in motion and changing during the winter it adds mass and in summer it breaks off and floats. Completely natural.',
 'They only changed it to climate change because morons kept pointing at every cold day or snowstorm in their local area and saying "hur dur what global warming".  Climate change is caused by global warming, it\'s still happening, it\'s still caused by humanity.',
 'A good way to frame climate change for people is that 1 in 1000 year storms become 1 in 100 years or even 1 in 50.',
 'Then the doom of climate change is not real? I am only speculating that if the problems we face are catastrophic then the response will be as well. I am not concerned for me as I don’t think we are quite there yet. One world government is not a conspiracy but a well known plan.',
 'Probably safe to just blame climate change',
 'The biggest chance for human extinction in the next 50 years is nuclear war.  Nothing else is close.  Then maybe catastrophic bio-war.  If the world escapes those two, then the excellent writer John Michael Greer thinks for general economic and climate change reasons, there will be a slow decline for 200 to 300 years, followed by a Dark Ages.',
 "In all fairness Reading is very much known as a young person's festival. It typically falls on GCSE results day or thereabouts so you get a lot of young people. I was recently at Green Man, which for reasons mostly related to travel, is way more middle aged than most festivals, and it wasn't nearly as bad as this. I wouldn't be surprised if the reason Reading is so fucked up is because a lot of young people can't be bothered to travel home with their tents and shit, and those same young people are the people statistically most likely to be concerned about climate change and the environment.",
 'Oh but you weren\'t the one directly attacking me, don\'t worry. \n\nIt\'s the guy who said that the issues with the schools is that they "push leftist agendas like LGBT" and teach stuff about climate change which they for some reason shouldn\'t, etc...\n\nI dismissed it because those are the classic conservative talking points, which I\'m tired of as they\'re always the same thing over and over again.\nI meant the actual issues of US schools (if there are), like , I don\'t know , the things they learn are too basic, or the organization is bad, all sorts of actual issues .\n\nIt\'s rather conservatives who wish what to believe. I say it because they all repeat the same arguments, as if someone or a group started to spread the arguments,  and people would just take it for word.\n\nLike I\'ve noticed that many people deny the fact that gender is a neurochemical phenomenon and not a social construct , something pushed by many conservatives figures like Matt Walsh . \nThat\'s just an example of how those people don\'t even seem to think about the argument and just parrot it around. I know not all conservatives are like that, but a decent amount sure are.',
 "Sounds very small fish you're frying there, given that the alternative will lead to a lot of deaths, wholesale reductions of the rights of the people in the UK, and failures across the board on important action on everything from climate change to trade deals.\n\nAnyone is electable if the people vote for them. Boris Johnson got elected for god's sake.",
 'Evidence that god controls the climate and that humans aren’t wrecking it. Everyone knows about cyclical climate change over the billions of years the earth has been here. That’s not the climate change we are talking about and you know it.',
 'Hahhaa bro I am not a cop. I can promise you that. Shit what do I gota do to prove it just tell me and il show you. I am also from Canada so that might be why.\n\nMy name is more towards the WEF and their agenda of a new world order and the shit that their doing to our countries without a vote from any of us people. Justice for all in my mind is to get the WEF the fuck out of politics and stop giving these people "fighting climate change " 1500 God damned  jets to fly in and out of conferences like Davos in this past may.\n\nI can see how it would look though from an outside perspective. It\'s also against the intelligence communities that have certain powers at play and use them against unknowing civilians. \n\nI gave over 76 charges and 32 convictions (here come the bad ass jokes when I\'m admitting  I was not a Good person) i stole from garages and sheds, cars, trucks, I was a dealer for many years. Raided three separate times over 6 years. Once was my parents home when I was young. I don\'t even know why I\'m telling you this. I have nothing to even prove to you, but I can tell you now I did 15 days for dboard in the hole, 23 and half a day 4 separate times and after each time I was was placed upstairs in placement with a cell mate and my canteen back only to get in shit and go right back down to solitary. All in all I did 6 months total of 23 and a half lockdown half upstairs with canteen and cellie  and half solitary.',
 '[Read the article in English.](https://www.deeperblue.com/activists-oppose-octopus-farms/) [](https://translate.google.com/translate?sl=auto&amp;tl=en&amp;u=https://www.deeperblue.com/activists-oppose-octopus-farms/)\n\n**Automated summary:**\n\n&gt; A coalition of 97 animal welfare organizations and scientists is opposing the establishment of farms meant to raise octopus and other cephalopods for food.\n\n&gt;“Government policy must focus on shifting to a global, plant-based food system in an attempt to tackle the dire climate change emergency we are currently facing worldwide.\n\n&gt;Banning these types of farms now, will allow countries to develop in complete alignment with the 2030 SDG commitments moving forward.”\n\n&gt;He first got the diving bug while in High School in Cairo, Egypt, where he earned his PADI Open Water Diver certification in the Red Sea off the Sinai Peninsula.\n\n&gt;Since then, John has dived in a volcanic lake in Guatemala, among white-tipped sharks off the Pacific Coast of Costa Rica, and other places including a pool in Las Vegas helping to break the world record for the largest underwater press conference.\n\n&gt;\n\n\n\n**More:**\n\n* [Bay Bridge March Marks World Day for the End of Speciesism | Indybay](/r/VeganLobby/comments/x0div9/bay_bridge_march_marks_world_day_for_the_end_of/)\n\n* [Farm leaders vow to overcome vegan dairy shutdown | Farmers Weekly](/r/VeganLobby/comments/wxtrd3/farm_leaders_vow_to_overcome_vegan_dairy_shutdown/)\n\n* [World Day for the End of Speciesism: PETA ‘Animals’ Feast on ‘Human Flesh’ | PETA UK](/r/VeganLobby/comments/wz6gtl/world_day_for_the_end_of_speciesism_peta_animals/)',
 'based on Ethiopias largest model: 460 geothermal plants can power the entire US. However after it’s built, no one would ‘profit’ because it’s a one and done deal. 100 trillion in stranded assets if climate change is addressed. Monetary market system is obsolete',
 "The people angry that their taxes are supposedly paying to forgive some student loans for other people say that we should spend our taxes fixing their state which is underwater due to climate change which they say isn't real.",
 'Technology is improving cars and making them more resilient to rust. Ontario and Quebec winters are also getting much less intense due to climate change. The average car in Canada lasts 13 years now.',
 'There are a growing number of us in “that generation” that are taking climate change very seriously and are mobilizing in non-trivial ways.\n\nWhat you fail to see here by focusing on one generation is that the majority of humans across any cohort are not productive members of society at all. They are net-takers and are happy to leave a message behind.\n\nThere a very small group of people (proportionally) that keep the world spinning. From custodian/garbage man to engineers/physicians - they are willing to put in the hard/dirty work to keep society going.\n\nWithout that tiny minority, this festival wouldn’t be possible.\n\nAnd yeah, that hypocrisy is not limited to the younger generations. I’d bet boomers (or whoever else) are just as bad if you were to measure.',
 "San Francisco for really mild weather. No extreme top or bottoms. Wild fires and earthquakes, though.\n\nDuluth, MN. Will be pretty immune to climate change. Just snows a lot. Maybe not mild in the sense but if you are worried about the future of weather, it'll put you in a good spot.\n\nI just moved to Cincinnati this past week. I chose it due to it's more mild weather. There are a good amount of thunderstorms and rain, but it's sporadic and passes quickly. Tornados exist but are a rare thing. It barely snows or gets below freezing - less than half the national average snowfall. It doesn't get as hot or humid as Texas where I was coming from - only a handful of days this past year were above 90. Weatherspark gives it some hot/humid day rankings, but Austin, where I was coming from, would be humid every morning and have a level of some uncomfortable humidity most of the summer. You couldn't escape it.",
 'I work 2 jobs right now, 1 is in sustainable agriculture. The other is supporting adults with disabilities. The one job I am live on site for, the other I commute 32 km to. Which job are you relocating? Half the staff at the group home commute there because the the town is less than 3000 pop. You want to move the disabled people out of their small town where Everyone knows them and to a big city, just so you can have your dogma?  Or are we somehow moving the farm closer to the workers, longest commute now for the farm is 20mins by car. Certainly aren’t running trains to my farm or to the tiny village that I work in. It’s almost like there are a multitude of use cases for personal vehicles. Am I bringing hay in on a train in the fall? Or renting a truck that I need to somehow pick up without a personal vehicle. How are those disabled people getting to anything in the area, there Drs are in a different town, hell their groceries are in a different town. We have staff at the group home commuting 1 hr from the north to work, moving the group home eliminates them as staff as the town with the most staff is 30 mins south of where we are currently. Those staff that live 1hr north now don’t have a job or a way to get one cause they live in the wilderness where there is no public transit  and there obviously never will be. I literally have no idea what that 1st sentence is referring to, since I work in ag, all climate change effects me.',
 "You can't shame or correct these conservatives (including their supporters) using gotcha moments and hypocrisy. Every single issue - you can catch the right making disingenuous arguments. They don't care. Coherence and consistency of rhetoric and action is not their goal. Power and hegemony are their goals, and hypocrisy is just one strategy for accomplishing it. In fact, it's become a sport for them: rack up the most points, and any opposition focused on your words will never be able to hold you accountable for your actions. Kayfabe and theatrics.\n\nIn effect what you have is a wealthy elite that have weaponized social media in order to outsource their own PR/propaganda/public defense to gullible and disaffected individuals. The fascist engine works by transmuting populist anger and stereotypes into political power. If you look at what right wingers say vs what they ignore, only one viable conclusion emerges: The rhetoric is a smokescreen. They only care about power, preservation of power, and withholding of power from people they don't like. And they'll say anything to further desires lurking in the muck of their souls. All the while contributing to a festering crab-bucket mentality that drives intra-class conflict. \n\nDon't look for logical or semantic consistency across fascists' statements and beliefs: the real commonalities are who they affect in that moment: protecting in-groups, or hurting anyone that threatens that. And by the way some behave, existence alone can seem a threat.\n\nEdit: Rather than just bleating about a problem, I could also suggest a solution. Don't give them the attention they need. Whenever you hear a fascist hypocrite serving up their steaming, crappy hot takes, just tell them to shut up. Ignore them, mock them if you want, but don't engage with logic. And more important than wasting attention on right-wing fascists - **get anyone and everyone you can to vote**.  When you talk politics, don't talk about hypocrites - talk about real issues affecting real people, and how progressive policies like taxation of the wealthy, mitigation of climate change issues, safe and fair labor laws, and access to healthcare and education can improve the life of the average person.",
 'I’m not saying that it is. \n\nClimate change is displacing people, either we make the climate a place where ice sheets are thinning less quickly, or we need to displace. I hope that we can do that while we still can. Many can’t afford to be displaced.  And I don’t know many who want to.',
 " these people only have to scramble around filling shifts, because if the worker's were treated comfortably, they would have more time to resist and build an alternative system. thus they are only really needed by the parasitic rich at top to run as interference to the workers, and aren't really needed by the worker's themselves. further as you stated, they don't really come up with new ideas or make things better in anyway. further, there's been ton's of historical examples of the workers themselves coming up with new ideas, or implementing things better, so the rich at the top are only there because of the weight of their ancestor's rigging things in their favor. the collapse will come to some places faster than others, primarily the red states in the path of climate change. while you might find new York rebuilding, no rebuilding's going to really take place in Louisiana. you can already see this in the data, both republicans and those living in red states have a lower quality of life and life expectancy, and it's only going to get worse from here. while MIT has it at 2040 for societal collapse, i think some places will collapse earlier than that, maybe 2030 for the poorest red states. further, this is just going to further inflame fascism",
 'Look like he didn\'t bother learning the difference between mood and emotion. Mood = long term, emotion = transient.\n\nLike weather, day by day the weather can change, but for a given region it will have a general climate.\nNobody says today\'s weather is "mild temperate oceanic climate" because that makes no sense.',
 'Climate change, Russia, January 6 and Genders',
 '"Yeah, you see, we had a normal summer, climate change is a myth, what? So what we have 30 degrees in december, we had a normal summer, climate change is a myth" You',
 'Due to a number of factors, including climate change, I think more people who actually would quite like children, will also be choosing not to have them.',
 'The globalization we chose in the 80s has played out pretty much exactly as those opposed predicted so you’re right, the 90s was a peak. Reagan-Thatcher-Mulroney-Xiaopong changed the world. \n\nClimate change is also going to be bad, already causing long droughts. We’re in pretty deep denial but it’s coming.',
 'Leo on dating sites like: \n\nDislikes: fully developed pre-frontal cortexes, climate change.',
 '&gt;Here is another scientific study \n\nlol...this statement literally means nothing anymore. "Science" has become nothing more than yet another controlled and manipulated "source" for furthering an agenda.\n\nCase in point...Convid aka ....the scamdemic. 2 years later we have all the "science based experts" completely reversing their previous bullshit "science".\n\nSure there is real science in the world but anything used as "science" in the mainstream pushing a specific narrative is 100% bullshit.\n\nLike your post about "climate change"\n\nDo some research. \n\nIt started out as "Man-made global warming"\n\nThen they changed the moniker to "global warming\'\n\nNow it\'s called "climate change".\n\nI wonder if you are smart enough to figure out why they have changed it over the years?',
 'I wonder how many of these people froth at the mouth about waste and climate change',
 'My brother thinks the NASA web page on climate change is fake news',
 'As long as a little drone flew around behind everyone who pushes this notion and would instantly shoot them for eating meat. Same thing with the climate change awareness jet set club.',
 "humlog bahut sahi time pe paida hue kyunki it's the information age kuch bhi sikhna is so easy. kabhi kabhi main soch bhi nahi sakta hoon ki pehle ke coders ko bug fix karne ke liye kitne books tatolne padte. internet has made everything faster, from having a conversation with someone to ordering groceries.\n\nclimate change se bahut dikkat hoga aane wale samay mein toh aane wale generation ko bahut dikkat hoga.\n\n70s 80s ke bollywood movies jo ki gaun mein shoot hote the usme door door tak factories aur sadak nahi dikhte the. nadi pahad yahi sab dikhta the. abhi toh har jagah concrete jungle banne lage hai.",
 'Ok well you keep living in your world of bubble wrap protection. Why bring climate change into this? Makes zero sense. Canada was one of the highest vaccinations rates in the world, we also had some of the longest mandates. People can isolate all they want, just don’t expect to paid for it.',
 'No, they are saying that evidence is inconclusive.  It\'s easy to do that when you get to cherry pick the data.  This is fucking Rand we are talking about.  They likely have 20 year old similar *studies* on climate change data being "inconclusive" based on the *evidence.*',
 'nah nice try not reading literal thesis on why climate change is horrible, soecially when that thesis is heavily bias.',
 'That and climate change, which changes the jet streams and creating hander more erratic winters.',
 "SS: Pakistan's planning minister has said that the recent climate change induced floods have caused at least 10 billion dollars of damage. It was a monsoon on steroids. These floods have submerged a third of pakistan.",
 'Lol, nobody has been “living like this” for a while re: COVID. It’s been a free for all for a while. \n\nIts pathetic that society expects people to NOT isolate during any transmissible illness. Pathetic that we will baby the economy by unnecessarily forcing people to commute but not touch climate change with a 10 foot pole. \n\nMost office workers can do most, if not all their work from home so there is no point in “switching back” to the normalized notion that going into work, school etc. while sick is a good idea. \n\nFuck going back, I’m fine with people isolating if it means less chance of contracting their illness. Also some humans will always live with weakened immune systems and isolation is a great barrier to keeping them safe. They’re not always visible so again, preventative measures works best. \n\nIf you want to go to work sick, fine I guess, just stay in your cubicle/forklift/space and definitely expect me to think you’re a piece of trash for risking my health.',
 'Because the "infrastructure bill" was mostly pork, for stuff like mileage taxes, and various "climate change" crap that would fuel even more inflation.',
 'That still happens. See any coverage of climate change.',
 "That's so strange...... It looks like all the 'money' is pushing the climate change narrative.",
 'There’s nothing interesting about humans leaving their fucking trash everywhere and not cleaning up after theMselves. \n\nWe deserve climate change if this is how we willing treat our fucking planet.',
 'Its not a mistake. Theyre wrecking everything under the guise of "climate change".',
 'Well you could think about your comment for a second before posting. See I was commenting on an opinion, climate change evidence is scientific, so no opinion needed there. If you can show me a consensus of 97% of set designers/costume designers/movie makers in general that agree the production quality of this show is garbage, then you may have a point',
 'Okay I’m not sure that’s my point. It sounded like you were saying “The summer was hot once and the media were stoking fears about climate change” and my point was “No, they were using the miserable hot summer to illustrate the problem, but the proof of climate change is actually a different story.”\n\nA well-documented and accurate story. But nobody was using one hot summer as the only reason to show that climate change is real. Last summer was just an example.',
 "&gt;  It's definitely climate change.\n\nFuckin knew it.",
 'None of those things address climate change. What specially makes you think we’re going to outlast the massive changes that are coming? Where’s there food and water gonna come from?',
 'There are ecological models predicting that climate change, pollution, and overfishing ~~wool~~ will transition the oceans to the point that jellyfish become the dominant form of life. \n\nI like that you can also view it as a metaphor.',
 'That is my point. People incorrectly use moments of extreme weather as evidence of climate change, which is just confirmation bias. Actual climate change would be a global phenomenon recorded over years.',
 'they are the factory for the entire world and have a population of over 1.5 billion people, ofc they’re gonna be one of the largest polluters but at least they’re actively trying to move to sustainable energy unlike America. besides, climate change is an issue that can only be tackled thru global cooperation, china alone can’t fix this.',
 '“It gets better” they say, climate change still unmanaged. Even the most beautiful castle is fucked if it’s built on a swamp.',
 'Dan wore long pants to a concert in Miami in August and blames climate change that he’s hot.',
 'I’m sure everyone there made a Twitter post about how they’re worried about climate change.',
 "So far I have these books on my reading list:\n\n* Overshoot: The Ecological Basis of Revolutionary Change\n* The Limits to Growth: The 30-Year Update\n* Collapse of Complex Societies (I don't know if I'm really going to read this one, though)\n* The Uninhabitable Earth: Life After Warming\n* Our Final Warning: Six Degrees of Climate Emergency\n\nI'd appreciate hearing about your favorites.",
 "Curious that in 2022 recommendation algorithms can't differentiate between positive and negative connotations. It's like getting recommendations for a sauna after you check stuff on climate change",
 "I don't know many younger people that even want these things anymore outside of owning a car and I don't think I even know anyone without one of those. How many women do you know that actually want to be a housewife or young couples that want to move out of the city to the suburbs same goes for having kids most don't want to spend the money or have their free time taken away or just don't like the way the world is heading with climate change etc..",
 "Seriously this. The carbon tax was originally proposed as a *Conservative policy* and was coopted by the Liberals, it's a way to use market forces rather than government regulations to push companies towards using less carbon. It's pretty much the most milquetoast climate plan possible that has a chance of accomplishing even poor results that are too little too late. But now that the Liberals put it in place suddenly it's the just the worst response to climate change possible and the Conservatives are scrambling to find an even less effective option.",
 'Lmao Fraser Insitute hack and former  conservative politician. Can safely ignore.\n\nIt would be highly irresponsible for pension funds whose investment horizon spans decades to ignore real tangible risks from climate change in their investment analysis and decision making.',
 'This is exactly how I try explaining to my trump neighbors who view the world through the lens of their farms being the whole world. If they want a say they have to bring their concerns and policies to the table with sane leaders who act on good faith and doing the right thing. Part of doing the right thing is not lying to their constituents over and over again and feeding them lies bigger than they can understand. They can’t understand anything other than that the sun rises in the east and sets in the west. They don’t understand that due to climate change their conservative ideas kill their whole entire lifestyle. They have been breeding ignorance for years and the brain drain is showing itself right now. I moved back home to rural MN after hopping around the country to experience life outside of the rural perspective. I like it better here and I understand where the rural ignorance comes from; it comes from the root of everything we all want, prosperity and peace. Unfortunately they have been voting in the bogeyman for so long they themselves have become the bogeymen.',
 "If you haven't noticed the reputational damage done to science the last few years, by trained scientists prioritizing their (primarily political) biases then I suggest you haven't been paying attention.\n\nSo let's refresh...my OC is that these particular individuals are choosing to act as political activists instead of objective scientists. They've admitted as much. So what exactly are you disagreeing with?\n\n&amp;#x200B;\n\n  \nEdit: to synapsebondfire, my favorite kind of interweb debate-lord who when intellectually pushed just the tiniest bit, panics and resorts to the tried and true strategy of replying to, and then immediately blocking, the person they disagree with. If I had an award to give..you earned it little buddy. Now, to keep the hierarchical flow:\n\n&amp;#x200B;\n\n&gt;Says the guy with nothing but vague accusations. I assume you’re talking about climate scientists. And if you’re denying climate change I just assume you’re an idiot, which is a very safe assumption at this point. Also done arguing with someone screeching ‘omg people are political at times and that’s terrible’ which inherently a political statement in and of itself. You have no argument just empty vague accusations that are meaningless to begin with\n\n  \nLet me help ya out here...I was referencing primarily the covid shenanigans of most 'science based' public institutions and the wokism infiltrating higher ed but absolutely some climate scientists (like the ones in the article) haven't done themselves any favors either.",
 'Kind of. We often plan for what we think the future is going to be. Like: “the consequences of climate change are going to suck, so we should reduce our dependency on fossil fuels and switch to electric cars” what happens tomorrow influences how we behave today.',
 "I am literally eating oatmeal with flex and sunflower seeds, nuts and multivitamin drink.\n\nI get this question right when I eat the healthiest meal of all time and it's also vegan so climate change will be helped A LOT too.",
 'So, jumping to conclusions like this are a bad idea.\n\nWhat happens if next year we get a mild year with averages of temperature, rainfall and everything else? Do we assume climate change is over?',
 'Thanks for the heads up on this! I was going to order the clear cover, but now will skip it. This planner is going to live in a drawer at work, so it’ll be fine to expose it to the world. Dog knows real polar bears are in greater danger from climate change induced habitat loss, neh?',
 "The air is warming. The oceans are also warming.\n\nWarmer air holds more water. Warmer oceans evaporate more water and generate more humid air. \n\n&gt;\tFor one to form, there needs to be warm ocean water and moist, humid air in the region. When humid air is flowing upward at a zone of low pressure over warm ocean water, the water is released from the air, as creating the clouds of the storm.\n\n&gt;\tAs it rises, the air in a hurricane rotates. Air drawn into the center of a hurricane curves to the right in the Northern Hemisphere and toward the left in the Southern Hemisphere due to the Coriolis effect — a phenomenon in which winds curve because of the Earth's rotation. At lower latitudes, where there is no Coriolis effect, hurricanes cannot form within 300 miles (500 kilometers) of the equator.\n\n&gt;\tStorms grow if there is a continuous supply of energy from warm ocean water and warm, moist air. Tropical storms can grow into hurricanes, and hurricanes can grow into stronger hurricanes. However, only a small number of storms grow into tropical storms. Even fewer become hurricanes.\n\n&gt;\tStorms weaken when they move over areas with cooler ocean water. There isn’t nearly as much energy in the water to fuel the storm, nor is there as much humidity in the air. Hurricanes also weaken when they travel over land.\n\nTaken from here: https://scied.ucar.edu/learning-zone/storms/how-hurricanes-form\n\n&gt;\tLarge sections of the global oceans have warmed 1-3°F during the last century, with some buoy locations having a more dramatic rise recently. Of the Gulf of Mexico buoys analyzed, the average water temperature from August through October has risen 1-2°F in the last 40 years. The buoys analyzed in the Atlantic have warmed even more during that three-month period — 2-4°F in the last 30 years. While at single buoys the water may have warmed faster or slower than other locations, globally, there is a clear trend towards hotter sea surface temperatures.\n\n&gt;\tWarmer water fuels hurricanes, and not surprisingly, the average number of major hurricanes — those Category 3 or higher — has increased in the Atlantic basin since the 1970s. Globally, climate models project that there will likely be an increase in the wind speed and rainfall intensity of the strongest hurricanes (aka tropical cyclones), even though their frequency may remain unchanged or even decrease by the end of this century. However, the extent of these effects from climate change will vary by region.\n\n&gt;\tEven with no increase in storm strength, the damage from storm surge is already worse because warming waters are also contributing to sea level rise. Seas are projected to rise up to at least another 3 feet by the end of the century, only adding additional water to any tropical system’s dangerous storm surge.\n\nTaken from here: https://www.climatecentral.org/climate-matters/us-coastal-water-temperature-trends\n\nTLDR: Atlantic, where hurricanes that end up in the Gulf of Mexico form, is warming. The Gulf is also warming. Warmer Atlantic means more energy to feed the air cycle of a nascent storm system and help it develop into a hurricane. The warmer Gulf will help ensure less energy is lost as the system moves from its forming grounds. Rising sea levels (they are already rising) will also make storm surges worse. \n\nThe best we can hope for is an increased frequency of storms that we can just about manage. The worst case is the same frequency we’ve seen in recent years, which is up on historic averages, but with more of the mega Cat 5 systems actually making landfall.\n\nI wouldn’t buy property anyway on the Gulf cost. NOLA is going to be destroyed again within my life time and it won’t be viable to rebuild. Miami will be underwater within my lifetime. Those refineries on the LA/TX border? They’re going to get a pounding one day (again) and it will be a major disaster.",
 'Robert Patrick doesn’t believe in climate change and would regularly walk around set shouting “MAGA MAGA MAGA.” He is not an intelligent dude.',
 'Yeah, sort of like how all the other climate change forecasts have turned out totally accurate and not insanely conservative compared to reality. The planet is on fire.',
 "Climate change is real and it's caused by man-made emissions of CO2. Global mean surface temperatures seem destined to increase by 3.5-5.5°C by 2100 as a result.\n\nHowever, none of that means Victoria Gonzales will have an average maximum temperature of 30°C on August 30th. Even by the year 2100, with the expected warming the average maximum temperature will be closer to 20°C than it is to 30°C, probably in the vicinity of 22-23°C.\n\nEdit: 3.5-5.5°C. I'm all thumbs.",
 'I like the guardian. \n\nDon’t really give a fuck about climate change though so can’t comment',
 'Bernie considers climate crisis the most valuable change to national security. He opposed the manufacturing of the Dakota Access Pipeline on the grounds that it will have a huge result on the climate.\n___\nBernie Bot -Bernie_Bot_03',
 '&gt;This week, the Lever, ProPublica and the New York Times  \n discovered the largest known political advocacy donation in American   \nhistory. We exposed a reclusive billionaire’s secret transfer of $1.6bn   \nto a political group controlled by the Republican operative Leonard Leo,  \n who spearheaded the construction of a conservative supreme court   \nsupermajority to end abortion, block government regulations, stymie the fight against climate change and limit voting rights.',
 'That was literally the PR policy of the oil companies when they realized climate change was going to be an issue.  Treat it as “just a theory”, demand “equal time”, deflect, deflect, deflect.',
 'Lul ain’t nobody scared of no climate change climate change is natural',
 '“Climate change doesn\'t exist" mfs be like.',
 'Its probably influence from sci-fi films, combined with the current state of the world and outlook of the future especially in regards to climate change.',
 "So what's their solution? Not find an alternative, but to pretend the oil is infinite and that climate change doesn't exist because we're all off to an eternal orgasmic afterlife anyway as long as we obey a randomly homophobic sky patriarch.",
 "I don't see how they can even pretend to care about the environment or climate change anymore, especially if they elect PP. You can't campaign on getting rid of the federal carbon tax (an economically illiterate position), getting rid of environmental protection legislation (bill C-48 and C-69), expanding pipelines, and expect people to take you seriously on environmental issues.\n\nConservatives don't want to conserve the environment. Everyone already knows this. Rick Perkins and many other Conservatives fight against fishing moratoriums. If a green conservative was in the leadership race, they'd be called a Liberal instantly (if they even get approved).",
 '&gt;Giving the American people the right to vote on Abortion is not extreme. What\'s extreme is taking away that vote by letting SCOTUS decide.\n\nShould we also give Americans the right to vote on whether blacks count fully or just for 3/5?\n\nOr if we should allow "Whites only" signs on water fountains again?\n\n&gt;As a country we\'ve always believed that there are things not listed in the constitution and that states have a right and obligation to handle those things the way they see fit. There\'s no right to Abortion in the U.S. constitution.\n\nAnd literally every time we\'ve done that it\'s been used to oppress people until the federal government had to step in.\n\nBut continue arguing in favor of the state\'s ability to *force* you to give birth, because that\'s not fascist at all.\n\n&gt;That being said, the only people concerned that Roe was overturned is the far left. Right now Americans can\'t afford gas, food, and necessities (or those necessities are gone). That\'s what they are focusing on and the left is doing nothing to fix it. We are in a recession and all the left can do is spend money on "climate change" when they should be trying to get us out of this recession.\n\nMan, it must be nice to be able to just completely ignore the half of the Senate *you control* refusing to agree on so much as a lunch order.',
 'Yep…bloody flat-earthers, climate change denying plague rats, won’t understand simple geography',
 'I bet these same people are highly concerned about climate change.',
 'I believe in climate change. I just know even if all the average Joe’s in the world did everything they were supposed to, the rich and powerful would still burn everything down on their own, so I refuse to be guilt tripped into eating bugs so celebrities can take their yachts to Malaysia.',
 "/unchurch\nThe fibrin scar tissue that occurs as a result of damage to the myocardium allows for/causes clots to form. These clots interfere with blood movement, sometimes resulting in pulmonary embolisms.\nThe injured heart tissue itself can also lead to complications of atrial fibrillation.\nNow, adding intense exercise exacerbates these issues and those who were unfortunate to have their body end up with these complications have a higher chance of mortality.\n\n/Rechurch\nIt's definitely climate change.",
 'I agree with this. Just saying it is just gonna scare away people on the fence. We’re in the realm of being able to focus on things like climate change, cancer, heart disease etc but we’re focusing on guns.',
 'if those gaudy islands in the middle of climate change is how they are spending their money, I would not put a lot of faith in that',
 "It's a legitimate anxiety to have.. Sorry to say. It's bad. But, society won't collapse until around 2070 (if no measures are taken). And even then, people won't start dying by the millions from it for another few decades. Once again, this is if nothing is done. Millenials and Gen Z will be coming into political and government roles soon, instead of old conservative whit dudes. That gives me hope. I've had to do several projects on climate change.",
 'Who needs fish? Who needs forests? Who needs land animals? Who needs insects? It is all a lie... we are to rule this planet..it is ours...we can do with it, whatever we want..climate change? Leftist propaganda...God wills it ...Deus Vult...\n\n/kappa',
 "No, you're oversimplifying.\n\nClimate change can cause drought, ocean acidification, polar ice cap melting in 30, 50 or 100 years. Or shorter. Or longer.  The clathrate gun permafrost might happen sooner. Or later. Or happen and be overstated.",
 "...I know what climate change is. it's a great side effect of veganism but *it is not what veganism is about*",
 "Yes, climate change will be the end of many species on earth if it isn't reversed. Say goodbye to penguins and polar bears if all the ice melts. Deserts in Australia and Africa are spreading. African wilife is struggling to find fresh water. Ocean acidification is only getting worse. Coral reefs are dying. Earth's wildlife is in serious trouble and will be pushed to the limit in the next few decades. \n\nAbolishing animal agriculture hopefully will happen in the next few decades. Or at least drastically reduced. I think it's inevitable at this point. This means we won't be using any animal products as we'll need all the resources we can get to feed humans when droughts and heatwaves are more common.",
 'The heat now is nothing compared to what it will be in 30 years lol, there have been heat waves all across the world and climate change hasn\'t even "started" yet. You are right though, humans are adaptable. In this case though, humans in the Southwest will likely have to adapt the way people in Dubai live now. Only coming out at night, never during the day, wearing cloth over their entire body head to toe, etc. Scary thing is all the rivers will dry up, so they will only survive on their groundwater, for however long that lasts... Not sustainable at all in the long term of humanity.',
 'Memo from pedo pete saids all the increase in deaths are because of the Jan 6 th insurgants,putin, antivaxer,conservatives, white heterosexual men and climate change.  Anyone who differs should be banned,censored and be labeled a fascist nazi.',
 'Climate change',
 'Climate change',
 '&gt;Pennsylvania\'s GOP gubernatorial nominee, Doug Mastriano, deleted 14 videos from his Facebook page in which he touted far-right positions, including referring to climate change as “pop science."\n\nDoes anyone have these videos?',
 'Worldwide it’s a small minority. In the USA, it’s more than 1/4 of the population.\n\nhttps://climatecommunication.yale.edu/publications/climate-change-in-the-american-mind-april-2022/toc/3/',
 'Would it make you feel better that Justin Trudeau says climate change is the reason for Everyone anxiety.',
 'Like climate change?',
 "climate change weren't we overdue for an ice age",
 'Nobody is making bugs for human consumption you chuds, and even if they were, it’s the least we can do for the climate.\n\nNobody is saying forever, just a little big longer.\n\nYou might be done with climate change, but climate change isn’t done with you.',
 'Not a summer in North Central Texas. \n\nEven with climate change cooking the books, these are early fall numbers.',
 "I am just tired of people arguing over covid / nuclear or not / whether climate change is real or not.\n\nFuck this. I'll come back once people have some sort of common sense in them.",
 'The EU will solve it by adding further restrictions on farmers in the name of fighting climate change.',
 'I’m not sure how productive this is because you are missing my entire argument. The point is that a small child thinks it is impossible to read before they are taught how to do so, and they feel the way that you feel about climate change. The overly pessimistic fear that there’s no way it can be accomplished can be overcome. \n\nThe same holds true with respect to facing the dangers that exist in the world today. Every single generation has had its own set of hardships and reasons why it thought it’s kids were in danger. That doesn’t mean they shouldn’t have kids.￼ it does mean we should teach them to face those challenges.￼￼\n\nAnd just to take the bait on your mass shooting argument, what that means to me is that we have to teach our children that there are ways to make changes in our society. If you don’t like the way things are going today, get out there and vote to make a difference. Get out there and talk to people to make a difference. Get out there and show people what you want the world to be like, and build a coalition around it. That is how our society works.￼ Your view is evidently different. \n\nI don’t want to be in your camp, where the existence of enormous problems such as climate change and mass shootings means we should allow humanity to go extinct.￼ You may think I’m crazy, irrational, overly pessimistic, naïve, etc. But there is simply nothing that you have said that leads me to believe that the best solution to our problems is the extinction of mankind.￼',
 'Hmmm I think we should write to our politicians about climate change. Maybe they can help',
 'First i was like "aww, sweet. They\'re still together and they\'ve given up on driving a car to battle climate change." Then i realized what sub this is in.',
 "I often try to include fundamental changes in my world's history. All in all makes it more realistic as change is the essence of reality. I even have maps from different stages in my worldbuilding process that are all valid because the world changed due to tectonics, climate change etc.",
 'Yes. They are all possibilities that should be explored. We would likely not hear about the geologic record having multiple periods of climate change indistinguishable from industrialized climate change unless it also had a spin that was acceptable for publishing. That’s a problem that should be addressed because if we have hypothetically taken a wrong turn it will take longer to correct when bias has enabled it to continue far beyond the scientific method. Facts are facts and they should be able to publish at face value and shouldn’t require an acceptable narrative to pass publishing. I speak partially of this article because it appears to me to be the case here, as I feel they’ve reached around the obvious and controversial to be accepted for publishing but even if that wasn’t the case here, it is a prevailing issue.',
 'I hear he’s also a climate change expert',
 'Will they still be around with all the climate change?',
 'Wait until they hear about [NFTs that help fight climate change](https://co2compound.klimadao.finance/)',
 "People aren't saying that the MBTA should pay for itself with the exception of only a few minor voices in the room. People are often accused of arguing that point when really they are just asking for the MBTA to be partially funded by fare revenue so utilization of a capacity constrained public service isn't fully free to the end user. Which is the change that advocates are trying to change by making the MBTA free. \n\nI can see the counter point as well which is that the poor largely get hit by that funding source but that is true of the gas tax as well which many pro-public transportation advocates actually want to increase for the climate arguments.",
 "it is my belief ghosts are part of another dimension. I believe ghosts will still be here because the PHYSICAL earth will NOT be destroyed, humans just won't be able to survive the climate change.",
 'Best to not say anything. Don\'t want to offend any of the poor oppressed children. Just do your job, get shit on for being in a lower social class and make sure our future leaders know their better than you.\n\n\nEven funnier is that these are the kids who march out of class to bring attention to Climate Change and pollution, while Ignore their contribution to it.\n\n\nAND EVEN FUNNIER, the response in this thread. From "blame the parents" to "Do your job janny". So much wokeness and equity....',
 'Ok so to answer your question hypothetically: nothing. If the earth is completely destroyed by climate change and say all humans go extinct then nothing. Ghosts would just be living in a now very polluted and empty planet, they cant die so yeah. The end.',
 'Its possible Ordered new corepads but since some other people report this issue I dont think thats the problem.\n\nSo what climate are you living in. You dont feel the initial friction change at all? when I first got it it was super speedy, and now its just mud pad sometimes.',
 "People could very easily correlate if they thought for just 5 seconds but they just don't want to when they don't like what science says. My main point was that people see scientists as the people bring bad news like things about covid and climate change and start being anti science while using the technology developed based on science. Kind of like someone on ACA claiming that Obama care is bad or someone getting their loan forgiven claiming that loan forgiveness is bad when others get it. Point is not that people don't know but that people loving something that helps them and hating it when it helps other or asks them to do something they don't want to do. Point is to point out the weird mental gymnastics they do.",
 'I\'m a man and I listened to Jordan Peterson, and I can say what he\'s trying to teach me is none of that. He is not doing that all in any context. He\'s using those phrases/words, but under the guise of being racist, sexist, and overall bigoted. Being responsible and cleaning my room are basic things I already learned from my mom. Jordan Peterson\'s appeal is that he tries to rationalize being a "man" with extremely outdated and harmful views, in an attempt to uphold the power structure White men used to have. \n\n  \nAlso he straight up doesn\'t know anything about [climate change](https://www.theguardian.com/environment/2022/jan/27/word-salad-of-nonsense-scientists-denounce-jordan-petersons-comments-on-climate-models)(but still thinks of himself as an expert on the topic). His comments on Black Lives Matter were horrendous(and shows how completely limited is viewpoint is), his PhD research has nothing to do with like 90% of the topics he talks about and has no expertise in.  \n\n\nIf I wanted to be someone who does no research and talks nonsense, sure Jordan Peterson is a great listen. If I wanted to learn to clean my room, there are people who I can talk who aren\'t a bigot like Jordan Peterson.',
 "veganism isn't about climate change.",
 'We wouldn’t have to worry about climate change.',
 "with the mountain of data supporting the cataclysmic danger man-made climate change poses, i'd say there's much more understating the danger in the media than overstating.\n\nthat said, it's not hopeless, there's been a lot of positive change in the last couple of years, so despite megacorporations kicking and dragging their feet the whole way, things are looking up. i don't know if you need to hear it's not hopeless to take the threat seriously, but if so, there it is",
 'I have a plan to fix climate change. What we’ll do is give more subsidies to oil and gas corporations, while mandating every small town has a never ending burning tire pile.',
 'Terrorism is performing acts of aggression in an attempt to cause fear and thus force someone to change what people are doing. The problem is that it doesn\'t work, the people that you kill will be replaced and the rest of the people will just dislike your ideas and beliefs more because the vast majority of people doesn\'t like violence. And now imagine that in a country where the government can spin that bombing into an already established narrative of "us versus the west". Yeah, you are just making Russians agree with Putin\'s rhetoric more...\n\n"Comparing someone dropping a candy wrapper on the ground to an invading army committing war crimes across your nation is pretty ridiculous TBH and I’m not even going to entertain that."\n\nComparing the damage done to the climate worldwide by large scale polluters to someone dropping a candy wrapper on the ground is pretty ridiculous TBH and I\'m not even going to entertain that...\n\nBtw, did you consider that maybe there is a person in those photos that may have done the bombing in question, but he is just having a day off to relax after a hard months work of secret freedom fighting/terrorism? Don\'t judge people for enjoying their free time, you go crazy if you just spend every minute of every day thinking about how to kill Russians. Or maybe you are already crazy...',
 'climate change, i mean thats not just in New York but you should still be afraid',
 'Or that perhaps the effect we are having on the planet is at the scale of previous natural occurrences of climate change.',
 "We need more climate change so it's consistently warm enough where I live to ride a bike everywhere.",
 'NDP and Liberals are showing how little they actually care about acting sufficiently on climate change. The Greens are also recovering from the Annamie Paul era.\n\n(Makes a lot more sense than the other comments here, given the most likely second choice for the Greens is NDP, and second/third choice for NDP voters is the Greens).',
 'I’m curious, and NOT trying to start a big thing, but why do people like Gore, Obama and various other climate change espousers buy beach front property that costs millions of dollars? Isn’t that shit going to be underwater?',
 "Not religious, and I live in the Atlanta area in the US. \n\nClimate change is going to cause a lot of harm to a lot of people, but it is not so dire that it's going to wipe out our futures. Especially here in the rich world. Unfortunately most of the harm is going to come to the world's poorest, those at risk of famine and more exposed to extreme heat. Mass migrations and fights over natural resources could get ugly. But they're not going to threaten civilization itself. \n\nAtlanta's weather is gonna look like present-day Jacksonville's if we get to worldwide 3C warming. Which is fine. I'm not planning on apocalyptic outcomes because the weather here looks like Jacksonville.\n\nAgain, we should do what we can to prevent climate change from happening. But if you think the world's gonna end, you're deluding yourself.",
 'People like you make me root for unstoppable, extinction-level, climate change.',
 'During this minute, people are being informed about climate change',
 "She's not a crazy person. She's super smart. She honestly probably does know more about the state budget than any other politician in Oregon.\n\nI'd never vote for her because her environmental record is pretty terrible. I don't know if she's a climate change denier per se, but she doesn't' really care about addressing it. But for some people that's probably a plus.",
 'Climate change works on a sliding scale though. It is not black or white.',
 "It looks like OP posted an AMP link. These should load faster, but AMP is controversial because of [concerns over privacy and the Open Web](https://www.reddit.com/r/AmputatorBot/comments/ehrq3z/why_did_i_build_amputatorbot).\n\nMaybe check out **the canonical page** instead: **[https://www.cbc.ca/news/canada/nova-scotia/carbon-pricing-climate-change-greenhouse-gases-1.6566471](https://www.cbc.ca/news/canada/nova-scotia/carbon-pricing-climate-change-greenhouse-gases-1.6566471)**\n\n*****\n\n ^(I'm a bot | )[^(Why &amp; About)](https://www.reddit.com/r/AmputatorBot/comments/ehrq3z/why_did_i_build_amputatorbot)^( | )[^(Summon: u/AmputatorBot)](https://www.reddit.com/r/AmputatorBot/comments/cchly3/you_can_now_summon_amputatorbot/)",
 'Spreading what bullshit? That what was once global warming was changed to climate change because global temps in many regions was cooling? And that climate change could be used regardless what “the weather” cycle was doing? It’s easy to look back at print and then digital docs that proved its all a money /power grab. They’ve been warming of a climate emergency for 60 years. According to Al Gore and other hoaxsters the state of Florida should be half underwater for the last 20 years, as should Manhattan lol',
 'You guys aren’t getting how climate change works are you?',
 'All I’m getting from you is “you can’t care about one topic unless you care about ALL topics”. Your posts are damn near impossible to read and maintain a coherent message. \n\nI do like how you blame capitalism and war for climate change. Especially since that’s not the case. It’s humanity that is to blame, not a mechanism of it.',
 'I alread know of them but thanks. I had no idea they were about climate change, I just thought the members were vegan',
 'This weather event does not imply the opposite of "warming" merely because that area of the world had a larger than average amount of ice buildup one day - you\'re not posting about "coverage of weather events", you\'re posting about a specific event and then making the claim that these events are ignored... they\'re not.. they\'re recorded and used to build a model of climate change, which indicates that we are warming the globe through human behaviors.. a model \\_you\\_ appear to be ignoring.',
 'watching the earth suffer from the consequences of climate change knowing it could’ve been avoided if al gore won in 2000',
 "The pop went up a 100k in last 2 years. Most of that was inward migration. It doesn't matter a damn if the birth rate declines. With climate change in the mix, the pop could hit 10 million in 50 years. Those boxes that shit money are only going to get more valuable.",
 "The Earth's climate is always changing. It will change whether there are humans on it or not.",
 'You know climate change is getting to a point where its irreversible right? (We might have even crossed the threshold)',
 '&gt;\tHowever her actual childhood hasn’t actually been affected by climate change\n\nhttps://www.politico.eu/article/excess-death-surged-heat-wave-hit-europe/amp/\n\nthisisfine.jpg',
 'Yes, fuck you in general. Now please stop hanging around a reddit thread about thousands of people dying due to the apocalyptic effects of  unrestrained global warming and climate change. Find something better to di',
 'Animals are precious, I’m a horticulture major who’s concerned about ecological deprivation and the rapid change in climate in the last 10 years alone. Also animal proteins are a bit harder on the digestive tract than their non-animal counterparts. Also a lot cheaper to eat vegan. Plus I have pretty bad contamination OCD and eating meat is a huge source of anxiety for me. Although that’s more of its own problem but I’m sure you get what I mean',
 '"While that may still be a concern, the larger concern today involves the overreach of the SCOTUS and the GOP\'s pivot towards more extreme positions relative the abortion debate (and the potential additional restrictions of freedoms could be taken by the SCOTUS and the GOP)."\n\nGiving the American people the right to vote on Abortion is not extreme.  What\'s extreme is taking away that vote by letting SCOTUS decide.\n\nAs a country we\'ve always believed that there are things not listed in the constitution and that states have a right and obligation to handle those things the way they see fit.  There\'s no right to Abortion in the U.S. constitution.\n\nThat being said, the only people concerned that Roe was overturned is the far left.  Right now Americans can\'t afford gas, food, and necessities (or those necessities are gone).  That\'s what they are focusing on and the left is doing nothing to fix it.  We are in a recession and all the left can do is spend money on "climate change" when they should be trying to get us out of this recession.',
 'Climate change',
 "Nobody is going to listen to Kevin from the council estate, when he calls for climate change action.\n\nIf high-profile people can't call for action without living in a yurt, then they're not going to do it.\n\nBetter that they fly in private jets and call for action than they fly in private jets and pretend climate change isn't real (like right-wing politicians).",
 'they should just halt it immediately, why wait? the climate isn’t waiting for us to change our ways',
 "&gt;Most of the deforestation is being done by large corporate   \nagribusinesses with international financing and that not only   \nirrevocably damages biodiversity (extinction is forever) and exacerbates  \n climate change\n\nHow is this different than what's going on in the US? And the regions of Brazil that are being deforested are also the regions of Brazil that voted for Bolsanaro.",
 'Who is slaying dragons? Nobody. As of now, nothing is being done. We are headed straight to disaster. When you’re deciding whether to bring children into this world, look at reality, not what you think reality should be. I swear, people treat climate change like some kind of vaaaague idea that may or may not happen at some point.\n\nAlso, by bringing children into a world of increasingly scarce resources, you’re not only hurting the child but also becoming a cause of the problem.',
 "Yes, I think I have stated my question multiple times now: Most people don't currently support the changes needed to curb climate change due to the costs and trade offs involved. **Why would they suddenly support the changes needed if the economic system changed to socialism?** I've yet to hear an explanation of that anywhere here.",
 'So many people here talking about climate change as the cause.. look closely. You can see at least 2 orcas moving the ice.',
 "So OP is talking about Consistent Trends versus Singular Events.  Like the difference between Climate Change and Weather.  It's literally watching people who cannot interpret how the definitions of words interact when placed in a sentence to communicate an event or idea...",
 'Bernie considers climate crisis the most valuable change to national security. He opposed the manufacturing of the Dakota Access Pipeline on the grounds that it will have a notable result on the climate.\n___\nBernie Bot -Bernie_Bot_01',
 'I\'ve argued EvC since the alt boards on monochromatic screens, and coming here I don\'t see any changes in their "arguments."\n\nOn top of that, anti-science is as strong as ever, from climate change deniers to anti-vaxers to essential oils, not to mention flat earthers who were back then a literal goof.\n\nAnd while religion may be in decline, here and there, rabid religious though is rising up. No one thought Roe would be over turned, not Texas is forcing schools to put up "in god we trust" signs, so I hardly expect "The push to get creationism in schools seems to have stopped" if it is the case, to be the case soon.\n\nIf you "ok boomer" this, you\'re in for the same disappointment every generation has.',
 "Don't forget how Toyota was lobbying against laws that would help with Climate Change.    \n\n\nSource: https://driving.ca/auto-news/industry/toyota-ranked-third-worst-company-for-lobbying-against-climate-policies-report",
 "Sure it wasn't climate change?",
 'You know nothing of being wrecked. Relationships, or whatever you call them, pale in comparison to, fiancé’s father committing suicide. Then fiancé trying multiple times to do the dame or your own father dying of a heart attack in his sleep. Abuse from a narcissist ex boyfriend. Not to mention many other things much worse then that. \n\nThose expectations you had suck. No denying that. But its just on a whole different level then what other people deal with. Its like a first graders homework vs climate change.',
 'This feels like that time Trump brought in a snowball to prove climate change wasnt real',
 "The post wasn't just about Pakistan. It was mainly how climate change is the result of what is happening in Pakistan.",
 'it seems you support climate change as long as it doesnt affect you. \n\nI already wrote what my actual solution is (relocating JOBS, not people, and expanding the train network) so you should stop strawmanning atp dont you think? If you have nothing of value to add, then dont add anything.',
 'Sorry, you have completely misunderstood my stance. I am gravely concerned about climate change, and have been for a very long time. But that is not at all the point of my posts.￼  \n\nRemember, we are talking about dragon slayers – not dragon deniers or dragon acceptors￼.',
 'Russia but its melting because of climate change. If you had waited a little bit there would be flames too, as the forests are set alight.',
 'What do we gain? We know we are fucked. Politics is doing nothing against climate change anyway.',
 "There is nothing climate change, apologies I mean global warming, I mean global cooling, can't do.",
 'Follow the money.\n\nCorporations want climate change to be a hoax because it increases their profits and they will spend a lot to push this narrative with paid propaganda.\n\nWho wants to be an unpaid worker for corporations and spread their message?',
 "No one will be forced to cut out meat entirely, but having to eat way less of it is highly probable in the near future due to meat production's huge contribution to climate change. There will be many substitutes introduced for the shortage.",
 "This may be a ahock to you, but the world &gt; the USA\n\nThe Netherlands are currently drier and warmer than they've been in decades and icecaps are melting at an alarming rate. And in the comments I read something about floods in the US.\n\nYou're taking one example of the effects of climate change, oversimplify it, and try to debunk it with anecdotal non-evidence on a local level.",
 'The gays are causing climate change and corporate erosion of democracy',
 'You know those inconsistencies of capitalism that Marx described, and predicted would eventually manifest?\n\nYes, the same ones that capitalists told you were bullshit?\n\nThis is them.\n\nNow just wait to see what happens next with climate change.',
 'Correct.\n\nThe rest of the state will soon follow.\n\nLarge portions of the southern United States are going to become increasingly uninhabitable over the next decade due to failing infrastructure &amp; climate change.',
 'Reckless development in historical flood plains definitely happens in places like Houston. But nothing remotely this large has occurred in the Indus Valley in the last 100 years. 100km inland lake caused by 2 months of non-stop monsoons has never been recorded there in modern times. The lack of archaeological evidence or oral histories points to it not occurring in the last 6000 years. \n\nIn fact recent flooding since about 2012 have started wiping archaeological sites that had withstood cyclical flooding for 4500 years. Don’t confound this climate change driven unprecedented flooding with 100 year flood cycles. This is the new normal.',
 "Last year I wore shorts and put sunscreen on the back of my neck the second weekend in October because the temperature was in the 80's. I didn't need a long sleeve shirt until the week before Halloween. Climate change is a beast, let me tell you.",
 "Climate change will always exist and always have, it's when these changes are too extreme or too frequent that pose an issue.\n\nCivilization exists, so extreme climate change must be mitigated.",
 'From the list, I wish the right would stay out of climate change, abortion, welfare for the disadvantaged, and protections for minority groups, as they seem to block a lot of opportunities just because its not "their" way. Just like I would like to see the left stay out of things like criminal justice reform, gun laws, and immigration enforcement. All of these examples are to an extent within reason, of course. I didn\'t include voting integrity because that is something that both sides have to take seriously and not just use it as some conspiracy platform to attempt to overthrow election results. Both sides have their strong suits, and both have their downfalls.',
 'You misunderstand, eating a variety of vegetables is something everyone should do to get proper nutrition.  The only change that I made when becoming vegan was my protein because I already ate a variety of fruit and vegetables daily like an adult.  You are acting like it’s some big fucking chore.  It’s not.  I take B12 because I no longer get it from meat (animals are fed this supplement because they no longer produce it naturally due to modern farming - I just take the supplements directly).  \n\nBefore I became vegan a dish that  I made might be meat, potatoes, broccoli, carrots, onions, corn.  Now I just replace the meat with tofu, seitan, beans, or sometimes some off the shelf meat substitute.\n\nSo back to your original question\n\n&gt; Why eat a variety of vegetables when one meat product does the trick.\n\nI already ate a variety of vegetables \n\nAnimal agriculture is horrible for the environment - one of the major contributors to climate change \n\nThe horrible treatment of animals in the meat and dairy industry\n\nHealth benefits.  Vegan and vegetarian diets dominate health outcomes with respect to diseases like heart disease, cancer, etc.',
 'Incredible!\n\nAlthough I did lose a pole down a deep crevasse on Gran Paradiso literally the second day I used them lol. I feel bad about littering too :( but there was no way to retrieve it. \n\nMy right knee held up. \n\nI had some pain on my left (the one with the reconstruction) after the summit day on Gran Paradiso. I actually slipped and hit a rock, this is when I lost my one trekking pole. And I didn\'t realize it at first but I did hit my left knee on the slip. I didn\'t realize because I managed to hit on a rock in the one small part of my leg that still has no feeling from surgery. Like a 2" square area. What are the odds? So I am unsure if I got an over use injury on the knee, or if it was fork the fall. \n\nBut hiking back to the trailhead, and then later climbing to the hut on Monte Rosa, it seems to work itself out and Mt knee was fine!\n\nMy larger issue was blisters. I practically rubbed the skin off both my heals. And I had issues with infection, spent a lot of money on Italian and french pharmacies. They were helpful though. The rental mountaineering boots were brutal on my feet. \n\nWe unfortunately were not able to attempt our ultimate goal, which was Mont Blanc due to the rock fall danger as a result of the massive heat wave in the alps. Monte Rosa was the plan B. But it was still an amazing trip. \n\nI am now definitely hooked! It was my first time on a glacier and it was absolutely magical. \n\nIn my view, as a civilization humans spend so much time and energy controlling the environments in which we live in. Completely transforming everything for livability and ease of travel and using mach ok ens for everything etc, that there is something so special about being in a place where you are at the whims of the natural world. And glaciers are so fascinating as they are always changing, and we can\'t control them. And we have to go with the mountain and just try to make it there. It\'s really almost a spiritual experience. \n\nWhich also makes things very sad, it is absolutely impossible to climb on glaciers and ignore the effects of climate change. It was a particularly bad year in the Alps and our guides were pretty open about the sad state of the glaciers. \n\nOver all I am hooked and already planning next trips. Will definitely be buying my own mountaineering boots to solve my feet issues.',
 'You are still trying to argue climate change\n\nThis post is about bias in coverage \n\n🤷\u200d♂️',
 'In the days of the old GOP the awareness of climate change was much less.',
 'The best part is half those people are probably protesting about climate change and what not',
 'The year is 2077. America has erased all its national debt. War and injustice is over. Kanye West is elected Supreme Emperor of the World for his revolutionary presidency. Republicans and Democrats agree that West is the only person qualified to lead the global population. The world is a place of technological innovation and social progress. Climate change is reversed. Racism is no more. Kanye and Kid Cudi have made up and are about to release Kids See Ghosts 32. Donda 17 is released. It is unanimously declared the greatest album of all time. The new YZY GAP line clothes the homeless and third-world residents. Pete Davidson is arrested. Drake no longer makes mid music. Kanye’s planet is one of prosperity and kindness. The kids will be alright.',
 'Tell us, what is the damage, how does it happen, why does it happen, and how do we change the thinking of those who cause the injury? FYI, I will be 77 next month and know that consensus means the consensus of climate studies are 97% in agreement that the rate of change is man-caused and accelerating at an exponential rate.',
 "{{The Sixth Extinction}} by Elizabeth Kolbert\n\nTalks about climate change, but also about the myriad other ways that human activity puts the planet's biodiversity at risk, and why that matters.",
 "[**The Divide: A Brief Guide to Global Inequality and its Solutions**](https://www.goodreads.com/book/show/32603498-the-divide)\n\n^(By: Jason Hickel | 368 pages | Published: 2017 | Popular Shelves: economics, non-fiction, politics, history, nonfiction)\n\n&gt;For decades we have been told a story about the divide between rich countries and poor countries.\n&gt;\n&gt;We have been told that development is working: that the global South is catching up to the North, that poverty has been cut in half over the past thirty years, and will be eradicated by 2030. It’s a comforting tale, and one that is endorsed by the world’s most powerful governments and corporations. But is it true?\n&gt;\n&gt;Since 1960, the income gap between the North and South has roughly tripled in size. Today 4.3 billion people, 60 per cent of the world's population, live on less than $5 per day. Some 1 billion live on less than $1 a day. The richest eight people now control the same amount of wealth as the poorest half of the world combined.\n&gt;\n&gt;What is causing this growing divide? We are told that poverty is a natural phenomenon that can be fixed with aid. But in reality it is a political problem: poverty doesn’t just exist, it has been created.\n&gt;\n&gt;Poor countries are poor because they are integrated into the global economic system on unequal terms. Aid only works to hide the deep patterns of wealth extraction that cause poverty and inequality in the first place: rigged trade deals, tax evasion, land grabs and the costs associated with climate change. The Divide tracks the evolution of this system, from the expeditions of Christopher Columbus in the 1490s to the international debt regime, which has allowed a handful of rich countries to effectively control economic policies in the rest of the world.\n&gt;\n&gt;Because poverty is a political problem, it requires political solutions. The Divide offers a range of revelatory answers, but also explains that something much more radical is needed – a revolution in our way of thinking. Drawing on pioneering research, detailed analysis and years of first-hand experience, The Divide is a provocative, urgent and ultimately uplifting account of how the world works, and how it can change.\n\n^(This book has been suggested 57 times)\n\n***\n\n^(63053 books suggested | )[^(I don't feel so good.. )](https://debugger.medium.com/goodreads-is-retiring-its-current-api-and-book-loving-developers-arent-happy-11ed764dd95)^(| )[^(Source)](https://github.com/rodohanna/reddit-goodreads-bot)",
 'Climate change means that the global climate is changing. This means, things won’t be consistent or constant. Without things being consistent, there won’t always be hurricanes, and sometimes there may be a lot. The ones we get are more likely to be extreme due to changing weather patterns.',
 'I legit cannot believe they blamed the increase in heart attacks on climate change. Fear mongering at its finest.',
 'Climate Change',
 '“The idea you turn on a television and see senior senators and congressmen saying, ‘If such and such happens, there’ll be blood in the street,’” Biden said at a speech in Wilkes-Barre, Pennsylvania, about public safety. “Where the hell are we?”\n\n\nPretty funny from the party of:\n\n\n"People are going to die." - Democrats speaking on literally every issue. Climate change, abortion, racial equity, border crisis, etc etc.',
 'Your source is extremely questionable \nElectroverse.co\nNot just because there is a massive Patreon banner right in your face, as soon as you reach the site.\nI’m having an easier time finding articles shooting down articles from Electroverse, than I am finding any sites corroborating any of the information presented to me\n\nhttps://climatefeedback.org/claimreview/electroverse-article-incorrectly-claims-the-sun-is-behind-climate-change/\n\nEdit Electroverse.co \nWhenever you see .co it’s a paid company. This looks like an article produced by OPEC…..',
 'I keep waiting for the people in charge to do something about climate change, and I’m getting more and more nervous the longer we go without any regulations or caps or something, the more nervous I get that NJ will be part of the uninhabitable zone in the USA, like Texas and Arizona.',
 "Taking a bus isn't going to solve climate change ffs 😂",
 'we will have even more ex-vegans soon if I am correct. 2019 was year when people freaked out about climate change the most and many went vegan. so 2024 will probably have a lot of ex-vegans.',
 'I know this thread is old af but..\n\nhttps://www.nbcnews.com/science/science-news/climate-change-increasing-chances-california-megaflood-experts-warn-rcna43079',
 "Climate change.  When I was a kid (10, maybe 11) we used to get quite a bit of snow back home, at least a foot for extended periods.  Since I graduated from high school in '96 it's been a little snow but mostly just rain through winter.",
 "This is a recurring theme in some circles. Highly educated does not mean smart - or competent in areas outside your field. You might remember a few years ago there was some statement signed by some large number of 'scientists' downplaying climate change. Basically none of them were actually climate scientists, meaning their opinion on the subject is worth about as much as that of any lay person.",
 "No, hurricanes are extreme weather.\n\nHurricanes are becoming more frequent.\n\nDoesn't mean they'll literally happen every month!\n\nAnd why do climate change deniers always have to politicize climate change?!",
 'I want to be armed in case of a societal breakdown occurring due to climate change or any other cause.',
 'Climate change says Hello.',
 'Economic implosion is a much more swift and deadly issue than gradual climate change, this is evidenced thoroughly in history it would be unscientific to think otherwise',
 'Pakistan has a major flooding problem right now, both because of rain but also melt from Himalayan glaciers melting due to climate change.\n\nSo... maybe start cricket teams?',
 'Onnengil ivanmarelam koodi bharich mudipikum allel pedikanda climate change , mullaperiyar, yearly flooding… ellam koodi akumbo oru keralam akum.',
 'Because climate change causes *more frequent* extreme weather, NOT *constant*',
 'The urgent need for decisive, uncompromising and ruthless action to force governments and companies to enact policies that will slow climate change.',
 'It was global cooling.\n\nThen global warming.\n\nThen "climate change".\n\nNow, I just call it "weather".',
 "Some of what he said sounds perfectly plausible (the future will absolutely be a shitshow because of climate change, resource scarcity, war, a coming economic crash, etc) but after reading about him on Wikipedia I am more inclined to take his warning with a grain of salt. \n\nIs WW3 on the horizon? Nobody really knows for sure. I mean, it is certainly possible, even likely, especially after Ukraine and Taiwan. But nobody can predict the future with absolute certainty. \n\nNo one controls the world either-- the WEF can try to do so, but it is my belief the Great Reset will ultimately fail, simply because the masses outnumber the elites by millions of people. People don't like being told what to do by oligarchs and moral busybodies who think they know what's best for us, and they also don't like starvation or high food prices (the latter of which has been proven to act as a catalyst for wars and disasters across history). \n\nAll empires rise and fall. The US and WEF will one day fall apart, just like Rome, Persia, Carthage, colonial Europe, and Macedonia before them. Just like the Qing Dynasty, the USSR, Nazi Germany, the Empire of Japan, the Gupta and Angkor Empires, Easter Island, the Maya, the Aztecs, and the Holy Roman Empire.\n\nPrepping and learning survival skills in general is a good piece of advice, methinks. The ongoing climate crisis threatens the survival of humanity and life on Earth so it would be best for those  who know to prepare now before the power goes out, the water runs dry, and the supermarket shelves are emptied a few decades into the future.",
 'The biggest problem is that the main water rights of the Colorado river were divided up after a period of very heavy rain. Do more water is allocated than typically exists even without considering climate change. This is a major problem that needs to be solved. \n\nAt the same time it’s true that the southwest is in the midst of a 22 year drought. If there is less rain people will need more water from the river. If it’s hotter people, plants and animals need more water. Water being less plentiful is going to make it even harder to fix the original problem of over allocation. \n\nBad forestry practices are compounded, by a longer and more severe fire season. It’s hard to do controlled burns if the weather doesn’t allow you to effectively control the blaze. \n\nWe wouldn’t care about climate change if it didn’t threaten our ways of life. Pointing out that human behavior is partly responsible for the risks isn’t a particularly useful insight.',
 'Those are what came with the car but I hear you can swap the mk6 climate gages with only needing to change the trim',
 "Lolling at 'climate change enthusiasts'. What a strange phrase",
 "[https://www.dailywire.com/news/california-asks-residents-to-avoid-charging-electric-vehicles-due-to-blackout-risk-days-after-unveiling-new-gas-car-ban](https://www.dailywire.com/news/california-asks-residents-to-avoid-charging-electric-vehicles-due-to-blackout-risk-days-after-unveiling-new-gas-car-ban)\n\nWell, the truth hurts. And our leaders are full of crap about climate change. Look at this article.\n\n&amp;#x200B;\n\nAnd we have an increase in private jet use. No one talks about that. \n\n[https://compareprivateplanes.com/articles/private-jet-fuel-burn-per-hour](https://compareprivateplanes.com/articles/private-jet-fuel-burn-per-hour)\n\n  \nNo one is asking the rich to stop using their private jets, right? =) You can be a sucker, you can be disgusted, but it'd be better if you were informed.",
 "Meanwhile your other senator is threatening climate change bills to ensure the carried interest loophole doesn't get fixed",
 "most conservatives don't deny climate change but the idea that somehow the suffering we'll have to go through like green tax, switching to ev's ,etc will have a considerable or change worth the efforts when the real reason for climate change are the industries , if anything they're the ones on whom laws and taxes must be implemented not the common folks.",
 "Hard to convince people when too many still don't even believe in climate change.",
 "so you're still contributing to the massive energy consumption done by nfts, literally worsening climate change considering we don't use enough renewable energy",
 'Well, the kids will actually never see the Great Barrier Reef, not because of climate change being detrimental, but because of the shit communitarism being installed, restricting everyone. *"You will own nothing and you will not see the Great Barries Reef"*.',
 "I feel like he was facing the complete ecological collapse, and calculated that the only way for humanity to make it through to the other side, was to accept some hard truths (like that people needed to be sent to die in the colonies doing environmental remediation). And that the Sons of Gilead were the only ones willing to make those drastic moves, rather than sit on their hands and take a wait and see approach (kind of like our own governments' approach to climate change).\n\nOf course he later found out the Gilead doesn't care about humanity, just control.",
 "It would require a phenomenal amount of energy to electrolytically extract enough hydrogen to lower the sea level by an amount meaningful enough to combat climate change. The volume of energy needed would probably exceed global energy production. Not to mention the energy would all need to be produced by zero carbon sources in order to avoid releasing a monumental amount of CO2 into the atmosphere which would more than likely undo any sea level reduction achieved by electrolysis. \n\nThe hydrogen gas would also not be able to be used as fuel, as recombining hydrogen with oxygen in a combustion engine or fuel cell would generate water that would eventually re-enter the oceans and replace the water extracted by electrolysis. The hydrogen would need to be stored, which is very difficult to do as it is extremely flamable and normally only exists in gaseous form. \n\nEssentially, it's not possible because it would be too expensive to generate the required electricity and prohibitively difficult to store the resultant hydrogen and oxygen.",
 "&gt;Of the 3,581 American respondents surveyed online, 54 per cent described climate change as a major threat, including just 22 per cent of those on the political right, compared with 85 per cent of those on the left — a spread of 63 percentage points.\n\n&gt;In Canada, 46 per cent of those on the political right called climate change a major threat, compared with 80 per cent on the left and 71 per cent who identified themselves as belonging to the political centre. \n\nWell at least those Canadians on the political right seem to be less in denial regarding climate change than their American counterparts? Compare 46% (Canada) to 22% (USA). At least the message is getting through here? So there's that.\n\nThe rest of it doesn't surprise me much though. \n\nGood article to read; thanks for posting it.",
 "Are you fucking kidding me, why try to deny climate change? It's fucking changing. Proof: I exist and can fucking feel it.",
 'This article pretty much proves mankind has nothing to do with climate change.   Its been happening for millions of years.',
 'That damn climate change!',
 "I don't know will Kerala exist till then. The climate change it real, and we have a mullaperiyar dam.",
 'With climate change and rising sea levels, we will end up underwater with the bloody French 😡',
 '\\*sigh\\* Not shaming anyone here, not even the writers, but, I\'m the guy who has a totally different first thought about what "Interstellar levels of fucked up" even means.\n\nOverall, I really liked that movie, but... I had to take off my biologist hat, because the Blight is not a thing. Scientifically, the movie was a good depiction of time dilation (and I can forgive the weirdness at the end), but, also scientifically, *nothing* about the Blight makes any sense. Like, just at a ground level, the whole idea of a single disease that affects the whole planet just isn\'t a thing, no disease is gonna be able to affect both the land and the sea, and the sea is *huge* and you can grow *lots of seaweed on it*. \n\n("Kelp, it\'s what\'s for breakfast!" --advertisement for the future Kelp Crunchies cereal according to the Civilopedia of CivIV\'s "Next War" scenario.)\n\nAnd the basic chemistry is either wrong or overstated: you can\'t derive energy from atmospheric nitrogen, like, the electrons don\'t work right for that; so, the idea of a disease that "eats nitrogen for sustenance" is not a thing. Alternatively, "eating nitrogen for sustenance" could mean "fixes its own nitrogen to make proteins", but, that\'s not a way to get energy, and fixing nitrogen is an energy-intensive process, so, it\'s not gonna be an advantage for a parasite such as a disease, over and above just eating the proteins in the crop you\'re parasitizing.\n\nAlso, there was that throwaway line about how the last two plants they could grow anymore were corn and... ***okra???***\n\nThat\'s... not a thing. *At all*. Corn is conceivable as the last crop on earth, only because of how much work we\'ve put into it, but the idea of okra outlasting wheat is just extremely silly. The most-disease-resistant plants aren\'t gonna be water-intensive tropical crops like okra (the vast majority of diseases require water for survival), they\'re gonna be drought-resistant desert crops like tepary bean or chickpea or any of the millets domesticated in West Africa that can survive the blight by attacking it while it\'s desiccated and vulnerable, whatever "it" is.\n\nInterstellar as a movie would\'ve made a helluva lot more sense if they\'d had climate change instead of a crop disease as the antagonist... and I wouldn\'t be surprised if a lot of people saw the images and just sort of assumed it was climate change, even though that\'s not technically what was in the script.',
 'How many of them believe in climate change?',
 'There are reasons being an airline pilot might not be suitable for a Jew, but Climate Change is not one of them.',
 'Ahh yes... like those scientists before who put blinders on and ignored the existential threats of:\n\n* [global cooling](https://www.nationalreview.com/planet-gore/remembrance-panics-past-ed-craig/)\n* [Antarctica immigration crisis](https://www.telegraph.co.uk/news/earth/earthnews/3353247/Climate-change-study-predicts-refugees-fleeing-into-Antarctica.html)\n* [no more snow](https://www.independent.co.uk/climate-change/news/steve-connor-don-t-believe-the-hype-over-climate-headlines-2180195.html)\n* [mass species extinction](https://en.wikipedia.org/wiki/Sidney_Dillon_Ripley)\n* [civilization will end by 2000](https://en.wikipedia.org/wiki/George_Wald)\n* etc, etc, etc...\n\n[https://fee.org/articles/18-spectacularly-wrong-predictions-made-around-the-time-of-the-first-earth-day-in-1970-expect-more-this-year/](https://fee.org/articles/18-spectacularly-wrong-predictions-made-around-the-time-of-the-first-earth-day-in-1970-expect-more-this-year/)',
 'The Pirates of Dark Water. I feel not knowing that there is an ending has cursed humanity to climate change.',
 '&gt;Animal agriculture is the second largest contributor to human-made greenhouse gas (GHG) emissions after fossil fuels and is a leading cause of deforestation, water and air pollution and biodiversity loss.\n\n[Source](https://climatenexus.org/climate-issues/food/animal-agricultures-impact-on-climate-change/)\n\nalso i wont respond',
 '\nClimate change has a variety of consequences, and it’s a perfect example of a few people actively screwing over billions... so why do people never want to discuss it, get tense when talking about it, and shut off or don’t talk about taking action? Like, even people I know that believe in it are apathetic and hopeless. So how do we support the idea something can be done and people can do something besides wait for the next named heat wave or mass crop failure? and then what’s something manageable with a 40 hour work week and a budget that can be done to help?\n\nFtfy',
 "I'm not following this... So you don't think climate change is a problem? Or you do? And who said we can grow concrete?\n\nHaha, sorry.. tis early still.",
 'thank you climate change',
 "Maybe because longer and more intensive heatwaves (or weather in general) have been predicted as a consequence of climate change for over a decade? Seems relevant to include in an article about the highest temperature ever in Britain during a very intensive heatwave all across Europe. Heatwaves/extreme weather have been increasing for years. It's clearly part of a trent.\n\nMeanwhile your example about Greenland is an incidental occurence, just like the graph you shared shows. Greenland has been losing unfathomable amounts of ice on average yearly and one of the highest rising average temperatures on earth. What you share is thus not part of a trent and it would be wrong to report it as such.",
 'its meant to advertise dystopian climate change',
 "Lol if half of the population would look at truth rather than listen to emotions they would know that climate change is fraud and just a way to explain god's way of doing unnatural things. Look at statistics. Climate change has been happening since beginning of time. Even without industrialization. Btw Vulcan farts more of the so called gas than the whole population in many years. So your telling god to stop destroying his own earth ?",
 "We are, but I'd put a lot of the blame on brainwashing. Read the news today and you won't be **allowed** to reach the same conclusions as above. You might figure it out on yourself though, by being curious about a very watered down article talking about IPCC's reports, and (actually) researching it yourself.\n\nLook at Russia. The people are brainwashed because they, basically like Japan, only read news and watch TV in their own language. And while Japan isn't technically as *evil*, I'd call them naive of climate change issues simply because it isn't really talked about in the news over there. I mean, at least **much less** than in even the west.\n\nKnowing is half the battle?",
 'Climate change is most likely to blame',
 'Must have been car exhaust fumes and climate change.',
 'Oh sure, that\'s pretty well known.  The Old Kingdom was heavily stressed by the general Bronze Age Collapse (basically around 1170 BC there was a huge crisis across the Mediterranean world, possibly driven by climate change, resulting in wars and chaos and whatnot, and then some local Dark Ages).  The New Kingdom survived for a while, but badly wounded.  But eventually there was a a string of bad harvests and incompetent Pharaohs and finally Egypt broke up into smaller states.\n\nThen after some time Egypt was reunited -- but not under Egyptian rulers.  The new Pharaohs were a dynasty descended from barbarian invaders: the Meshwesh, a tribe of Berbers from the deserts of Libya.  \nAfter a century or so the Meshwesh kingdom fell apart.  Then Egypt got reunited *again* by *another* set of barbarian conquerors -- Nubians this time, Africans from further up the Nile. They ruled Egypt pretty well for a while, but eventually the Nubians got defeated by the Assyrians.  Egypt broke up again, then was partially vassalized by Assyria, then was briefly reunited under one last dynasty of Egyptian pharaohs, then was conquered by Persia.\n\nNow, on one hand, all this took generations.  Like, those last few paragraphs covered about five hundred years.  So people lived whole lifetimes under Meshwesh or Nubian dynasties.  But on the other hand, all this happened pretty fast compared to the Old, Middle, and New Kingdoms, which were stable empires that lasted for many centuries.  So all these -- breakups, Meshwesh, Nubians, Assyrians -- get swept together into the "Third Intermediate" and "Late" periods.  Basically these are periods when Egypt mostly either wasn\'t united, or if it was, was united under foreign rule.',
 'Your concern is that we should also be freaking out about this because it’s indicative of the climate change models? Sure I guess. It is concerning. The media has to keep it simple though. Presenting counter-intuitive, albeit true information, doesn’t grab attention in the same way.',
 "Critical thinking in the media -&gt; think critically about climate alarmism in the media -&gt; climate change isn't a big deal",
 'This post is how weather is reported.\n\nNot about "understanding climate change"',
 "les use of fossil fuels is beter in the fight against climate change, it's the only benefit imo, next to giving ukraine a fighting chance.",
 "Not to be That Guy but I feel like I'd rather just die on earth from global warming and climate change than go to Mars.",
 '[wrong](https://www.climate.gov/news-features/understanding-climate/climate-change-global-temperature). Measurably, obviously wrong.',
 'God damn with so many other important issues such as climate change, abortion rights, lack of healthcare, gun control, poverty, and many more why are trans rights such a huge issue? I support trans rights but I’m not focused on such a small issue that is sure to over shadow actual important issues that still affect trans individuals.',
 "This isn't the point though. The point is that people will complain anyway. Doesn't matter if this is a precedent or whatever, the fact is that people are never happy.\n\nThere's literally no way to know if a change of DLC policy is good or not, because people will hate it anyway.\n\nIt's like if no matter what we did, coal, nuclear or renewables, climate change was still happening. At some point then yeah the industry would simply not bother.\n\nIf this community was half as smart as it claimed to be, it would be able to have difference reactions depending on what happens.",
 'This happens every year with Reading and Leeds. Seems more ironic this year since the festival had a collaboration with No Music On A Dead Planet which encourages music fans to come together and fight against climate change. \n\nThey also had an eco campsite this year. \n\nThe festival organisers seems to do very little apart from putting up a few signs saying please take home your tents.',
 'It wasn’t an article. \n\nIt was the entire screeching hyena media going  easy about how we are all going to die because it was hot at the airport\n\nIt too was just weather \n\nhttps://www.nbcnews.com/news/europe/britain-hottest-day-on-record-hot-sunlight-heat-soaring-temperatures-rcna38840\n\nhttps://www.msn.com/en-us/weather/topstories/record-breaking-heat-wave-in-europe-will-be-the-norm-by-2035-analysis-shows/ar-AA116aIh\n\nhttps://www.nbcbayarea.com/news/local/climate-in-crisis/london-record-heat-climate-change/2948903/',
 "Your right. It is foolish and selfish not to be vaccinated.\n\nNature gives rules to live by with consequences.  Climate change is another.  What is going to happen if we don't stop burning fossil fuels? When is it going to happen? What do you think?\n\nNatural selection for intelligence. Increasing the planet IQ.",
 'Greta Thumberg says how dare you to the crowd and lectures them on climate change for 2 hours.',
 'More like too many people’s income/fortune depends on denying climate change.',
 'Almost as if we are discussing the mass Suffering of hundreds of millions of people due to climate change and your concern is border control rather than helping people in need. \n\nCrazy how that almost sounds like you care more about lines on a map than actual human lives.',
 'Jonny’s radio before National Anthem catches a Lex Friedman podcast on climate change and Thom makes everyone listen to it for 3 hours. Then they leave.',
 'Let me guess, winter vagina and climate change.',
 'Three days is not a long time if its only normal routine maintenance. The war is just politics and people of Ukraine and Russia suffer because of politicians. Eventually Putin will move on and Russia will still be a resource rich country.  Russia and US need to make peace and stop the war. Perhaps stopping Nord 2 was a mistake; it could have been backup supply line. \n\nWorld needs to tackle  problems like climate change urgently but politicians in so many countries are focused on  attaining  and maintaining power.  Water levels are dropping in too many locations and Amazon(Brazil, not Bezos) is being destroyed.  Biden should work with Russia, China, India and EU to bring about peace and tackle more important issues.',
 'Bernie considers climate crisis the most essential change to national security. He opposed the establishment of the Dakota Access Pipeline on the grounds that it will have a drastic result on the climate.\n___\nBernie Bot -Bernie_Bot_03',
 "I'll answer it when you write a detailed plan to address climate change which involves the current levels of car use.",
 'Climate Change',
 'I live in MN, we have a lot of collapse and climate change aware people here',
 'Ridiculous issues such as climate change and student loan forgiveness?\n\nAnd inflation reduction?',
 'Interesting to note climate change impacts in Colorado then blithely add that one of the experts is moving to Phoenix.',
 '"Scientists find breakthrough fusion battery that cures cancer and reverses climate change!"  \nYeah someone wrote a paper- but nobody confirmed it was a *good* paper',
 "Ok j'ai trouvé la source : [https://www.oecd.org/climate-change/international-attitudes-toward-climate-policies/France.pdf](https://www.oecd.org/climate-change/international-attitudes-toward-climate-policies/France.pdf) \n\nLà c'est pour la France, mais on peut accéder à chaque pays individuellement en changeant le pays dans le lien !",
 'Well we have the stats, excess deaths are way up, around half is due to covid and half is non covid. Why would non covid deaths be up so much? Must be climate change I guess.',
 "&gt; a person who said not to interact if you didn't believe in climate change (so I asked her what her favorite policy to reduce emissions was)\n\nsend ~~nudes~~ white papers",
 'NOT RELATED TO CLIMATE CHANGE! The monsoons come every year to this area!!! \n\nhttps://tribune.com.pk/story/590504/history-of-disaster-floods-affecting-lives-economy-since-independence\n\nClowns "connecting the dots" to NOTHING! Sheesh!',
 'Nothing to do with "climate change". This area has been flooding for centuries. What has changed is more people building in the flood plain.\n\nNote that this list only starts in the 1950\'s because Pakistan became it\'s own country 1947.\n\nhttps://tribune.com.pk/story/590504/history-of-disaster-floods-affecting-lives-economy-since-independence',
 'Yes, I can imagine that any Pacific nation erased by climate change will probably be recognised a a sovereign entity with no territory, as the Military Order of Malta is now. They would probably work out some special arrangement for their citizens with New Zealand, similar to what Tokelau and Niue have.',
 '“CLIMATE CHANGE IS REAL!”',
 '- The change is happening faster. \n- E.G. The unintended consequences of Social Media and Climate Change.',
 "People build in flood planes in the US too because the land is cheaper. There I'm guessing they do it because they have better access to water and fertile farmland. Climate change means 100 year floods will be happening every few years now.",
 'Climate change',
 'Climate change',
 "Maybe if you live in the US? Coverage I've seen from major outlets in my region relate all of this back to anthropomorphic climate change. \n\nAlso, you should specify that you mean emission per capita.",
 'Climate change denial... what a word. So i say climate change is being weponised and you claim i deny climate change? lmfao. The word smiths are really taking from the 1984 books newspeak arent they.',
 'Ironically, they\'re all like "AOC is not qualified because shes was a bartender", yet bat not an eyelash at all the punditd sounding off on climate change, wage inequality, economics, etc with nothing but their posterior',
 'Attended by the same people who protest climate change and glue themselves to the roads. I’ve helped clean up this site and it is disgusting what gets left behind.',
 'Tick infestations are another class of indicators that climatologists are researching as the world grapples with climate change and adaptation or lack thereof.',
 'Are you reading some articles in which the catastrophic weather, floods included, isn’t being called climate change? It’s a flood, yes, *and then the cause is described*… or are you judging all of this based on headlines?',
 'Climate change.',
 "dams are in no danger of collapsing--thats a BS story put out by Govt\n\nGovt just says dams are full--where's the proof\n\nbesides dams are never that full\n\neven of they were--there are ways of slowly releasing the water without dumping it into rivers\n\nthis is deliberate sabotage to sell the climate change lie &amp; high food prices",
 '{{Climate Change: A Very Short Introduction}}\n\n{{Global Warming: A Very Short Introduction}}\n\n{{Climate Change: What Everyone Needs to Know}}\n\nThere are more in both of these series, on different topics. Accessible overviews/summaries of large topics.',
 'Do my best to avert climate change getting as bad as it has.',
 'I mean if you follow the money, the sources for climate change denial are invariably funded by the major players in the hydrocarbon industry.',
 'For the past two years, the Netflix/Attenborough ruse to blame climate change for walruses falling from a high cliff to their deaths seemed like a silly PR stunt.\n\nBut it appears the film’s real purpose was to convince a far more important audience than paying Netflix customers that a global ‘climate emergency’ was going on: The elite influencers and world leaders who attended the World Economic Forum in Davos in January 2019, where the dying walrus film clip was introduced and interpreted to the audience by Sir David Attenborough himself.\n\nIn the spring of 2019, Netflix released its ‘Our Planet’ documentary that included a clip of walruses falling from a high cliff in Siberia to their deaths on jagged rocks below, which Sir David Attenborough’s narration told viewers was due to lack of sea ice caused by climate change.\n\nIt moved people to tears, as it was meant to do.\n\nOf course, we now know that already-fat polar bears deliberately frightened those walrus over the edge – using a particularly effective, site-specific hunting strategy that avoided fighting the dangerous beasts head-on – and it all had nothing to do with walrus struggling desperately to survive in a warming world.\n\nThis explanation not only makes biological sense, but it’s also backed by solid evidence. Since it is now clear that the truth would eventually come out when the BBC aired its own documentary in November 2019 showing bears driving walrus to their deaths, the Netflix/Attenborough ruse seemed like a silly PR stunt.\n\nNow I know otherwise.\n\nAttenborough must have known after seeing the footage he was to narrate that the walrus death sequence was a visual even better than a starving polar bear for convincing people that a human-caused climate emergency was at hand. He knew he had dynamite and wasn’t afraid to use it.\n\nThe possible significance of the video being played at the Davos meeting, as Kochnev described it, went over my head as anything of significance when I read it in April 2019.\n\nAs I said, I’d never been interested in politics. Even when it was clear the film had had such a profound emotional impact on viewers yet was a blatant lie, it seemed only to be the usual sort of thing environmentalists would do to get attention.\n\nBut now, watching the world’s covid responses morph into a sinister-sounding masterplan coming out of Davos, this manipulation of polar bear science has taken on new significance.',
 "I don't live in your country and cars should be banned for every reason, climate change most of all, in addition to the absurd amount of road deaths",
 'This woman is certifiable. \n\nhttps://time.com/6209432/climate-change-where-we-will-live/\n\nEverything she says is wrong.',
 'Maybe it’s that climate change affecting vax takers?',
 "Funny how you don't even adress what he said, you just insult and give a nonsensical example that says nothing.\n\nEvery year more ice melts in Greenland and it has one of the, if not the, fastest rising average temperatures on earth. You focus on singular instances which is just weather. Just like a warm day in London is just weather but if several years in a row there are higher average temperatures and longer heat spells, it indicates climate change. It's about the yearly average trends.\n\nEDIT: I see your username and account now, waste of time.",
 'We are comparing why sometimes it’s Climate Change and sometimes it’s weather\n\nAt no point did I make the case for what you are claiming to prove\n\n🤷\u200d♂️',
 '&gt; Scarcity will increase as climate change destabilizes current systems of production.\n\nWe already have an excess of water, and food produced on this planet.\n\nWe have an excess of housing in the US, I can attest.\n\nIn the US, we have the ability, not the desire, to provide health care for everyone.\n\nSeems to me, we are damned near the "post scarcity" society, today.  Which is a big reason why the contradictions of capitalism are becoming more, and more widely seen and felt.',
 'You missed the line about this plant also makes batteries for "HEV" which are ICE?\n\nToyota does not get enough hate for their climate damage. They were so good with the Prius, but lately have been in the top three against climate change action, just behind Exxon and Chevron.',
 'Well damn. Spot on. Climate change is a global issue.',
 "Yes, but we're talking about man-made climate change.\n\nGod promised not to destroy the earth. He didn't promise to prevent us from harming ourselves.",
 "That's kinda because climate is made up of a little bit of everything. A single flood or drought isn't climate change. But more floods and droughts, and more EXTREME floods and droughts, compared to previous years and consistently rising, is part of climate change.",
 'No silly\n\nWhen it’s a hot day in London it’s climate change\n\nWhen Greenland gains ice its weather\n\nIt’s why you didn’t do this little speech anywhere when the hyenas were screaming about that hot day in July 🤷\u200d♂️',
 "She's been very vocal about Climate Change for quite a while now, so I would say it was less of finding the right voice for this and more of collabing with someone actually interested and vocal about the subject.",
 'Then how do you bring up the issue of climate change if no one will acknowledge it exists and think about it what are some tactics you can think of',
 'This will only get more and more prevalent as oil companies pay right wing grifters to push anything that’s good for the environment and doesn’t provide profits for oil and gas companies as being part of the culture war, if not outright denying the existence of climate change',
 'Considering nearly the entirety of "climate change" research is the generation of predictive models based off INCREDIBLY short data collection perdiods compared to the time frames they\'re attempting to generate...I think seeing variation is OK.',
 "I mean, the fact that there are SOME things that DO demand a majority consensus - civilization-scale threats like climate change, for example - means that even if we compartmentalize some things, there's still a need for global governance in some capacity.\n\nI'd rather have a soviet-style or Chinese council system for that reason. Local issues can easily be decided locally, and industry issues can be decided at the industry-level through industry councils/unions, but that representation continues in a tiered system all the way up through councils at every scale, to global.\n\nYou kind of need a government big enough to address your biggest issues, at a minimum.",
 "What is anyone's retirement plan? Mine is to die to climate change before I can imagine thinking about retiring because cost of living keeps going up but wage doesn't",
 'When can I get the climate change shot',
 '&gt; The Grapevine-Colleyville Independent School District near Dallas, Texas, recently adopted guidelines that ***prohibit acknowledging the existence of*** transgender or nonbinary people\n\nAs right-wingers and Christians take over more and more schools, expect the pattern of "*prohibit acknowledging the existence of _________*" to be applied to more and more things: \n\n1. prohibit acknowledging the existence of abnormal weather and climate change\n2. prohibit acknowledging the existence of atheists\n3. prohibit acknowledging the existence of non-Christian religions\n4. prohibit acknowledging the existence of deadly pandemics\n5. prohibit acknowledging the existence of government corruption\n6. prohibit acknowledging the existence of certain rights\n\nEtc...\n\n[Literally the authoritarian equivalent of this](https://c.tenor.com/FE849oomI04AAAAC/cant-hear-you-lalala.gif)',
 'Oh yeah, its not as bad as the Black Death or a world war. Lucky.\nWill be soon though, thanks to climate change.',
 'Legal does not mean ethical. Tabacco, alcohol, walmart employees on food stamps, fast food, literally all the overseas manufacturing of the items you buy, amazon warehouses, the pollution and climate change caused by legal businesses, etc.',
 'We’re to treat others how we’d want to be treated.  When we enact policies of green policy we destroy 3rd world nations land by strip mining them of lithium cobalt etc and pollute their water.  The cow fart issue of eating less meat said the world could impact climate change by 14%.  When you look at the US that is 2% .  It’s the 3rd world countries that have the highest impact and we should work to get them to use more sustainable energy then wood and diesel.  Coal is a start then natural gas but nuclear would be the way to go as it has the lowest impact on carbon footprint.  Recent permaculture farms in the US and Australia are showing a net positive on carbon sequestration.',
 'Well, punctuation would be a good start!\n\nI think it\'s because people don\'t want to be confronted with danger. And when they do want to be confronted with danger, they want to be able to take swift and decisive action to prevent it. But climate change needs massive, structural changes that individuals cannot implement easily. Furthermore, people don\'t like having to change their comfortable way of life.\n\nIn essence, there\'s a psychological issue, where people don\'t like scary things; there\'s another psychological issue where climate change is too nebulous a problem to be easily "grokked" by the individual; and there\'s a cultural issue in that people don\'t want to give up their lawns and cars and airplane holidays.\n\nEven bigger cultural issue, people don\'t want to give up capitalism.',
 'Thank you for the detailed response. It seems that even though one might not be tackling specific climate problems one can still effect change at a high level. \n\nYou gave me a lot to consider; I appreciate that!',
 'Are you equating the construction of the Tower of Babel with technological/social change designed to curb climate change?',
 'And the governor won’t even use the term climate change.',
 'Sorry, don’t blame me, blame climate change.',
 'Might help with climate change.',
 'Now do vaccines, masks, and climate change, doofus.',
 'Guess what, sounds like there is mismanagement, possibly climate change and other issues.\n\nOh, wait, something that is NOT EVIDENCE AGAINST CLIMATE CHANGE? That is not allowed here. /s\n\nNah, seriously, consider this: People are not gonna care about that kinda shit until it hits themselves. It is like that with everything. "Why should I care about a drought somewhere I could not even point at on a map?" disturbingly quickly turns into "I been there last year, that ain\'t far away". Oughta consider as well, that mismanagement leading to this might be caused by corruption/ lobbyism and simply a lack of belief in facts (whether man made climate change lead to that matters not; just look at the statistics and tell how a river\'s water level is evolving, you might be able to make a statement about how the future looks and what to do or at least what not to do and what to expect). \n\nBecause nothing that big comes without warning.',
 'We will have to rapidly upgrade our infrastructure to adapt to climate change, I have no problem putting a deadline on that.',
 'Lol, yeah… cLiMaTe ChAnGe',
 'In the late 1970s, a pick-up truck drove me and friends to our rented ice fishing huts on Put-in-Bay.  I remember catching a perch and throwing it out on the ice to freeze.  Then I watched a gull swoop down to eat it, but the fish already had frozen so hard the gull couldn\'t swallow it.  I gladly would have fed that starving bird, and I often think of that event when I consider the consequences of climate change on world food supplies.\n\n(The popular media hasn\'t yet picked up on this story:\n\n[https://www.foodbeverageinsider.com/ingredients/worst-drought-millennium-creates-tomato-shortage](https://www.foodbeverageinsider.com/ingredients/worst-drought-millennium-creates-tomato-shortage)\n\nBTW, it\'s not a drought.  It\'s aridification.)\n\nNow, air boats are used for "ice" fishing on the western basin.',
 "Marijuana, Climate Change, Building Code Regulations, and Abortion are the ones I know for sure he's pushed back on frequently",
 'Climate change strikes again!',
 '&gt;Climate change is a topic I am deeply interested in, and I do hours of research nearly every day.  \n\nThen you should have known immediately that this paper is bunk.  \n\n&gt; I am asking for evidence and I remain open to any  \n\nThen you should have immediately recognized that the paper you submitted *does not supply any evidence for their claims* and known immediately that it was bunk.  \n\n&gt;this is part of my research  \n\nYou\'ve shown zero evidence of "research".  All you\'ve done is post the paper and demand others to look things up *for* you.  Which they\'ve done multiple times now, and then you try to criticize everything that goes against this paper\'s claims.    \n\n&gt;I also do not feel I am "arguing in support of this paper,"  \n\nYou are demanding to see evidence *against* the paper, which means your starting point is "the paper is true and correct".  You are also dismissing any and all evidence that the paper is biased and incorrect.  \n\nThe fact that you\'re still here "jUsT tRyInG tO gEnErAtE dIsCuSsIoN gUyS" is pretty telling.',
 "&gt;Not all scientists believe in Climate Change.\n\nI wouldn't trust the tenth dentist, honestly.",
 "It looks like you shared an AMP link. These should load faster, but AMP is controversial because of [concerns over privacy and the Open Web](https://www.reddit.com/r/AmputatorBot/comments/ehrq3z/why_did_i_build_amputatorbot).\n\nMaybe check out **the canonical page** instead: **[https://www.independent.co.uk/climate-change/news/climate-change-science-deniers-boris-johnson-environment-leak-a9094631.html](https://www.independent.co.uk/climate-change/news/climate-change-science-deniers-boris-johnson-environment-leak-a9094631.html)**\n\n*****\n\n ^(I'm a bot | )[^(Why &amp; About)](https://www.reddit.com/r/AmputatorBot/comments/ehrq3z/why_did_i_build_amputatorbot)^( | )[^(Summon: u/AmputatorBot)](https://www.reddit.com/r/AmputatorBot/comments/cchly3/you_can_now_summon_amputatorbot/)",
 '[one of the other many articles](https://www.bbc.com/worklife/article/20220225-how-climate-change-is-re-shaping-the-way-gen-z-works)\n\nLike i said, just one of many articles.',
 'Also the irony of being a gas company with climate change being the biggest extreme weather event.',
 '&gt;You believe the claims in this (non-scientific) paper without any "supporting evidence"\n\nI never said I believed the claims of this document - I merely said it caught my attention.\n\nClimate change is a topic I am deeply interested in, and I do hours of research nearly every day.\n\n&gt;you dismiss any and all evidence that the claims are incorrect and the signees are biased\n\nI have not dismissed any evidence - I am asking for evidence and I remain open to any.\n\n&gt;This has all been provided to you. If you actually "did your own research" like you claim, then you wouldn\'t continue to be here arguing in support of this paper.\n\nAgain, this is part of my research. "Experts told me" is not evidence - it\'s a claim - which can guide research, but is not considered solid evidence.\n\nI also do not feel I am "arguing in support of this paper," as I am simply asking for perspectives and then asking for the supporting data behind those perspectives.\n\nMy opinion on the veracity of this paper has never been stated because I am undecided.',
 'Bernie considers global warming the most essential change to national security. He opposed the establishment of the Dakota Access Pipeline on the grounds that it will have a big effect on the climate.\n___\nBernie Bot -Bernie_Bot_02',
 '&gt;The answer is no. The moment we parse out blame is the moment we are not finding good solutions and hoping to shirk responsibility of dedication required. It will take a GLOBAL effort to remedy climate change. Everyone should be helping Pakistan, not just the US.\n\nThis is contradictory. My question was "should the US be more accepting of climate refugees." If your answer to that is no, why would you then say "but everyone has a responsibility to help." Shouldn\'t part of that responsibility mean being more accepting of climate refugees\n\n&gt;You are assigning bias on this data to prove a correlation. By the data you provided alone, the insights we should be considering are things like:-More people per car (carpooling or public transportation)-Eat less-Use less water for non essential things (sorry guys your grass isn\'t one of them)\n\nThat\'s not a logical connection to the information I provided about the US being one of the heaviest polluters, with a military that pollutes more than several countries. The Middle East does not "blow past" the US or China based on proportional per capita metrics.\n\n&gt;the largest food exporter in the world\n\nThe US is not the largest food exporter in the world, at least not by the link you provided. The link you gave says that "The Americas" are the largest net food exporters, specifically. Countries in the Americas like Mexico, Brazil, Columbia, Peru, Guatemala, etc., are the largest exporters of food, not the US. **The US is actually one of the larger importers, not exporters.**',
 'Give us more money to change the climate. I hope truss rolls up his letter and smokes it',
 'What gets me is the majority of regular people who claim to care about climate change but then drive a large vehicle for every little trip.\n\nThe celebs and politicians get me too, tbf.',
 "Yes check with your friends about their opinions about issues and if they're against this fascist rise and climate change and theological law make sure they vote. Get them all together and drive them if you have to. Organize it all and make it happen.",
 'The iPhone replaced it because of adaptability of having more screen. However try to type out a message with your iPhone under a table vs with a physical keyboard without looking at it. Completely different experience especially when adapted to our scenario of try to change climate control while driving. The stupid thing in fords case with the lightning is the have the xlt and lower trims with the 12 inch screen and physical buttons. The 15in vertical screen really doesn’t give you more usability than the 12 inch horizontal. And if I were to buy a Lightning I’d like the physical button option.',
 'He’s also a climate change denier.',
 'Seattleite here. Increased summer temps, heat dome, wildfire smoke, lower quality produce in stores… Global Climate Change is the major event.',
 'https://fortune.com/2016/07/11/bernie-sanders-climate-change-democratic-platform/',
 '&gt;I\'m open to accepting that these are lies, but I need to see compelling evidence to support such a position  \n\nYou believe the claims in this (*non-scientific*) paper without *any* "supporting evidence", you claim you "do your own research" on a paper that has absolutely zero cited studies or scientific works, and then you dismiss any and all evidence that the claims are incorrect and the signees are biased *which multiple people have now given you*.  \n\nThe founder of CLINTEL literally made and makes his money from selling research to oil companies.  A significant portion of the signees either work directly for or have some form of financial ties to the fossil fuel industry, specifically mining companies for many.  A significant portion of the *remaining* signees are not even in a field related to climate study.  All of the claims made in the document have been proven to be false or misleading anti-climate-change tropes.  \n\nThis has all been provided to you.  If you *actually* "did your own research" like you claim, then you wouldn\'t continue to be here arguing in support of this paper.',
 'Vermont, Maine, and New Hampshire are possible choices.  \n\nStay away from oceanfront communities because of sea level and climate change issues.  Interior Maine can have insane snowfalls.\n\nEdit:  [https://www.pewresearch.org/fact-tank/2016/02/29/how-religious-is-your-state/](https://www.pewresearch.org/fact-tank/2016/02/29/how-religious-is-your-state/?state=alabama)',
 "I've posted a link to nasa on here, that's a credible source as it gets. They conclude climate change is real and caused by humans",
 'You’re the one who missed the point. \nClimate change is always happening. Warmer than colder.',
 "they need us to believe the climate change propaganda so that we will comply with the carbon tax\n\nand yet, we are carbon based lifeforms, we exhale CO2, what are they going to do, tax the air we breathe for the CO2 in our exhale emissions? it's absurd and normies are too delinquent to see the play\n\nlove the question posed in the title, i know people on my fakebook who would jump down my throat if i asked them a question like that, and yet that question is precisely the kind of critical analysis that they need to come to terms with in order to see past the play, but because they can't see past it they take questions like to be abhorrent to them",
 'Yeah, I kind of feel that humanity is that little penguin, and this is a metaphor for climate change. 🤞',
 'I do hate Putin. Unfortunately, he’s not the cause of climate change. This made me laugh, fellow Redditor. Well done.',
 'The climate change one is a good example.  Because while being a person "who sees both sides" about something like that may seem neutral, not doing anything about climate change is actively causing harm.  So it\'s an issue, among others, where I don\'t think "taking both sides" is a morally justifiable position, as neutral as it sounds on the surface.',
 'The atmosphere certainly cares where the carbon comes from. Carbon from wood is just part of the life cycle of trees and vegetation. Every season plants release carbon as they die and decompose (or burn) and consume carbon as they grow. It’s like rainfall - water evaporates from the seas the falls back as rain. The total amount of water and carbon in the biosphere remain the same in both scenarios. Carbon released from the extraction or burning of fossil fuels is quite literally “new” carbon into the biosphere. This is what is increasing the atmospheric concentrations of carbon. That’s what is causing climate change.\n\nSolar arrays cost anywhere from $10k to $20k and pay for themselves in 10 years.',
 'Is this another one of those “I don’t understand the concept of climate change” posts? \n\nIt literally results in more ice in the up season and less in the down. There are several ways to measure this and none are perfect, but measuring input and output is one of the ways. \n\n&gt; In addition, warmer ocean waters weaken the floating ice shelves that form in key places along the coast, especially in Antarctica. These ice shelves help to hold back the flow of ice on land, so as ice shelves shrink or disappear, ice from the ice sheet can flow into the ocean more quickly. Some of the losses are offset, because warmer air allows very cold interior regions in Greenland and Antarctica to receive more snowfall. \n\n&gt;',
 "Blackrock is on the ban list... You're telling me Blackrock doesn't prioritize ROI? This is clearly a political move and nothing to do with finance. If you're  against climate change or for human rights, Texas doesn't want your business anymore, apparently.",
 '- Get out and vote for the party that is trying to fight back against climate change (and Fascism).\n- Vote like your life depends on it…\n- Not planning to vote in mid-terms? Then you cannot complain about our flawed system.',
 "Not really, sure climate change, but I'm up north and always dress like it's frigid outside",
 'https://www.bbc.com/future/article/20220504-why-the-wrong-people-are-blamed-for-climate-change',
 'Does it? Should I be discussing climate change instead',
 "This. Climate change is unfortunately VERY real. Likely caused by Putin, who is doing everything he can to stir things up and discredit the Biden Administration and his son Hunter.\n\nI don't know how I'm going to cope with this, other than getting vaxxed, I don't know what the right thing to do even is.\n\nI'm looking for help setting up a Go Fund me, to help Climate Change, Hunter Biden, promote vaccinations, and fight Putin. Are you in?",
 "Did Bill Gates ever even say that climate change caused obesity in children? I would say that heavy industrialisation in urban areas may be leading to both, maybe that's what they're trying to take out of context?",
 'If you are worried about that, have you made other changes in your life that can make a larger impact on climate change?\n\nDo you drive an electric vehicle? Do you maintain a vegan diet? Change your keep your AC and heater within eco-friendly parameters?\n\nOverall airlines contribute about 2% to overall climate change, if you are worried about making an impact do something mentioned above, it will have a much greater impact.',
 'Go read his comments about how "based" Trump is and how climate change isn\'t real then maybe decide if that\'s the ship you want jump on.',
 'Please, Everybody Please listen to me! Stop driving cars! Stop turning your heating on in winter! Stop flying! We need to kill climate change straight away! Humans are the sole reason the world is dying! Please just be better 🙏🏼',
 "&gt;Climate change can't wait another decade for us to switch\n\nBut you just said they take 15 years to build...",
 'Because the term [Milquetoast](https://www.merriam-webster.com/dictionary/milquetoast) has fallen out of use. Not taking a side on an issue, especially when it comes to oppressor versus victim, is a way of trying to avoid all conflict or choice.\n\nIf I dont say "Climate change is something caused by humanity and we should try to stop or undo our impact" and instead say "Well I\'m sure both sides have a valid point" then I never have to think. I never have a chance of being right or wrong. I can always say that I wasn\'t "wrong". \n\nWhich is really shit. The only way we grow and better ourselves is by testing our assumptions and opinions. If I think cajun food isn\'t good and then go try cajun food and find out it\'s amazing then I\'ve grown as a person. If I avoid the subject all together I\'m not "wrong" but I dont grow.',
 'Most of the deforestation is being done by large corporate agribusinesses with international financing and that not only irrevocably damages biodiversity (extinction is forever) and exacerbates climate change, it also harms the indigenous people already living there; why isn\'t there the same level of concern for those people? The ranchers are flat out murdering them, including burning an eight year old alive a few years ago. \n\nBut the "let them destroy it" crowd always profess concern for "starving families." It\'s at best concern trolling. ((Nor does it logically follow that because European settlers laid waste to the natural world we (as in, human beings) should continue making the same mistakes))\n\nAs far as Bezos (or any billionaire for that matter) sure I\'d love it if they swooped it and bought vast tracks of land, but that said Bezos doesn\'t\' control the stories WaPo researches and publish.)',
 "Google has changed the way it calculates the climate impact of air travel in a way that dramatically undercounts key factors in aviation's contribution to climate change.\n\nThe company started excluding all global warming impacts of flying besides carbon dioxide from its climate calculator tool as of July. And that might not sound like a big change, but as a result, estimates of carbon emissions per passenger are now drastically lower than they were before the change.",
 "Still. Until it happens to their communities, Americans just won't care. And even then, their republican politicians will convince them it's somehow not climate change or at least shouldn't be acted upon.",
 'But I thought Big Climate was naming storms willy nilly to convince people that climate change is real!',
 "&gt; Ohio winters are noticeably diminished with each passing decade.\n\n1 or 2 in this last decade were pretty rough in Cleveland, but I'm coming up on 9 hears here, and I can tell it's usually lighter than the previous year. :( Climate Change.",
 "Well, I mean, there's obviously the animal suffering angle to it, with 70+ billion animals killed each year, but also how meat production is one of the largest sources of greenhouse gases actively driving climate change.\n\nIf you don't think these are moral issues, I don't know what to tell you.",
 'It’s been an incredibly hot summer where I live. Water levels are still uncomfortably low. If this is the norm, it’s going to be rough. I wish climate change wasn’t real.',
 "Dude, the meme is referencing some other post.\n\nThis meme, is about referencing how some other meme, is not political. It doesn't make reference to itself. \n\nAnd these two posts, have an overarching political topic: climate change. \n\nHow ever many posts you make, saying your posts arent political, when they are, you're just making more meta, political posts.\n\nIt's a topic during political debates, because it has to do with politics as well. \n\nUr Batting .000 and confused, my guy",
 "I'm still waiting for a source though, when did people complain about a scientific climate change meme?",
 'Humans have overcome every challenge ever put in front of us. Climate change will be no different. Human ingenuity is boundless.',
 'Son I know we can’t get potable water easily because of the Great Drought but don’t worry, you’ll defeat the climate change dragon easily enough once we get over this bought of Dragonpox.',
 'Yeah. Now is time to SAY ALL THE WORDS and ACT WITH THE ACTIONS. Change is the current political climate.',
 'Any mention of climate change and many other topics are considered as agendas when they aren’t.',
 'Of course climate change is happening.  What would it take to stop the course of greedy humans? God',
 "Personally, I'd prefer either a second island with a different climate, or the ability to change the climate of the current island we have. For example: One with a snow covered mountain summit with snow themed animals to catch, or one with evergreen trees and temperate zoned animals. It's a big ask, and probably won't happen though.",
 'All these kids love Climate Change agendas and going green. Smh.',
 "The problem is that we need change now in the span of 3, maybe 5 years. Give it 7 years, if you like that number better.\n\nIn countries like Germany, there is the problem of huge reliance on fossil fuels for energy, in contrast, only about 1,2% comes from German nuclear.\nRolling out nuclear isn't an option, because they take too long to design/grant/build and are atrociously expensive per kilowatthour compared to renewables.\n\nYou can do much more with renewables with less money and that's what it comes down to. \n\nIf we want to achieve the German 'Energiewende', which means change of energy, we simply cannot wait for new nuclear innovation and point to it as the solution for climate change. Physicist Sabine Hossenfelder made a video outlining just why nuclear isn't going to be a short term solution and probably won't have an impact in the next 20 years. It's this one, I can highly recommend it and to top it off, she is a nuclear supporter at heart: https://youtu.be/0kahih8RT1k\n\nThis does not mean that in the more distant future there might be profitable nuclear energy again but I doubt it. Because the fundamental technology has and causes problems that we still have no real solution for, especially the waste.\n\nWhat we have are designs, prototypes and commercial promises, in which we shouldn't trust too much. Research might as well take 20 years or more until we actually have a way of dealing with the waste but then it could take even longer for it to become economically viable and thus possible on a great scale.",
 'My Point was never to Fix income Inequality, my Point was to fix Birth rates no idea how you made that conclusion and yes obviously there are many Factors to Retirment age such as Productivity but the most Immediate Threat is Falling Birth Rated whcuh will lead to huge Tax Burdens on the Next Generation.\n\nWe generally like to complain that Boomers and such didnt do anything abour Climate Change or Income Inequality but the Truth is Raising Taxes to help us in the Future will equally screw over New Generations.',
 '&gt; in another 5 or 10 years they can sunset those cars too.\n\nI guess we differ in our climate change commitment. IMO 2035 gives us  plenty of time to make the change.',
 "That's the plan!\n\nA feature not a bug.\n\nKinda like Climate Change.",
 "I'll put a bet on it that most of these people preach about climate change.",
 "It looks like you shared an AMP link. These should load faster, but AMP is controversial because of [concerns over privacy and the Open Web](https://www.reddit.com/r/AmputatorBot/comments/ehrq3z/why_did_i_build_amputatorbot).\n\nMaybe check out **the canonical page** instead: **[https://www.cnn.com/2021/10/09/weather/weather-record-cold-antarctica-climate-change/index.html](https://www.cnn.com/2021/10/09/weather/weather-record-cold-antarctica-climate-change/index.html)**\n\n*****\n\n ^(I'm a bot | )[^(Why &amp; About)](https://www.reddit.com/r/AmputatorBot/comments/ehrq3z/why_did_i_build_amputatorbot)^( | )[^(Summon: u/AmputatorBot)](https://www.reddit.com/r/AmputatorBot/comments/cchly3/you_can_now_summon_amputatorbot/)",
 'Lucky for us Antarctica just had the coldest 6 months on record \n\n\nhttps://amp.cnn.com/cnn/2021/10/09/weather/weather-record-cold-antarctica-climate-change/index.html',
 'Maybe a great investment for the parent, but that’s a pretty selfish perspective. They’re not a puppy, they’re going to have their own life and emotions and you’re bringing them into a world thats becoming increasingly uninhabitable from climate change and exorbitant cost of living. It’s delusional to pretend that isn’t the case.',
 'What does being Jewish have to do with being a pilot,  what does being a pilot have to do with climate change, and what does being Jewish have to with climate change?',
 "Nobody said some regions are more specialized than others, my point is that the building blocks of civilization are available in abundance everywhere.\n\nANd you still didn't answer my question, what do you expect, so long as civilization exist climate change shouldn't?",
 "Climate does not change. It's the weather that change. Probably you meant weather and not climate.",
 'What about climate change?',
 'Pardon, but science doesn\'t work like that.\n\nOne big thing when it comes to applying science is reducing variables.\n\nIf something occurs, we need to answer what caused it before we can start making predictions. Before we say "it\'s different this time," we have to show that what caused it in the past **isn\'t** causing it now.\n\nIf we can\'t explain why a hurricane did or didn\'t make landfall in a major city, then we can\'t really blame anthropogenic climate change any more than we can blame earthquakes on how many feral cats there are.',
 'Damn you climate change...\n\n*DAMN YOOOOUUUUUUU*',
 'Add in death by flood due to climate change and there’s another nice little triple lock for the poor old things. Congratulations on voting Tory.',
 "The old GOP denied climate change because the Koch brothers made them. So no, I don't think they were really any better. Trump just has more pull.",
 'Yeah! I’m sick of their political agenda to...warn the world about the effects of climate change and what we have to do to mitigate it',
 'Was the meme about climate change itself or the laws surrounding it? Because laws are politics.',
 'The conclusion of that video was that walruses would go where walruses should not go because it was too crowded on the beach.\n\nIs my memory playing tricks? I seem to remember that the David Attenborough crew that filmed the falling walruses due to climate change were ushering the beasts towards to cliff to get the controversial footage.',
 '**Varför är MP emot kärnkraft** när den stått för [näst minst mängd dödsfall per kWh](https://ourworldindata.org/safest-sources-of-energy) av alla energikällor samt har [lägre Co2 utsläpp](https://en.wikipedia.org/wiki/Life-cycle_greenhouse_gas_emissions_of_energy_sources#/media/File:CO2_Emissions_from_Electricity_Production_IPCC.png) än solenergi **enligt IPCC**(The Intergovernmental Panel on Climate Change)?\n\nÄven slutförvaring är löst enligt svensk modell och [byggs just nu i Finland](https://svenska.yle.fi/a/7-1360349).\n\nHur är Miljöpartiets kärnkraftsmotstånd förenligt med eran strävan mot ett bättre globalt klimat?',
 "We can farm insects genetically designed to amass protein and die quickly, much like broiler chicken. The consequences of them escaping are then minor compared to the benefits, such as solving world hunger and slowing climate change.\n\nWe could make sure people don't farm them irresponsibly the same way we currently do so for  livestock; government oversight and regulations.\n\nAnd if the ecological argument is so compelling to you, read into the ecological impact of meat production.",
 'Ah yes, climate change.',
 "Right. Just like how the oil ompanies aren't responsible for climate change. We just gotta take quicker showers, turn off our lights and use paper straws.",
 'Maybe this?:\n\n[FEMA designates $50 million to Native American tribes to fight climate change](https://www.foxnews.com/us/fema-designates-50-million-native-american-tribes-fight-climate-change)',
 'not enough scientists that he isnt into the anti-climate change grift lolol',
 "r/confidentlyincorrect\n\nOk, first, the Paris accords was just an informal agreement to try to meet certain standards. It was not law. Second, the IRA is law. Third, there's very little directly related to inflation in it. Forth, it is the largest us investment in climate change ever. \n\nYou don't know what you're talking about so stop talking.",
 'That goes for writing any message in any forum. I assume that Reddit won’t ban a person randomly just because someone reported them.\n\nLike, are you afraid that I might report your latest comment for hate speech? Obviously not, because no moderator would consider it that way. If you report as suicidal a comment that can actually be interpreted as suicidal, that’s a legitimate use of the report function.\n\nIt’s pretty easy to understand which reports are a troll and which are not. If we are talking about climate change and you report my message in which I present the data that demonstrate global warming, it’s pretty disingenuous to claim that it’s a reasonable action for a good faith user.',
 '**Abstract:** Life cycle assessment studies of large-scale lithium-ion battery (LIB) production reveal a shift-of-burden to the upstream phase of cell production. Thus, it is important to understand how environmental impacts differ based on the source and grade of extracted metals. As lithium is highly relevant to several current and next-generation cell chemistries, we reviewed the effect of varying grades in different sources of lithium (brine and spodumene) worldwide. The review covered the Ecoinvent database, scientific literature, and technical reports of several upcoming production facilities. The results showed that lower-grade lithium brines have higher environmental impacts compared to higher-grade brines. However, spodumene-based production did not show such a trend, due to different technical process designs of the facilities reviewed. Water use impacts are higher in lower-grade sources and are expected to increase with decreasing lithium concentration. This could specifically be an issue in brine-based production, where brine is extracted from already water scarce regions and evaporated, thus increasing the risk of freshwater availability. However, these aspects of water use are not addressed in existing life cycle impact assessment methods. In the context of large-scale LIB cell production, the reviewed lithium hydroxide production routes account for 5–15% of the climate change impacts.',
 'Something something natural cycle.\n\nClimate change deniers are the densest people on the ~~planet~~ disc world.',
 'Unfortunately tons of people who don’t believe in climate change also make claims that windmills break down so easily and the cold would shut them down',
 'The fact that the title of this document refers to the phrase "Climate Emergency" yet immediately conflates that with a generic "global warming" is enough for me to dismiss it altogether.  \n\nBut let\'s go further.  In their claims, they make the following conflicting statements:  \n\n  *  The gap between the real world and the modeled world tells us that we are far from understanding climate change.  \n  * Climate models have many shortcomings  \n\nThey cherry-pick a single model that oh-so-conveniently allows them to draw the conclusion they want (even though they don\'t actually *link to the model or any supporting studies*, yet at the same time *dismiss all climate models as inaccurate*.  While simultaneously not providing *any* supporting references for any of their claims.  \n\nOn top of all that, who *is* [CLINTEL](https://clintel.org/about-us/)?  \n\n&gt;CLINTEL wants to take the role of independent ‘climate watchdog’, both in the field of climate science and climate policy.  \n\nOh, look.  They want to get funding to do exactly what they say is bad and incorrect.  \n\nOk, now who are the people that [signed this document](https://www.abc.net.au/news/2020-02-27/who-are--scientists-professionals-who-say-no-climate-emergency/11734966?nw=0&amp;r=HtmlFragment).\n\n&gt;an examination of the declared professions of the signatories on the petition reveals that many of their professional occupations are unrelated to climate change, or even the environment.  \n&gt;The Australian contingent includes a barrister and solicitor, a radiologist, a managing director of a winery, a retired aircraft engineer, a security engineer and an "astro economist".  \n\nSo *they\'re not even qualified* to be making these grandiose statements *in the first place*.  \n\n&gt;Through descriptions on the petition, or through further investigations and interviews, we identified that 22 (29.3 per cent) of the signatories claimed a current or former connection to the mining industry  \n\nAnd surprise, surprise.  They have a significant conflict of interest.  \n\nA further analysis of the (entirely unsubstantiated and unreferenced) claims made in this document [reveals](https://climatefeedback.org/evaluation/letter-signed-by-500-scientists-relies-on-inaccurate-claims-about-climate-science/) even more bias and inaccurate statements.  \n\nC\'mon, man.  Do better.  \n\nedit:  \n\nAnd now they\'ve blocked me.  Pathetic.',
 '**Abstract:**\nLife cycle assessment studies of large-scale lithium-ion battery (LIB) production reveal a shift-of-burden to the upstream phase of cell production. Thus, it is important to understand how environmental impacts differ based on the source and grade of extracted metals. As lithium is highly relevant to several current and next-generation cell chemistries, we reviewed the effect of varying grades in different sources of lithium (brine and spodumene) worldwide. The review covered the Ecoinvent database, scientific literature, and technical reports of several upcoming production facilities. The results showed that lower-grade lithium brines have higher environmental impacts compared to higher-grade brines. However, spodumene-based production did not show such a trend, due to different technical process designs of the facilities reviewed. Water use impacts are higher in lower-grade sources and are expected to increase with decreasing lithium concentration. This could specifically be an issue in brine-based production, where brine is extracted from already water scarce regions and evaporated, thus increasing the risk of freshwater availability. However, these aspects of water use are not addressed in existing life cycle impact assessment methods. In the context of large-scale LIB cell production, the reviewed lithium hydroxide production routes account for 5–15% of the climate change impacts.',
 'No unfortunately it didn’t.  You talk about complexity but don’t understand the complexity of sustaining a population. Ten billion people when the west is already at a breaking point when it comes to fresh water?  Where are we getting this farmland to feed all these people?  Where will they live?  \n\nWhen you say things like that, or cast doubt on solid science of man-made climate change, it becomes a pointless argument.',
 'https://www.google.com/amp/s/www.foxnews.com/opinion/climate-change-congress-must-act-cleaner-healthier-world.amp\n\nThat sure is some far left propaganda, fox news...',
 'Need to pay those tithes to the church of Global Warming and Climate Change',
 "He's not. Meat consumption (more accurately: production) is a huge factor in climate change.",
 'cLiMaTe ChAnGe AnXiEtY!!11!!',
 "So it's hard to say exactly, we can generalize and estimate. However we still don't know a lot of how the world works biologically because many systems aren't studied enough. If you're thinking in terms of next year the chances that we will have similar conditions to this year that's a pretty solid bet. It might get a little worse in terms of frequency and severity of weather events. So more casualties and likely loss of some/many biological communities. Generally studies don't look at year by year changes so I haven't seen many papers on that time frame. It's in the decades and further time frames to come where we'll see other aspects of climate change.",
 'Literally flooding in the middle east. Just like in the biblical times. People are STILL denying Climate Change.',
 "Because nobody wants to deal with it.\n\nAnyway, it's a matter of climate change and environmentalism now, not just animal ethics. Dealing with it is a foregone conclusion, everybody's just avoiding doing so for as long as they can.",
 'Meat production is a major contributor to climate change.',
 '&gt;I don’t think anyone is denying climate change in the UK\n\n\n...anymore. There has been decades of denial, minimisation and whataboutism which is only now becoming unpalatable.',
 '^^A ^^heat ^^wave ^^intensified ^^by ^^climate ^^change',
 "Even if we took this at face value and even if you don't believe we need to do anything to mitigate climate change, oil and gas are still finite so we need to figure something out to replace them before they run out anyway.",
 "I think we'll see more of these sort of views very soon. I'm a strong believer in a somewhat selfish anthopocentric (if that's a word) view on climate change.\n\nLong term (like really long term) the earth will be fine. Life will find a way. The only reason to limit climate change is to protect makind and our way of life and our economies. The mountains and lakes and skies don't care, and some sort of life will go on no matter how much we fuck up the planet.",
 "Earth doesn't like oil depots burning, even if they are russian... I just hope this will all be over asap, so we could start focusing on beating climate change again, as this definitely doesn't help it.",
 'I remember reading an article that said if we had the technology to terraform Mars, then we would be able to end climate change on Earth and that should be our priority.',
 '"Historians". i guess "there is a scientific consensus among climate scientists that human activity is the cause for climate change" also just boild down to "many people are saying"',
 'These are the people who are supposed to do something about climate change and education when they grow up...... Yeaaaa',
 "And people still insist that climate change isn't real.",
 'We have known since the 80’s we would have an aging population and we had 40 years to prep for it and we did nothing. We have known about Climate change for the same period and did nothing. We are going to keep winging it like we always do.',
 'This meme doesn’t make sense, all of Reddit nutts over climate change.',
 'These guys are the reason why climate change is even a thing.',
 "Man even crops have life,don't eat that too\nYou should know that livestock has such population because of cattlefarming or else they wouldn't have survived sunch encroachment and climate change anyways",
 'Yeah, every water can be made drinkable. Salt water, piss and poop, etc. \nTechnology is there.\nAgain, climate change is a lie to regulate everything. Weather is manipulated. Technology is there. Look up the patents.',
 "&gt; If we don't get the climate change under control\n\nLooking at this, it's already too late in my layman's opinion. Are any one of you willing to stop buying iPhones and computers and products at their current prices?",
 'https://www.theguardian.com/environment/2017/jul/12/want-to-fight-climate-change-have-fewer-children\n\nThere you go then. That was just the first I found. Note the image used',
 'Climate change is not political shit',
 "You're dreaming. You can't scale wind turbines, they throw a wind-shadow up to 100km and change the local climate. Besides that, without a world class economy and copious amouts of cheap energy, you can't scale the production and deployment  or buy the resources needed. And if the wind doesn't blow, you have no energy. \n\nIf Europes economy goes down cause of unsustanable energy prices, it is gone and with it every possibility of europe pulling themselves out of this misery.",
 'You really misunderstand flood statistics and climate change.  I say that as a geologist/hydrologist.  Beyond the reality of the underlying mechanisms (e.g. greenhouse effect + anthropogenic release of enormous quantities of CO2 and CH4) ) and the overwhelming evidence, people, including myself, have noticed dramatic long-term changes within their lifetimes.',
 'Start in 2000. \n\nWhile his father George HW Bush was best known for corporate deregulation, George W Bush popularized being ok with Halliburton Dick Cheney getting corporate interest deeper into politics. He also popularized being a goofy doofus in office with many classic gaffs like believing WMDs were in the Middle East, declaring mission accomplished too early and being the first president to popularize climate change denial.\n\nThis gave W the “I’d get a beer with him” libertarian edge McCain and Mitt Romney never had. It gave Fox the free reign to get defensive about being called stupid. Gave republicans an anti-intellectual pro-corporate ideal to point to.\n\nMany issues resulted in Donald Trump but corporate interference causing climate change to become political sparked anti-intellectualism and helped fuel the flames of big foot hunters, flat earthers, anti-vaxers and holocaust deniers. Donald called his entire political strategy early when he demanded Obama’s birth certificate on   //// the belief that those who might give that idea a chance would be too anti-intellectual or too racist to believe any counter argument////',
 'How is this not being talked about more? Is this a direct result of Climate Change?',
 'Lol what? 2 out of 3 people voted in the 2020 election. And it’s our fault for climate change? Sure America does it’s damage but don’t act like America is the cause of climate change',
 'Your thoughts on Christian’s being one of the most likely groups of people to deny the existence of climate change?',
 '"Xi continues to act like climate change is western propaganda." said the western people that have 2 or 3 times more carbon emission per capita than China with all the manufacturing industry located in China.',
 'As our regular readers and viewers know, we do not believe in conspiracy theories. And yes, some call us “naïve” for believing that dangerous zealots are dangerous merely because they are zealots. To repeat, when people say things you disagree with and do things you disagree with, it’s because they think things you disagree with. But our position admittedly gets harder to defend when a social media platform not only slaps a finger-wagging warning across our material (“Climate change refers to long-term shifts in temperatures and weather patterns, manly caused by human activities, especially the burning of fossil fuels”) but also attributes this propaganda to the United Nations. Still, we must point out that if it were a plot, they’d certainly leave off the last bit.\n\n….\n\nA Yahoo! News story from Agence France-Press starts “A British television interview viewed tens of thousands of times on social media features a man claiming that global warming is natural, CO2 has no provable effect on the climate, the greenhouse effect is negligible, and computer models used to measure climate change are skewed. But the speaker is not a climate specialist, experts say the claims are false, and world scientists overwhelmingly agree that humans are heating the planet by burning fossil fuels.\n\n➖The author, predictably, “is not a climate specialist” himself, holding a PG Diploma in Journalism from City University and a Master’s Degree in Literae Humaniores (Classics) from Oxford, which doesn’t impair his confidence in his climate science fact-checking chops. Whereas the guy he’s dissing studied physics).\n\nWhereupon the piece waves Mann’s hockey stick without alerting the audience to any controversy regarding that artefact.\n\nIn our view it would be helpful if both sides refrained from dark hints that their adversaries are up to something dishonest and sinister. The best way to refute wrong statements, whatever their origin, is to expose them to the sunlight of open discussion. But in any case we feel that if global warming alarmists were in some gigantic plot involving the WEF and the UN, one as cunning as it was sinister, they almost certainly would not wave the pale blue banner in such a conspicuous manner. Dan Brown’s villains never do that kind of stuff.\n\n###[Declaration Saying “There is No Climate Emergency” Trends on Twitter – Greenies Have Meltdown](https://www.reddit.com/r/climatedisalarm/comments/wxa0cp/declaration_saying_there_is_no_climate_emergency/?utm_source=share&amp;utm_medium=ios_app&amp;utm_name=iossmf)\n\nIf CLINTEL Declaration Signer / Nobel Laureate Ivar Giaever can be dismissed because he is not a climate scientist, then by default ICN can come down like a ton of bricks on Nobel Peace Prize laureate Al Gore because he is not a climate scientist …. and ICN could then crucify Greta Thunberg because she is a high school drop-out. As is United Nations climate issue speaker Leo DiCaprio.',
 "It's funny how most commenters seem to think that in 2000 years after we and climate change makes food next to impossible to grow in most places, everything will just be the same as basically now.",
 "Hyperbolic comments like this are why people don't take climate change seriously",
 'Industrialized nations have already contributed to cumulative climate change for centuries. The current climate issues are because of them, not recent or future industrialized regions.',
 'Yeah, right. Give me a second to stop climate change and defeat Russia.',
 "I think you're correct on the impact of wealth distribution - poorer people will consume more\n\nThe horrendous irony of it is the terrible impact that it will have on global climate change. More consumption trends to go hand in hand with more energy consumption, leading to higher emissions.",
 "I'd like to retire for just a few years before the nasty smelly climate change turd truly hits the turbofan.",
 "It'll probably take 30 years for the climate to stabilise and stop changing due to existing human emissions. Further change could occur due to indirect disruption on natural environments. It could take centuries for natural processes to return the climate to its previous state without human intervention, if they ever do.",
 'We’re on the cusp of global collapse of basically every foundational pillar our society is built on\n\nReligion is overall dying out while extremists are hastening pushing reasonable persons out \n\nThe economy is collapsing while global productions are having supply issues, transport means and costs are causing disruptions, wages are suppressed while inflation costs are rising fast\n\nCovid and other diseases are not being handled appropriately by governments because they need to keep feeding the insatiable economy\n\nWe’re closer to direct conflict with Russia since the 70s. WWIII is likely based on the above other concerns\n\nOh yeah, and our Great Filter. We’ve spent 250 years frantically burning as much fossil fuels as we can while removing connection to our natural planet. Climate change is now-now and is far past mitigation \n\nThese aren’t things people are ‘sensing’, these are the realities that government, media, and personal bias have blinded many to. Time for humanity to wake up and take its next step, away from the consumption of capitalism. Otherwise the species dies off',
 'Maybe harder to get a mortgage, which is what will matter. I see your point.\n\nThe one thing I don’t believe is that people will finally see the light on climate change. If they haven’t by now, I don’t know what will fix them.',
 "Very nicely written. I don't think climate change measures will target the poor though. Generally lower carbon modes of transport are also more equitable and affordable when done properly. i.e good public transport. \n\nCouldn't agree more with the last paragraph. I'm sick of the hand wavy band aid policies these 4 come up with. I guess they appeal to the average disengaged voter that votes based on news headlines.",
 'USA cannot handle 10 billion, and climate change is definitely driven by man.',
 'Can’t have scientists showing proof that climate change is not man made now! \n\nOr that eating bugs is not good for you or the environment',
 'Think about it this way, a lot of people in a small area polluting a little bit creates a more polluted environment than a few people over a large area polluting a lot. But this only for local atmospheric conditions. \n\nLook at the difference in emissions per capita or total emissions for a more accurate picture of how much a country is contributing to climate change.',
 "Given Texas's population size, proclivity for using large personal vehicles, notable sprawl in urban areas, high climate control energy expenditure per person, and just a lot of hot air in general ... Someone get the President, we may have solved climate change.",
 'Climate change is a hoax.',
 'It is now legal to do more than the bare minimum to stop climate change',
 "And a climate change denier, let's not forget. Helping keep us fossil fuel reliant, thanks mate!",
 'Thank you for submitting to /r/unpopularopinion, /u/br0wnb0y. Your post, *Bottling water is a huge problem when it comes to climate change.*, has been removed because it violates our rules:\n\n\nRule 1: Your post must be an unpopular opinion.\n\nPlease ensure that your post is an opinion and that it is unpopular. Controversial is not necessarily unpopular, for example all of politics is controversial even though almost half of the US agrees with any given major position on an issue.\n\nKeep in mind that an opinion is not: a question, a fact, a conspiracy theory, a random thought, a new idea, a rant, etc. Those things all have their own subreddits, use those.\n\nIf there is an issue, please message the mod team at https://www.reddit.com/message/compose?to=%2Fr%2Funpopularopinion Thanks!',
 "My University professor for physics was a die hard Trump fan that didn't believe in climate change.",
 'Waterproof hiking boots. I wore Merrels and sprayed with Scotchguard. Did the climb this February and while it was supposed to be their dry season, it rained/sleeted/snowed 5 days out of 8. The guides said, that because of the climate change, the weather became too unpredictable.',
 "This isn't just Europe but the entire world . From California , Middle east , North China , Africa and Australia and central America , the weather is becoming hotter and drier and lot less forgiving for agriculture and human  habitation. \n\nWater is nessecary for everything from humans to agriculture and industry . With climate change, a lot of nations may not able to cope and collapse . From Deglobalisation, Climate change and major unrest in nation , i predict we are facing a new age collapse . It isn't going to be pretty.",
 'It is illegal, heck even changing climate with old school buttons, radio station, cd, cassette, etc in the past was illegal while driving. When I passed my driver\'s license over 20 years ago I remember the guy that did the test cranked the heat up a bit saying he was cold and it being a "trick" cause when it got too hot I had to ask him to turn it down vs reaching for it myself. \n\nYes, new cars have a huge touchscreen now, but many of them (if not most/all) also have buttons on the steering and voice control. If I want to set my GPS or change songs I literally just need to tell my vehicle what I want. I can tell it to turn the heat up/down, A/C, sync the temp or separate temps per area, ask it to play a specific song on Spotify or a specific radio station, volume up/down, ask to send a quick  "I\'m driving, ttyl" text, and much more without my hands ever leaving the steering or my eyes off the road.',
 'What gets me is politicians and celebs who claim to care about climate change but then fly private for every little trip',
 'Buy a prime farm that is climate change resilient.',
 'Oh I agree. I personally think the extinction of languages  is abhorrent and a sign of imperialism and very much part of the same issue as the mass destruction of ecosystems (extinction of species, climate change).\nBut I think it is important to also view language extincton objectively.',
 'There definitely not Lions supporters. I work in the Brisbane CBD they demonstrator’s demonstrating about immigration, wages, climate change, shortage of teachers you name it.  They usually a march about once a week at various times and days often Fridays about 4pm the traffic comes to complete standstill.',
 'Yeah sure you can link the report. I mean we can look at climate change from the 3 perspectives --  **Vulnerability, Resilience and Mitigation**. There is one more I think which is **Adaptability** but lets focus on these 3 for now.   \n\nLets first discuss Mitigation, I think the political structure of the United States is much more rigid and inflexible which makes decision making a long drawn out  process, like how the first climate change act was passed very recently even though there was scientific consensus and even leaders of both party knew that its gonna be a problem but the politics of the nation creates a much more longer cycle of decision making **Meaning** \\-- It takes a problem to become that much bigger before there is wider consensus about what rationally needs to happen. In India however it is politically easier to take  decisions quickly by the state, this often devolves into tyranny and then courts have to intervene and all that but in the context of Climate change mitigation, the three level political system I think is structurally equipped to deal with the mitigation part of it. \n\n&amp;#x200B;\n\nIn the last few years, because of the decreasing prices of battery powered vehicles there has been rise of e-mobility and that has penetrated right down to the village level. The rise of e Rickshaws, Most of the major auto providers are going to launch their own brand of cheap, mid range and expensive e-vehicles in next 12 months. There is focus by the government on diversifying the energy basket meaning, Indian currently depends on  energy for crude oil which against restrains the geopolitical and economic choices because increase and decrease in international crude oil leads to inflation which the "state" can\'t control so there is this incentive for india to spend in renewables so that it is financially independent in that sense. I think it is the decreasing price of renewables which is the major factor which favours mitigation and India has Deserts and other Solar surplus regions, it has windy regions, it has areas with coastal waves energy projects, it has 10-11 I think Nuclear projects so I think the economics will make it easier. But Coal will still remain important for some time.  The first process of the Climate change mitigation is the realization within the wider economic structures that **Decarbonization doesn\'t necessarily have to  lead to economic contraction. This is the first step.** This step usually takes some time because often decarbonization is expensive but if you can show it to your people that investing in renewable doesn\'t mean contraction and can lead to economic growth now that\'s something which is revolutionary. So I think India is well placed to do its bit on the Mitigation front and there is no anti-climate change lobby here and citizenry and civil society is structurally well placed to transition. The only choke point which I can remember right now is the Energy storage capability and Power Grid stability meaning the solar and wind are intermittent power sources and are active only for some part of the day and require storage and some continued production sources to compliment it which can pose a problem**. Its not the will of the people to climate action which is the restraining factor but the technology and the economics of it**.\n\n2. I think there is considerable **Resilience** within the government and Non government structures to deal with any crisis. I mean it took about 2 years to vaccinate over 100 crore people and that is a impressive task. Every district has Disaster response plans, which  is a very recent phenomena so there are bound to be gaps but then overtime these institutions are bound to strengthen. India has a good Satellite and remote sensing network. Increased digitization has also made it very easier for state to do targeted social welfare to the bank account of those folks who are facing disaster within days or weeks. India is one of the biggest pharmaceutical producers and one of the worlds largest food producer and has about 60-70 million tonnes of food grains in storage for emergencies at any given time so that\'s something. I mean the presence of state capacity, information network, cheap internet, food and medicine self sufficiency makes it easier for societies to deal with problems. However there is considerable inequality within society and that gap is usually filled by government subsidies which is considerable and NGOs also pitch in.  But then again India is low income country, there is not much fiscal space. I think if India grows upto be lets say about 10 trillion economy in next decade or so then Government can do anything in case of emergencies. I mean remember global help is welcome always but in times when everyone is suffering as seen in Covid19, countries tend to look inward. **I don\'t think there are chances of government instability at all. The Indian political structure is just too wide and inclusive, there is no single focal point around which people can mobilize to so a regional instability can spread.** What I mean is -- sure there can be regional problems but structurally it is very difficult for those problems to spread because again all those fractures which I mentioned in the last comment comes into play. In fact people tend to come together when there are societal level problems. Disasters themselves actually lead to much more social cohesion and togetherness, it is the lack of resources later which leads to social contentions. If India is rich meaning it has lots of economic activity which can be diverted when need arises then I think It should remain stable. More so sustainability is inherent in the religious teachings and forest and nature plays a important role in rural societies so even animal conservation is culturally important phenomenon  and there are various schemes through which problems like excess pollution and deforestation and land degradation are being dealt with. Again all this depend on the per capita wealth of an average citizen which is considerably lower than average global citizen which leads to a state with less capacity to provide services at scale. As economy grows so will the resiliency. Its like that person who is not poor but is one major injury away from Poverty line, the person who is upper middle class is better equipped to deal with problems. Science and technology related solutions require economic capital to solve problems at scale and when people truly work together, they can solve most of the problems. Climate appropriate agriculture which is decentralized is one such initiative. \n\n3. Now I do think **India is one of the most vulnerable country** when one talks about Climate change.\n\n\\-- There are low lying areas which might face saline water intrusion \n\n\\-- There are water stressed areas and drought and flood prone areas.\n\nSo yeah I think Vulnerability to various disasters is high that can\'t be changed I guess.',
 "Climate change is a teenage issue? OP got you with the meme I guess. And that's __funny__.",
 'I get that, but our society is not set up to provide that. Some people’s best source of protein, in today’s world, is meat. Even if it wasn’t, eating an animal that lived a humane lifestyle isn’t morally wrong. It’s the circle of life. Worms and bacteria are going to eat me when I am dead. \n\nHowever, I understand that the meat industry does not treat its livestock humanely. I strong agree with vegans there (and about how it affects climate change), and I think the meat industry needs to be strictly regulated and shrunk.',
 'Climate change',
 'The sun isn’t getting hotter, the increased c02 in our atmosphere just prevents that ‘heat’ from leaving. it’s called the greenhouse effect. This causes climate change and global warming',
 'https://news.mongabay.com/2016/04/big-oil-covering-knowledge-climate-change-even-longer-thought/\n\n\nKdyž i BP desetiletí krylo a stále kryje že se něco děje, protože si nechali vypracovat studii a ta studie je zhrozila že se něco děje, tak je fakt něco špatně.',
 '&gt; US car centric culture spews out a ton of emissions and helps accelerate climate change\n\n&gt; global climate change leads to heavier than normal downpours\n\n&gt; US gets away with contributing to the issue but not dealing with the impact (for now)',
 "Thanks for signing up to BBC facts! You will now receive fun daily facts about the BBC. \n\n**Fact 24.** The BBC published an article claiming that climate change laws are [“used as a stick to beat the government.”](https://i.redd.it/5sj8eyl80m661.jpg)\n\nFor another BBC fact reply with 'BBC impartial'. To unsubscribe call me a 'bad bot'.\n\n#[Click here to cancel your TV License and stop funding right wing propaganda today.](https://www.tvlicensing.co.uk/check-if-you-need-one/topics/cancellations-and-refunds-top7) \n\n*I am a bot, and this action was performed automatically. Please [contact the moderators of this subreddit](/message/compose/?to=/r/GreenAndPleasant) if you have any questions or concerns.*",
 'It’s also what happens when we ignore climate change',
 "Because it's either nuclear or worse climate change than what we already have. Tertium non datur.",
 'Sorry but this is stupid. Youre not raising people who are going to naturally or easily navigate record inflation, pandemics, mass shootings, climate change and increased cost of living. If you live among dragons then maybe you should work towards solving that first so you dont have to raise dragon slayers.',
 'If you think JP is a fascist you need your head checked, Fridman also had Sam Harris and Michiu Kaku on his show, are they anti-vaxx right wing climate change deniers too? Never heard of Fridman before I saw this post, he seems to keep an open mind, unlike you.',
 'Show me a funny meme or don’t waste my time. If I want to hear about climate change and other teenage issue, I’ll go on twitter',
 "Humans gonna human. It's *almost* remarkable how limited the imagination of a culture can be when it comes to speculation of this nature. Which takes existing observable circumstances and pushes them out to encompass ***inordinate*** amounts of the universe—in time and space. For example: \n\n* The horizon is a straight line. Hence, [the world is flat](https://en.wikipedia.org/wiki/Flat_Earth).\n\n* Computers keep advancing. Hence, [the simulation hypothesis](https://en.wikipedia.org/wiki/Simulation_hypothesis).\n \n* The sun moves in the sky. Hence, [it revolves around the Earth](https://en.wikipedia.org/wiki/Geocentric_model) (which [1-in-4 Americans believed in 2012](https://www.npr.org/sections/thetwo-way/2014/02/14/277058739/1-in-4-americans-think-the-sun-goes-around-the-earth-survey-says)).\n \n* Civilization is between 6,000 and 10,000 years old. Hence, [all of the Earth is the same age](https://en.wikipedia.org/wiki/Young_Earth_creationism).\n \n* Coal burns hot. Hence, [the Sun is made of coal](https://archive.ph/JyWGl).\n\nNow, given that this is speculation based on *some* limited observational evidence, there is some *limited* chance it could be true. But, in looking at the list above, thinking patterns of this quality don't exactly have a very good track record in ***accurately*** explaining nature or the world. It could even be said to explain, *in some circumstances*, [the roots of racism and other cognitive biases](https://sites.bu.edu/ombs/2013/10/16/why-are-people-sexist-racist-and-judgmental-behind-cognitive-bias-and-prejudice/). With the most tragic example being the ridiculously [oversimplified ideas advanced by Yale economist William Nordhaus](https://archive.ph/FvzXS#selection-1069.0-1101.371) in shaping climate change policy over the last 50 years. Although I'm not immune to this flaw myself; extrapolating ***too far*** out—with limited data—is prime cut human idiocy. Most especially if it's done with any confidence or assertive claims of authority. As some of the examples listed were for periods of time in human history. \n\nEdit: Clarity",
 'Looks absolutely daunting. Climate change impact this year has been massive.',
 'Amen.  The climate change sales pitch is iffy and the average person isn’t taking one for the team and destroying our way of life over it.  We love our vacations, cottages, fishing trips, road trips, sports cars, muscle cars, etc.\n\nThe other thing that bothers me about this push to Evs for the common folks is that the elites will be able to get around it all and pay whatever fines or fees are required to continue to enjoy whatever it is they do that burn fossil fuels.  I get stuck in a Prius but they get to keep perusing around in a Lambo Aventador.  Yeah, no.',
 'Who cares about a recession when we’re all dying of heat waves, snow storms or other extreme conditions we’re causing by not fighting climate change enough?',
 'Climate change',
 "Climate change (while real) is a scam. Miami won't be underwater in our lifetime. People saying they're going to save Miami from climate change by getting a better FEMA rating are clowns",
 'Yo bro people talking about climate change,discovering space in MENA we are talking about hijab.let people wear whatever they want.u dont have to have opinion on everything',
 "When did they talk about american intervention? Again, you are deflecting the problems, talking about America all the time and never about your shit.\n\nWe were talking about indian problems that are not being adressed by your government but are mostly caused by them. They are reducing freedom of expression, they are letting rise the hatred between muslims and hindus. They are corrupt. They are still investing in polluting your country that will be hit the hardest by climate change. And so much more that i don't have time to list.\n\nKeep thinking that the world is out there to get you while they are just trying to warn you. \n\nAlso do you know those are third parties reports and not american gov ones? But I guess in your mind everyone is paid by the americans to destroy india or something. \n\nI repeat, a mature person instead of deflecting his problems tries to solve them.",
 "Show them climate change is a nightmare, here's why by upIsNotJump. It's millennial humor but it's very informative",
 'I get that. I guess we‘ve been talking past each other cause that’s exactly what I meant originally. If you don’t like the topic, just don’t engage. I like the topic of climate change and I happen to have a very strong opinion about people hating on humans. You gotta accept what and who we are, the good and the bad. \nMy response was a little salty cause I took your „don’t participate in earth“ as a backhanded way of saying „go kill yourself“.',
 "there are deaths right now due to energy shortages in WESTERN COUNTRIES imagine what is happening in the less fortunate ones.\n\nhttps://www.theguardian.com/world/2012/feb/03/europe-cold-energy-death-toll\n\nhumans need energy. people's lives and economies are literally at stake, as well as the climate change issue being real and escalating. gas is the fastest to build and the cleanest fossil fuel. its the stepping stone to total renewable.\n\nim a 10+ yr Greens voter but there isnt enough renewable production capacity to meet demand. it is growing but until there is enough renewable capacity we burn gas or people die.",
 'I just posted this on three meme sites and have gotten about 5 people already trying to argue that this meme is political. It’s accurate to what actually happens when you say climate change anything on Reddit, a bunch of people think it’s very existence is up for debate in congress or something when that’s not how it works. Facts don’t care about their feelings.',
 "Idk why folks are downvoting you. They just mad cuz you're right.\n\nI'm sure the other person *meant* to say Climate Change",
 'Climate change*',
 'I don’t think anyone is denying climate change in the UK, just the ways to deal with it.(nuclear vs wind and solar etc)',
 'I’m a fan of rational reminder and I’ve seen that episode. The problem is there are other papers and studies that suggest coastal real estate has not priced in climate change. I do agree that green stocks are at a premium but I’m not sure the market doing a good job of pricing in climate change as Ben suggests. https://www.bloomberg.com/opinion/articles/2022-04-02/the-ocean-is-coming-for-homes-that-s-not-priced-in#xj4y7vzkg',
 "No, it's just climate change doing its thing. 😢",
 'They had droughts that lasted hundreds of years, otherwise known as climate change.\n\nEdit: Why on Earth would you downvote a concrete historical fact? Belief isn’t just a river in Egypt, kids. Learn to question your own opinion, unless you’re anti-science.',
 'That stupid climate change',
 "By not delivering any climate change policy promise, I've never seen such a small summer temperature gap between north and south. 5D chess and all that.",
 'This is posted on a website where liberals make up 81% of total users.\n\nYou\'re incredibly ignorant if you think "reddit" is against you posting climate change memes.',
 'Regardless, even these not-humans will needs things like food and water, and thus life on earth has exactly a billion years to escape, or else move the Earth to a farther orbit.\n\nIf we all suddenly adopted a view of "there\'s no point rushing if we have a Billion years", then that\'ll get passed down and nobody will *ever* bother to try.\n\nAnd then in 950 million years, whatever sentient species is alive then will be having the same dumb argument we are now about climate change.\n\nYou talk about hubris, but it\'s not exactly humble of yourself to draw fun from people choosing to think about the long term.',
 'And we should expect even worse human development indices in the next decade thanks to climate change. India will become a powder keg of civil unrest once the weather becomes more inhospitable and the resources are scarce.',
 'Bernie considers global warming the most timely change to national security. He opposed the establishment of the Dakota Access Pipeline on the grounds that it will have a notable influence on the climate.\n___\nBernie Bot -Bernie_Bot_03',
 "&gt; And if you give it a whirl you can eat like a fucking King; wild “fresh” shrimp, New York steak medium rare drizzled with raspberry infused olive oil, lunches was eating rolls of fresh cut meat with wasabi, chicken wrapped bacon, home cooked taco salad.\n\nHow about bugs? I see a disturbing lack of maggots and cockroaches in your diet. Don't you worry about climate change?",
 'Where I’m from it’s illegal to water your lawn on a rainy day. Despite climate change we are not [yet] at shortage of water.',
 'I’m for more of the scientific and humanitarian perspective, try and do as much as we can as a collective species. Generosity is the founding virtue of Buddhist principles, we should *not* ignore this problem of climate change and pollution, as it will lead to suffering.',
 "[here you go.](https://www.climate.gov/news-features/understanding-climate/climate-change-global-sea-level)\n\nYou must not live near the ocean... I'm European. We must deal with this stuff every day.",
 "And the fact that Spankie, Ra'Jah, Sasha, Lala and Chedder could all reign together will end climate change",
 'Yea but we’ve been in a drought for years now. It’s long since been overdue that we build new infrastructure to collect and store what little rainfall and melted snow we do get. This is the new norm, and they’ve been warning us for years about the effects of climate change, yet haven’t done much about it. In retrospect, I’d rather have infrastructure maintenance issues for a single area, than have to build new infrastructure to provide water for the most populous state in the nation.',
 "It's Depressing that trying to spread awareness about climate change,is seen as a agenda.",
 "Any capitalist society will always incentives business owners to minimise wages. It's called class antagonism. The business owners make more profit if they pay their employees less. You can fix this (and all the other problems: market inefficiency, over-extraction of natural resources, climate change, boom-bust cycles, inequality, e.c.t) with legislation, but then you end up with knock-off communism. Why not go with authentic communism?",
 'Greenpeace started as an anti-nuclear movement.\n\nI am convinced that they are responsible for the climate crisis. If they did not spearhead the criticism against nuclear power in the 60s, we would have had the nuclear powered future we were promised in the 50s. There would have been no carbon emissions and thus no climate change, because nuclear power would have become the standard.',
 'Memes can contain anything including politics, or in the case of climate change: science.',
 "You meant it sarcastically, but with very few exceptions the existing Democratic party has very much been responsible for failing to limit climate change. They haven't even been trying until very recently.",
 'What is political about it is the question of what, if anything, should be done. They aren’t at the podium in congress voting on whether or not climate change is real. Your Senator and representative isn‘t going to colege for the relevant degrees and then spending decades in the field collecting data.',
 "&gt; climate change won't allow me to see retirement.\n\nYou're delusional if you believe this",
 'Climate Change is a big problem this century\n\nDo you think there will be enough mitigation and resilience to handle extreme droughts, heatwaves, famine and flooding?\n\nThere have been government reports in the US that there is a high probability of government instability due to the above problems by mid-century,\n\nIt’s pretty worrying that India being unprepared could turn into an unprecedented crisis in a few decades (USA should try to fund a certain amount of assistance;)\n\nI’m curious of your thoughts, as this on the horizon and few countries are prepared (can link the report if you are interested)',
 'Yes \n\n0-14 exceeded the 2021 total year by week 30 in 2022\n\nexcess deaths in 0-14 age range aligning with the rollout of the medical procedure to the same age range ... \n\nBut it is just all due to climate change, right?',
 'Yes, the Netherlands, a country famous for its dry weather.\n\n(Major snowstorms are rare but they still happen. Climate change led to more extreme winter weather including harsh cold snaps on occasion. People still bike.)',
 "Yes and no one can ever truly know.\n\nYes - new climate change studies can be directly reflected in price changes. For instance, real estate market actually priced-in reported changes on a niche research paper within the first couple days of it being published. This was discussed specifically in a Rational Reminder episode but I don't recall which.",
 "It's fine. Climate change is fake. /s",
 'I don\'t think they will lockdown for a virus again anytime soon. The next lockdowns will be for "climate change", God knows how they\'re gonna pull that off',
 ' Build on a flood plain, it will be fine. Place hasn’t flooded in a century. ITS BECAUSE OF CLIMATE CHANGE. QUICK, INTRODUCE A CLIMATE TAX TO FIX. This ladies and gentlemen is why you don’t erase history.',
 "Yes. Climate change shouldn't be political. But one political faction made it so",
 'The fact that whether or not you acknowledge climate change has become a political point is depressing.',
 'And if she ain\'t dead she will be soon from energy rations and climate change carbon taxes. \n\nWhoops turns out she wont need that retirement fund or pension afterall.\n\n*HOW CONVENIENT*.\n\nFunny how shit always rolls down hill and gold, milk and honey always flows up.\n\n*"Do not gaze at me because I am dark,\n    because the sun has looked upon me.\nMy mother\'s sons were angry with me;\n    they made me keeper of the vineyards,\n    but my own vineyard I have not kept!*"',
 'Southern Ontario has had no excess rain going back decades.  The mayors are using "climate change" as an excuse when downstream properties are impacted by overdevelopment upstream.  They don\'t want to do proper flood mitigation so just pass the costs on to the downstream homeowners.  My home was not floodplain when I bought it and the plain keeps moving as the developers north of me pave over the farmland.  That apparently is completely legal.',
 'Bernie considers climate crisis the most valuable change to national security. He opposed the manufacturing of the Dakota Access Pipeline on the grounds that it will have a big influence on the climate.\n___\nBernie Bot -Bernie_Bot_02',
 "It simply doesn't work. We cannot beat climate change with the same virus that got us into this mess. There is no way we can consume or invest our way out of this. Green technology is important, but it alone will not do much. Green investment is just VCs going with the current trend to make their money. The only truly feasible way is to scale back consumption and to not treat the planet like something that can be exploited and used to reap unholy profits.",
 "That would be true if fossil fuels were below ~10% of our energy use.\n\nBut fossil fuels are responsible for 85% of global primary energy use, and they're not shrinking.\n\nReplacing renewables with nuclear is folly, but replacing nuclear with renewables (like Germany does) is also folly. \n\nWe need to remove fossil fuels as rapidly as possible, which means both renewables and nuclear, and after we destroy fossil fuels and dance on their grave we can see if we can get to 100% renewables or not.\n\nBut right now, anti-nuclear sentiment is burning the planet. Germany could have decarbonised significantly and be independent of Putin's natural gas, but they chose to fight nuclear power instead of fighting climate change.",
 'All the negative energy put out these past few years is the reason for “climate change” as well',
 'Note that the statement by Pakistan\'s federal minister for climate change, Sherry Rehman, that a town called Padidan got "70 inches of rain in one day" is almost certainly wrong. Meteorological data for the area shows it has gotten 70 inches *since June*.\n\n[https://en.wikipedia.org/wiki/List\\_of\\_extreme\\_weather\\_records\\_in\\_Pakistan#Heaviest\\_rainfall\\_of\\_400\\_mm\\_or\\_above\\_during\\_24\\_hours](https://en.wikipedia.org/wiki/List_of_extreme_weather_records_in_Pakistan#Heaviest_rainfall_of_400_mm_or_above_during_24_hours)',
 'Should have stayed in south Africa. Climate change is not an issue over there',
 'Nope.\n  \n\n  \nhttps://www.pop.org/overpopulation-myth/\n  \n\n  \nhttps://www.vox.com/the-big-idea/2017/12/12/16766872/overpopulation-exaggerated-concern-climate-change-world-population\n  \n\n  \nhttps://sustainablereview.com/overpopulation-is-a-myth/\n  \n\n  \nhttps://www.theworldmind.org/home/2021/12/10/the-dangerous-myth-of-overpopulation\n  \n\n  \nThis overpopulation narrative has to end.\n  \n\n  \nAlso, besides, seriously who are the ones who are always screeching about overpopulation? And also, which demographics are constantly being told to not breed? Why those with overpopulation concerns NEVER starts with themselves?\n  \n\n  \nIt\'s always an urban, "educated" PMC or upper class, telling others (but more often than not poor people) to not breed.\n  \n\n  \nIt\'s ALWAYS "Too many of you, not enough of me".\n  \n\n  \nAnd I\'m sick of this.',
 'I think the last thing we really need to be doing as a species is wasting the remaining  amounts of our fresh water on mere aesthetics and the desire to have grass everywhere. Climate change is a thing and turning every possible inch of our habitable areas into grassy fields is a part of why we see extreme swings in our climate.',
 '“Ha, take that, climate change!”',
 '&gt;No.  There are nearly 8 billion people now and still growing.  This population is not sustainable at the levels of consumption that most of the world aspires to.\n\nNope. \n\n[https://www.pop.org/overpopulation-myth/](https://www.pop.org/overpopulation-myth/)\n  \n\n\n[https://www.vox.com/the-big-idea/2017/12/12/16766872/overpopulation-exaggerated-concern-climate-change-world-population](https://www.vox.com/the-big-idea/2017/12/12/16766872/overpopulation-exaggerated-concern-climate-change-world-population)\n  \n\n\n[https://sustainablereview.com/overpopulation-is-a-myth/](https://sustainablereview.com/overpopulation-is-a-myth/)\n  \n\n\n[https://www.theworldmind.org/home/2021/12/10/the-dangerous-myth-of-overpopulation](https://www.theworldmind.org/home/2021/12/10/the-dangerous-myth-of-overpopulation)\n\nThis overpopulation narrative has to end.\n\nAlso, besides, seriously who are the ones who are always screeching about overpopulation? And also, which demographics are constantly being told to not breed? Why those with overpopulation concerns NEVER starts with themselves?\n\nIt\'s always an urban, "educated" PMC or upper class, telling **others** (but more often than not poor people) to not breed. \n\nIt\'s ALWAYS "Too many of you, not enough of me". \n\nAnd I\'m sick of this.',
 "that's right the Democrats caused the climate change that's causing the drought in the south-west US.\n\nwait, what?",
 'If anthropogenic climate change and capitalism weren’t sure to fuck us all up and fry us alive, then maybe I would reconsider.\n\n*Just maybe.*',
 'C\'est super, moi aussi je me posais cette question. \n\nRien à voir ou presque avec le sub (car je pense que c\'est une façon de lutter contre le capitalisme ou du moins le pouvoir de ces groupes et l\'impact qu\'ils ont malgré nous sur le monde...) mais pour ce qui est des désastres écologiques, en mettant de côté l\'éthique, supporter et participer au véganisme est une des meilleures actions que l\'on peut entreprendre (en tant qu\'individu, donc seul et qui ait réellement un impact).   \n\nLes aliments d\'origine végétale émettent moins de gaz à effet de serre que la viande ou les produits laitiers, et ce, quel que soit leur mode de production : https://ourworldindata.org/food-choice-vs-eating-local. Ce n\'est pas tant d\'où viennent les produits que l\'on mange qui pollue mais, en réalité, ce que l\'on mange qui a réellement un impact.  \n\nIl y a beaucoup de choses qu\'une personne peut faire pour réduire son empreinte carbone, notamment : avoir moins d\'enfants ou ne pas en avoir, recycler davantage, marcher, faire du vélo et utiliser davantage les transports en commun, utiliser moins les voitures et les avions, utiliser moins d\'électricité et éduquer les autres. Mais l\'un des meilleurs moyens, dont on ne parle pas souvent, est de devenir végétalien et de passer à un régime à base de plantes. \n\n"Quel est le meilleur régime pour lutter contre la destruction de l\'environnement ? D\'après la plus complète analyse de l\'impact de l\'agriculture sur la planète, l\'alimentation à base de plantes est la plus efficace pour lutter contre le changement climatique.\nJoseph Poore, chercheur à l\'université d\'Oxford, qui a dirigé l\'étude, a déclaré qu\'adopter un régime végétalien est "le moyen le plus efficace de réduire votre impact sur la planète Terre…" : https://livekindly.co/eating-vegan-is-the-most-effective-way-to-combat-climate-change-says-largest-ever-food-production-analysis/. \n\nC\'est une solution simple (dans le sens où elle ne nécessite de ne faire passer aucune loi, on peut lutter sans l\'acquisition de cette dernière) à l\'utilisation massive des terres, qui est responsable d\'un nombre important d\'extinctions d\'espèces, les émissions, la pollution des cours d\'eau, des rivières et des océans, la mort des océans, la surpêche et le déversement de quantités folles d\'engins de pêche en plastique, la résistance aux bactéries et les zoonoses créées par l\'agriculture (salut le COVID, Ebola, ...), la liste est longue.\n\nPour couronner le tout, il est incroyable que des produits qui aient comme ampleur une telle destruction soit subventionnés par l\'argent des contribuables au lieu d\'être taxés. Alors qu\'une bonne partie de la population est intolérante aux produits laitiers (60 à 70%), que de nombreuses religions interdisent la consommation de certains animaux et que des personnes s\'opposent à l\'exploitation et à la maltraitance des animaux pour des raisons éthiques.\n\nSavoir ceci est quelque chose qui m\'a beaucoup rassuré car on peut quand même faire quelque chose et que, surtout, il y a des solutions... Car autour de nous, dans les infos, tout ce qu\'on dit c\'est que ça va plus ou moins mal et lorsqu\'on creuse on se rend compte que ça ne va pas du tout et pour autant, aucune solution que l\'on puisse réellement appliquer semble être envisageable... Ne pas utiliser sa voiture, c\'est bien mais pour beaucoup c\'est impossible. Beaucoup des solutions proposées ont un coût que nous ne pouvons pas nous permettre...  \n\nJ\'espère que, bientôt, on pourra absolument tout changer comme il le faudrait',
 'Bernie considers climate crisis the most essential change to national security. He opposed the establishment of the Dakota Access Pipeline on the grounds that it will have a big effect on the climate.\n___\nBernie Bot -Bernie_Bot_01',
 'Nobody will be using level 2 chargers as the end goal of charging, these are basically what you’d have at home overnight. We have the advances already done, they are called DC fast chargers and there’s already a usable if sparse network of them that allows for long distance EV trips across the country. See [this](https://www.jdpower.com/cars/shopping-guides/what-is-dc-fast-charging) article, these chargers are being rolled out right now even before the likely boom of EVs with the most recent climate bill. Atm a lot of the limits on charging are coming from the cars themselves, which actually are mostly lagging a bit behind as only a few models can utilize the highest powered fast chargers, which will of course change as the fast chargers are implemented. In a decade you wont be waiting 12 hours to charge at a station, probably something like 10-15 minutes to bump your charge up.',
 'After regime change is just the right time for shock therapies and social experiments. Eastern Europe embraced capitalism and adopted "business-friendly" policies, "flexible labour markets" and all that jazz. Privatization in this business-friendly climate made first domestic oligarchs, and later foreign corporations, politically powerful, and further "unpopular but necessary reforms" were rammed through.\n\nMuch the same thing happened in western Europe as well, where social democracy was substantially dismantled during the same time, except both the starting point and the ending point were less extreme.',
 "I'll admit I actually made this thread because I wanted to laugh at climate change deniers and watch them handwave the very real and observable problems that come with it. So far it's a kick.",
 'Climate change isn’t going to bring the weather you’d wish for.',
 'Moralni problemi - [profiti za vrijeme "krize" ](https://www.current-news.co.uk/news/shells-profits-continue-to-grow-amid-wider-energy-crisis), [mito da bi se zaustavili propisi za smanjenje stakleničkih plinova](https://www.huffingtonpost.co.uk/entry/bp-shell-climate_n_5f6e3120c5b64deddeed6762), [puštanje nafte u rijeku niger](https://www.google.com/amp/s/amp.theguardian.com/environment/2011/aug/03/shell-oil-spills-niger-delta-bodo) i [još malo o tome koliko om je tribalo za reagirat I kako pokušavaju izbjeć plaćanje](https://www.amnesty.org/en/latest/news/2018/03/niger-delta-oil-spills-decoders/) i [odgovorni su za ubojstva, silovanja isl. u Nigeriji](https://www.google.com/amp/s/amp.theguardian.com/global-development/2017/nov/28/amnesty-seeks-criminal-inquiry-into-shell-over-alleged-complicity-in-murder-and-torture-in-nigeria). Ima još [ovo](https://www.ejatlas.org/print/shells-toxic-legacy-in-curacao), [ovo](https://www.theguardian.com/environment/2015/oct/10/shell-exxon-gas-drilling-sets-off-earthquakes-wrecks-homes) I [ovo](https://nltimes.nl/2017/02/28/shell-accused-ignoring-its-climate-change-warnings-decades)',
 'David Attenborough: “This is the yellow-footed, broad-chested Dragon Squirrel. It breathes fire and attracts a mate by mimicking the ‘Freebird’ guitar solo by using found sticks and vines.”\n\nMe, stoned: “whoooooa”\n\nDA: “because of poachers and climate change there are 3 left and actually two just got run over by a tractor.”\n\nMe, still stoned: “…”',
 "And won't worsten climate change...",
 'Climate change hits hard',
 "A lot of the media. You'll see pieces discussing climate change and talking about how having children and the consequent population increase is bad for the planet. Always discussed in relation to Europe/US, where populations are heading towards decline. Never mentions that 80 percent of the world's population, and the bulk of population increase is outside the West",
 'Climate change could alter ocean currents, and if they do, Europe will suddenly find itself quite far from the equator.',
 'Your money is invested, so the return on investment is greater than inflation and fees. As to your other point a private pension is your only shot at not working till you die or living in poverty in your old age. Maybe climate change will prevent you from retiring, but not having some sort of a pension definitely will.',
 'Climate change advocates would be up in arms',
 'Bernie considers climate crisis the most essential change to national security. He opposed the establishment of the Dakota Access Pipeline on the grounds that it will have a huge effect on the climate.\n___\nBernie Bot -Bernie_Bot_04',
 'Climate change.',
 "My fear is that 1. The spending power of 1 euro will go down so much that the money I put in now will have little benefit in the future. And second, that climate change won't allow me to see retirement.",
 'Climate change is a hoax. Ignore the fact that almost every year since 2000 is either the warmest on record or has some other extreme weather event like drought',
 'It would be a straw man if I claimed he did not believe in climate change, as a ‘refute’ to him claiming that he believes in this particular science.\n\nI just asked a question. The answer could be yes or no.',
 'And there I was, under several layers of mud landslide climate change emergency in my motel emergency accommodation after just having been smoked in the cross fire of a gang shootout, I was obese tho so I took the hits to my oedema well, and had just got my 56th booster so all my blood had clotted anyways, but there I was just about to go spend up on my COL payment and get some new Nikes when a goddess-like apparition appeared before me wearing a mask and burka and it was her!!! OMG, it was her and she hugged me! She actually touched me, OMG guys, guys, [GOIZ](https://i0.wp.com/couchpimps.com/wp-content/uploads/2020/05/Yellow-Teeth-Soyboy-NPC.png?fit=2083%2C1842&amp;ssl=1)!!!\n\n*c00m squirt*',
 "Go look at his comments.  It's his job to defend PFAS specifically he works for one of the companies producing it. Even if he doesn't directly work for one of the companies, he does still work for them.  Also I explained I was going to school for a chemistry degree, but my brother who is a chemist , who worked for a lab reverse engineering drugs explained those type of researchers, are the ones that skew their data to meet their client's needs.  He used the same talking points DuPont used.  This is similar to having doctors tell you cigarettes are safe, or Oxycontin isn't addictive, or that fossil fuels don't cause climate change.  You can hire scientists to say whatever you want if you have enough money.  Also that guy defending PFAS against birth defects is asinine, because they have know that they cause birth defects since the 1950s and were sued over it, but it's sensationalized media right?",
 'Never once experience rolling blackout nor any of my friend.  Yes, we have forest fire due to climate change.  But hardly the level of completely running out of water under an incompetent republican.',
 'For those who can’t access due to the paywall:\n\nhttps://archive.ph/2022.08.30-202028/https://www.washingtonpost.com/religion/2022/08/30/evangelicals-climate-change-bible/',
 'It was fox news has been saying this everytime some celebrity says something about climate change.',
 'this is just a reminder of what climate change will do',
 'If you support the side denying climate change then you’re not a good person, wether you agree or not it’s tacit support',
 'As someone from Bangladesh.  I thought india and Bangladesh would be the first to go underwater. Climate change keeps throwing 360s at us',
 'I just saw a video where they were talking about the "safest" places in the US when Climate Change hits. Mainly these places were near the great lakes. Some people in the comments have already talked about moving there and buying land and then living there autonomously and regretted others who didn\'t come up with the same smart idea. Then I just thought to myself, you poor soul, have you forgotten who lives in the south and who will then make their way to you ? And I\'m not talking about people across the border but fellow americans. Americans with AR 15s who think the world belong to them.',
 "&gt;A relatively small number of fossil fuel producers and their investors could hold the key to tackling climate change\n\n\nWhich is an absolutely ridiculous way to present the issue. Like it or not these companies are producing fossil fuels, because we buy them, because we NEED them for just about every aspect of life.\n\n\n\nIf you drive your car to work, then YOU are reasonable for those emmisions. It has nothing to do with fossil fuel companies.\n\n\nEven if we could somehow pin all the plan on those companies, the fact that 100 of them happen to produce 70% of the emissions is a matter of how much of the market is controlled by relatively few companies. It's not a measure of how good or bad they are doing on the climate. If there were a million tiny companies providing those fossil fuels instead of 100, would that some how be a better situation?\n\n\n\nNo, the article is filled with baffling idea that if these 100 companies would just get with the program and stop existing, the entire issue would be solved in one afternoon. I mean, for godsake, the top company on the list is Coal (China), what exactly is the guardian recommending that every coal producer in China do? Stop selling coal and move into the fast food industry?",
 '- Amukan massa itu gak bisa mikir jangka panjang.\n\n**Apapun yang cuman berdasarkan dopamine hits dan nafsu belaka doang itu gak bisa mikir jangka panjang.**\n\nIni sama aja dengan Kemenkominfo - Belum dua bulan aja udah balik ke normal.\n\nJuga,\n\n- Serius, mindset "Kamu gak bisa renang dan gak basah" dan "muh freedom, autonomy &amp; individualism" itu harus dibrainwash jauh di pendidikan. Liberalisme udah bikin orang cara mikirnya kayak remaja abadi ngambekan.\n\nMau demokrasi multipartai yang ideologis ya berarti constant compromise dan muncul partai yang cara mikirnya bukan kayak elo doang. PKS itu demokratis. Mereka gak pernah nyerang norma demokrasi itu sendiri.\n\nMau ngatasi climate change ya berarti harus OK pas Pemerintah nyabut subsidi dan mulai ngedorong hingga maksa untuk ganti ke kendaraan listrik.\n\nMau ada Pancasila ya berarti mau mempertahankannya dari ideologi sinting.\n\nMau welfare state gede ya harus OK dipajak tinggi.\n\nMau smol government ya berarti korporat ngisi sisanya.\n\nKalo pingin Internet bebas ya konsekuensinya ngadepin orang bego / bahaya / 4chan.\n\nMau komunitas? Ya konsekuensinya tetangga resek. Mau privasi? Ya konsekuensinya anomie dan atomisasi dan masyarakat yg dimana orang mati terus gak ditemukan sampe 6 bulan kemudian pas mau di kick dr kost / apartemen, yang berevolusi jd kayak Korsel dikerjarodini sampe ancur demi pensiunan generasi tua 50 tahun kemudian\n\nDan seterusnya. Mau kesempurnaan ya di surga, bukan disini. (Sekarang paham kenapa afterlife itu makes sense?)\n\n- Presiden itu kekuatannya jujur terlalu OP dan harusnya dikurangi.\n\nPresiden di hierarki perundang-undangan bisa bikin PP DAN Perpres. Bisa bikin Perpu, bisa ngajuin UU.\n\nMenurutku sih UUD harusnya diamandemen biar jd UUD Civil Law yg baik (UUD Civil Law biasanya agak panjang), terus di UU lanjutannya maupun lainnya itu PerPres dihapus dari hierarki.\n\nPresiden bikinnya PP dan harus koordinasi juga ama Menteri, harusnya juga Tap MPR dihapus total dan UU harus dibuat biar lebih detail.\n\nKenapa, Presiden itu terlalu OP sementara DPR cuman bikin "Aturan main lanjutan", bukan "Permainan.\n\nSaat ini UUD itu "Aturan main dasar", UU = Aturan main lanjutan", PP kebawah baru permainan. Harusnya UUD itu "Aturan main" dan UU itu udah "permainan".',
 '&gt;Fridman almost exclusively platforms and agrees with right-wingers, I checked his twitter last week and the last 5 he hosted were anti-vaxx, climate change denialists or anti "woke" grifters.\n\nI got curious and checked his youtube. Jordan Peterson was the only controversial guy he interviewed in the last 5 episodes. The rest were a comedian (Duncan Trussell), a poker player (Liv Boeree), a chess grandmaster (Magnus Carlsen) and an African entrepreneur (Magatte Wade). I doubt that all of them were right-wing grifters or climate change denialists...',
 'You\'re missing that 1. no one said this was particularly much, 2. Previous Russian losses were when they were aggressors, now these losses are when they\'re the defenders. It\'s always much harder to attack than defend, but Ukraine was still successful, and 3. any Russian loss is a good thing.\n\nOr are you thinking like, "we already know people are killed/climate change is fucking up our planet/companies exploit workers, no need to report news about it anymore"?',
 "With climate change, there will be massive migration to Spain. There's a billion people in Africa, 400 million in the Middle East, and 2 billion in South Asia. How many millions will make the trip to Spain?\n\nAs we see in Pakistan, with estimates of 10-50 million people displaced  (vs. 47 million people in Spain) or impacted by the flooding, there's a lot of people that will be moving to Europe and plenty of them will be heading to Spain.\n\nFurthermore, since by 2100 climate change Spain massively - I suspect a lot of people from other regions will be moving to Pais Vasco or Galicia, places with cooler climates and more water.",
 "They can interpret their great book of myths to mean anything they want. Something about climate change finally penetrated their think skulls and scared them so now their book supports climate action. But they could flip flop tomorrow and declare now it doesn't.",
 "There will probably be studies in future.\n\nBut working out direct+indirect effects will be really hard. CO2 emissions of blown up stuff are small fry for climate change - humans emit tens of billions of tonnes of CO2 per year so this war is a blip.\n\nHigh fuel prices mean less use and a strong push for efficiency and faster renewables buildout, but that'll take time to appear. Short term you'll also see switching from gas to coal and oil in power plants.\n\nMy gut feeling is long term less CO2.",
 'See guys. Climate change is a good thing. Look at all the legendaries we are going to find.\n\nI hope I find an Isu bow before we all die.',
 "They don't really, they call it climate change because small minds like yours can't understand that it's not as simple as 'weather gets hotter and if it dosent its all fake'",
 "Tell it to the Republicans who've been tirelessly working to deny and downplay climate change and action for mitigation.",
 "Climate change. If you choose anything else, you don't understand climate change enough. Climate change will either make all issues worse (consider the 1 billion climate change refugees) or redundant, because there is no society left to care about those issues.",
 'Probably due to climate change',
 'Carbon tax will fix climate change in a second. The problem is how to make one that works',
 "no, weve avoided an extinction event, but climate change and it's effects will be irreversible in less than 3 years",
 'Rivers drying up while major flood in Pakistan. Looks like climate change to me.',
 "Small country, environmental rules that are archaic, lot's of demand from both indigenous people and immigrants, old city centers that require a complex balancing act to keep habitable, cheap money, ie low mortgage interest rates which raises prices, rising prices for raw materials and manpower for building and maintenance, a tax system that taxes income more then wealth, thereby making owning multiple buildings lucrative, and here we are.\n\nEventually some parts of that complex equation will change, in 25 years or so, the babyboomers will start dying, and free up houses. All of the environmental troubles will be negated with technology, money and by sheer necessity, but it will always be a small piece of earth, the delta of a couple of rivers, with a lot of people living in it. Basically we're Bangladesh with more money and a colder climate.",
 "Czech here. SPD are pretty much right-wing populists, they'll stand for anything if it means more votes. The real fascists are Trikolóra. They're anti-immigratiom, anti-LGBT rights, and they deny climate change. Oh, and they want to overthrow the government.",
 'Funnily enough the climate change provision is actually there to address climate change.',
 'Climate change? Old and busted. New hotness: energy drinks... /s\n\n[https://www.express.co.uk/life-style/health/1660882/blood-clots-risk-energy-drinks-signs](https://www.express.co.uk/life-style/health/1660882/blood-clots-risk-energy-drinks-signs)',
 "The fact that you can't tell tells us something. We are living in a satire. I soon expect serious calls to deport woke people from Texas to Cancun because they are too weak to withstand Climate Change.",
 'Depot 1: ca 75% vom Vermögen aufgeteilt auf 4 ETFs (EM (8%), EU (16%), World (24%), World Climate Change (26%) und 2xSPY(26%) und ein Cannabis Penny der nichts mehr wert ist (-94%) den ich aber aus sentimentalen Gründen halte um mich an meine Dummheiten zu erinnern. Die Aufteilung der ETFs ist auch einfach frei Schnauze, kein System dahinter. Bespart wird genauso - ab und zu mal ein paar tausender nachschieben wenn man auf Arbeit beim Kacken sitzt :D Sparpläne wären wahrscheinlich sinnvoller... Achja und ich hab noch einige Unzen Gold im Safe liegen, die ich vor Jahren mal gekauft hab und nen Bausparer (jaja ich weiß :P), den damals meine Eltern abgeschlossen haben, den lass ich auch noch weiterlaufen weil es 1% Zinsen gibt, quasi mein Notgroschen. (Ich bin über 35, ich darf das :P)\n\nDepot 2: IBKR Trading Depot mit stetig wechselnden Einzelaktien (keine Derivate). Momentan noch EDU, PANW, WOLF, COGT und AXSM aber da hab ich immer Stops für alle Positionen. In dem Depot sind ca 6% vom Vermögen, aber natürlich ist das Ding tiefrot.. :)',
 "It stops where the law and the science say it should stop. I imagine this policy is strictly when these conspiracies involve public safety. You also have to ask at what point does the freedom of expression of doctors put the public's health at risk? Another comment mentioned, there will always be peers who review your work. There's anti climate change scientists and anti vax doctors, but they shouldn't be taken seriously for a reason. This is for a profession directly involving public health. This makes any doctor think twice about spreading anything but what's accepted by the available peer reviewed research",
 'After one of the strongest typhoons ever landed on my country, a Filipino inquiry found that big polluters are legally liable for climate change. It barely made the news.',
 'I like it when a country that has a large population of climate change deniers do a collective surprise Pikachu face when their hyper capitalist destroys their own source of survival.',
 'Farmers are the ones *causing* climate change and environmental damage, but you think they want to preserve farming in the name of environmentalism?',
 'The tweet is from a German climate change disinformation group. Note the graphs in the tweet do not back up the claim. Please downvote this post and/or report it.',
 'I’m hoping we’ll figure out some new tech that can directly address the problem but I get the whole hey man the worlds gonna be screwed method of reporting climate change because people usually aren’t down to change so easily so to get things moving In the right direction you gotta light a fire under folks asses',
 'Farmers are gonna be hit first and hardest by global warming, you know that right? Or do you deny the existence of man-made climate change too?',
 'He means that the weather has  happened before climate change worries were a thing.',
 "Why does the weather need to go up to 100? In normal day to day life how finely grained an understanding of the climate do you need? I put on a cardie or light jacket sometime between 15 and 10, a woolen jersey around 5, an overcoat as we get to freezing, and full winter kit below that. Above 25 I'll roll up my sleeves, above 35 I'll lie in the shade and fan myself (as a fat man from a temperate country I don't do heat well). \n\nCan you really tell me that you will change your clothes or your plans going from 50F to 55F. I don't think most people would even notice that difference and you're using 5 whole degrees for it",
 'When adults set the example of not following social distancing rules, when we\'ve all been at each other\'s throats over politics for over six years now, when "parenting" is giving a kid a tablet and an internet connection, and when we get more news seemingly every day about how the world is going down the shitter—whether due to climate change or something else—why should we be surprised that our children aren\'t properly initiating themselves into the social contract?\n\nEveryone is so hopeful for Gen Z, but I fear what the lockdown generation will become . . . and frankly, whatever it is, we deserve it.',
 'While I’m right leaning I do believe climate change is an issue. Although I don’t believe its an issue caused by individuals per se. Most people I know recycle, bus or cycle to most places, do whatever they can to help the environment.\nMajor companies/corporations without faces are the ones pillaging the earth for profit and leaving a deadly trail in their wake. \nThey destabilise poor countries, oceans and forests while painting a good image of themselves through bs words like sustainable, environmentally safe, bla bla bla. When asked what does that mean they have no idea, they can’t explain how they sustainably destroy entire ecosystems. Then comes the media with their endless fearmongering and guilt tripping normal people who do the best they can into thinking its their fault the world is going to shit!',
 "They weren't communist. I'm afraid you are victim of a decades long anti-communist/socialist and pro capitalist propaganda. \n\n\nCapitalist dogma is ingrained into you since you were a child. Keep defending the status quo as the material conditions of the working class continue to deteriorate. Keep believing the capitalist dogma that nothing matters more than never-ending profit growth (on a planet of finite resources &amp; ever-increasing global human population) as the world grapples with the horrors of climate change and crony corporatist greed.\n\nYou don't know shit about neither communism nor socialism. Only what the nation state and corporate media feed you.\n\n\nWatch this video. Maybe you'll learn something.\n\nhttps://youtu.be/fpKsygbNLT4",
 'Pakistan Floods Caused by China?\n  \n\n\nNot content with destroying their own ecosystem, the Chinese / CCP employed extensive cloud seeding to create rain in 10 provinces, pumping hundreds of tons of silver iodide into the Atmosphere.\n  \n\n  \nThen almost immediately Pakistan suffers it\'s heaviest rainfall and flooding in 30 years.\n  \n\n  \nWhen I put two and two together I thought I was being paranoid, then I read a GlobalTimes headline:- "At least 10 Chinese provinces use cloud-seeding to combat heat wave; ‘no effect’ on climate change, ecosystem"...\n  \n\n  \nhttps://www.globaltimes.cn/page/202208/1273964.shtml\n  \n\n  \n\n\n  \nThis deserves a through investigation by SerpentZA, Laowhy86.',
 'God damn climate change...',
 "We are on our own in this world in every sense of the word, Unless you're white or 'European' as they put it, You're ducked, Pakistan should start making preperations for climate change.",
 'And we wonder why there is manmade climate change with genius ideas like that',
 'Season 8 GoT  lol and inaction on climate change… yeah that too..',
 'Stopping climate change',
 '&gt;mass consumerism will end anytime soon is fanciful.\n\nClimate Change: Hold my CO2.',
 'Unfortunately, Climate Change has been politicized, and we know the current state of the Politics. \n\nImran Khan had good ideas though',
 'That sounds like a policy from the 1970s or so. With climate change and everything, how is that even possible.',
 "They properly replicate his methods many times, in different part of the world, but didn't manage to replicate the results. In science, it mean it doesn't work. I don't know what kind of proof you want. Here another article explaining why it's really unlikely that any kind of grazing technique will reverse climate change : https://skepticalscience.com/holistic-management-rebuttal.html\nMore technical version : https://skepticalscience.com/holistic_management_climate_change-advanced.htm",
 "Stop voting for climate change initiatives. Climate change activism is a scam that's endangering all countries. The US is starting to wise up.",
 'Bernie considers climate crisis the most valuable change to national security. He opposed the manufacturing of the Dakota Access Pipeline on the grounds that it will have a notable effect on the climate.\n___\nBernie Bot -Bernie_Bot_00',
 "What is the climate you're cycling in? Is there a step hill? Even with an ebike environmental factors will cause you to perspire. Also if you pedal a bit on the furious side your body will be still generating a bit of excess heat when you dismount, give yourself 5 minutes to cool off before entering the workplace. You could also try switching tops for something light while you ride, then at least that way you have a minimal change at the other end",
 'Agreed.\n\nClimate change is something he understands how to solve, and so he’s looking past it to what the next threats would be. \n\nReproductive preferences of a myriad of different people is something he doesn’t seem to understand as well.  So he’s simply extrapolating current trends far into the future.',
 '... Connecticut, Cupido, and Climate Change?',
 'Bernie considers climate crisis the most valuable change to national security. He opposed the building of the Dakota Access Pipeline on the grounds that it will have a big result on the climate.\n___\nBernie Bot -Bernie_Bot_02',
 'I just saw that china is pretty much boiling ( as in rubber shoes melting into the road ) , and Bengaluru is getting hammered\nMan climate change is insane',
 'Pretty sure we fucked nature. This is not normal. Climate change is real.',
 'No. It is 100% undeniably due to water and energy usage.\n\nhttps://www.drroyspencer.com/2022/08/lake-mead-low-water-levels-overuse-not-climate-change/',
 'To stop climate change.',
 'Is it just me or is everything that Extinction Rebellion does completely antithetical to their goals? Like if I were Exxon and wanted to make people hate climate change activists I would fund a group like Extinction Rebellion.',
 'Awareness for climate change, they apparently play a message in the stadium in that minute.',
 'Climate change is transitory, we could not see the transitory transition coming.',
 'If you buy climate change I have a bridge to sell you',
 'Climate change will be like recession where the establishment denies it until its very obvious and then they will pull a janet yellen "we could not see this coming"',
 'Bernie considers climate crisis the most timely change to national security. He opposed the establishment of the Dakota Access Pipeline on the grounds that it will have a huge influence on the climate.\n___\nBernie Bot -Bernie_Bot_03',
 'calm down, ABC. The document release this article is based on is the ESOO. \n\nWhich stands for the \n\nELECTRICITY STATEMENT OF OPPORTUNITIES\n\nand comes out each year. notice the key final word there, o p p o r t u n i t i e s\n\nhttps://aemo.com.au/en/energy-systems/electricity/national-electricity-market-nem/nem-forecasting-and-planning/forecasting-and-reliability/nem-electricity-statement-of-opportunities-esoo\n\nThe document exists for the purpose of saying clearly what needs to be done, so that developers can go and build new capacity into these specific gaps, of which there is already pipeline for some of what’s stated as gaps. This is how the process works!\n\ntldr: the sky is not falling\n\n(ok except for climate change, and of course yes the ukraine war driven fossil fuel price apocalypse, and maybe a shooty war in the taiwan strait in a few years and yeah pakistan is in a real bad way rn my god, but just not this thing specifically)',
 '&gt;Claims of anthrogenic climate change would be more persuasive if they were not being used to pass laws that massively enrich corporations whose owners/management  were not politically engaged with the Democratic party and who do not donate massively to the Democratic party.',
 "I remember seeing the quote contributed to Bill Nye, but it was said we could quit trying to convince climate change deniers. It was going to get to the point they couldn't deny it anymore. I wana say this has to be one of the first signs, but someone is bound to call it fake news.",
 "Yo momma's dryness is the reason climate change is happening.",
 'Claims of anthrogenic climate change would be more persuasive if they were not being used to pass laws that massively enrich corporations whose owners/management  were not politically engaged with the Democratic party and who do not donate massively to the Democratic party.',
 'I should have worded my post better. I don’t mean admit defeat - but I do mean admit that a certain amount of climate change is likely to happen no matter what. \n\nAt the moment there’s a lot of false hope going around and people seem to think that if they recycle their rubbish they’ll stop it all, which isn’t true. I understand it’s a hard balance to strike between hope and fear, but it seems strange to be allowing people to continue to buy homes in low laying areas (for example) when we know they are likely to be under water within 100 years.',
 '“Mitron, duniya khatam hone aayi he isliye hum Bharat Vikasit Desh 2047 ka sapna pura nahi kar sakte he.”\n\n- Modi from wheelchair with portable ventilation attached to him\n\n“Lekin mai yeh bhi kehna chata hun mere pyare desh vasi o se ki hame ladna he is bigdate hue climate se, Nehru parivar ne jo liberalisation kiya is vajah se aaj Bharat me ye sari musibate aayi he, vo chahe pur ho ya samudra ki badti hui rekha, dukal ho ya garmiyon me baarish. Aagar aapne muje chuna hota 1947 me to main Bharat pe ye saari aaptiyan kabhi nahi aane deta. Aap 2049 ke chunao me BJP ko jitaiye aur dekhiye kese ham vishwaguru bankar ye sara climate change kar dete he.”',
 "It's the climate change!",
 "if everyone on earth consumed and generated as much waste as the average person living in the west we'd accelerate climate change by like 5 times",
 "Any article that talks about 2050 without addressing climate change and climate refugees shouldn't be taken as gospel.",
 "I stopped taking you seriously after you said Canada's RE scene is affordable,property prices are only going down there because no one is buying them. And Yeah it's a wonderful idea to build cities near river banks ,when we are victims #1 in regards to climate change and facing floods the size of Germany. What a big brain idea.",
 'Scary thought. Who decides what is "misinformation" and what isn\'t?  It\'s line this climate control crap. Just follow the money. You can hite an "expert" to find anything in your favor. Thats what the government does. 90% of the scientists who claim global warming...I\'m sorry I meant climate change since the global warming fear was a HUGE bust, the scientists didnt specialize in weather. They were in other fields. Much like Bill Nye being an alleged scientist when he\'s actually an engineer.',
 'Fight climate change?',
 "It's not a bad idea to do a PID tune, but I would first try printing something now that you cleaned the thermistor port.\n\nIt's a change-one-thing-at-a-time thing. That way you know whether the paste was the culprit. Unless you live hiiiigh above sea level, or are in a climate that's ungodly humid/cold/hot, factory PID values are surprisingly decent as far as defaults go.\n\nBtw, you can always flash the PID back to default by running a gcode file with the following two lines in it:\n\nM301 P19.98 I0.88 D50.29\n\nM500\n\n(This code is for Ender 5 Plus, btw; not sure what values other printers may have)",
 'Lol.  Should be climate change.',
 "It's not inevitable. Our governments are just too cowardly to implement the policies to prevent it. Climate change is an active policy decision.",
 'Win the fight against climate change?',
 "I work in the environmental movement, and environmentalists aren't supporting nuclear power. Anyone working analytically with energy rejects nuclear power as a mitigation for climate change due to: cost, time, political instability (see ukraine), environmental instability (king tides, rising sea levels, drying up of rivers/lakes/reservoirs), difficulty of nuclear waste storage, the weakness of undiversified power grids, and of course the availability of cheaper, more efficient, and actual renewable energy.\n\nAnother key point are the learnings from today's situation with huge fossil fuel companies. Their political and economical power/control has made falsification of science possible. It has also disabled citizens, ngo's, and researchers to take any meaningful part in decision making, meaning that there is a huge rejection of creating similar structures with new energy - whereever it may come from.\n\nThe only people i hear talking about nuclear power are run-of-the-mill citizens with little to no experience in energy politics, as well as governments which focus on short term solutions rather than long term",
 '"“Bio-physical risks” such as the potential for city wildfires to become more common had not been assessed in the report, he added."\n\nMaybe it\'s not just the pollution, it\'s the people dying inside the burning buildings. \nAlso, where I live buildings are by law required to have a strong foundation. This can\'t be done with wood, I assume. Let alone building skyscrapers. \nAnd while they are right that more surface would be available for planting trees if people stopped eating meat, first you have to get people stop eating meat... \n\nSorry for playing devils advocat. I\'m happy to read counter arguments.\n\nEdit: this article seems to go into some of the stuff I said, including constructing multi-story buildings: https://thehill.com/policy/equilibrium-sustainability/3621018-high-tech-wooden-cities-could-fight-fire-climate-change-study-says/',
 "Population has a momentum. India has achieved replacement fertility levels in most states.. china on the other hand is looking towards a demographic collapse due to forced one child policy. Rather than the voluntary population control policy followed in india. \n\nI don't act like india is perfect. I am simply calling out hypocrisy &amp; lack of understanding wrt climate change.\n\nCalling out pollution in indian cities when US cities are more resource heavy is what I'm pointing out. For e.g. you eating corn fed cow's steak is detrimental to environment.  And so, is your personal car. \n\nWhile most people in china &amp; india rely on public transport for such things. And don't eat as much beef.",
 'Reversing climate change.',
 "climate change isn't just caused by singapore. Its a global phenomenon and even if singapore reduces its cardon emssions and plants more trees, the climate change is still going to happen. (I'm not sure if we will experience it slight lesser, willing to be educated haha). In that case, many people might still prefer to use cars to travel especially people who have to travel very long/ who stay very far away from work/school.\n\nI mean personally i travel 1hr 10min x2 to go school and back with public transport. And its superrr crowded in the mornings. I understand if there are people who want to use cars. Image the families with 3/4 kids and elderly.",
 '&gt;He’s had Mark Zuckerberg and Ray Dalio on.\n\nNot really helping the "not a right-winger" case here. \n\nFridman almost exclusively platforms and agrees with right-wingers, I checked his twitter last week and the last 5 he hosted were anti-vaxx, climate change denialists or anti "woke" grifters.',
 'As much I have a lot of problems with California, I’m thankful about it and staying. Sure things are expensive and that should be controlled, but I’ve seen so much changes to help out the cities and finding ways to combat climate change.',
 "You're saying climate change like something that is going to happen, it IS happening and there's fuck all we can do.",
 "True, but they figured out that they don't need to. They are just a step ahead of the curve. With climate change coming up neocolonialism is around the corner. Somehow wealthy governments will have to keep the poor of Africa, South America and India in check to extract resources and prevent mass migration. In my opinion it looks bleak for us.",
 'The image is extremely misleading and 1/3rd of Pakistan is not currently a lake. The flood was definitely catastrophic and massive and affected the whole Indus valley though. Flooding in that area happens every year, just not usually to this extent. The flood waters will disperse into the surrounding farm land and out to the ocean over the course of the next week unless more rains come through.\n\nOP of the bottom right image claimed the source here:\n\nhttps://reddit.com/r/interestingasfuck/comments/x0vk7o/_/imbod87/?context=1\n\nIf you go to zoom.earth you’ll notice this image is what Pakistan always looks like on that website, before and after the flooding there is no change. The poster in this thread then compared it to a google earth image which makes it seem like you can see a lake from space, but the blue/dark area is just a filter zoom.earth uses to make their live satellite imagery seem higher definition that it is. \n\nHere is a source with pictures that will give you an accurate idea of the scale of the flooding, including a real satellite photo:\n\nhttps://www.npr.org/sections/pictureshow/2022/08/30/1119979965/pakistan-floods-monsoon-climate',
 "In any case, this is a direct consequence of the option germany took. It's the price it is paying. For protecting the fringe non EU/non NATO countries\n\n&gt; and invade another country a few years later \n\nNot NATO and not Germany in any case.. \n\nThe one clearly wrong decision germany took was to ramp down nuclear.\n\nAnother might have been doubling down, with start of NS2\n\nIt's not as if energy choices have no other ill effects. Costs, industry boosting, and climate change among them\n\nAnd the US supports Saudi Arabia via its policies, which some also warn about ; the us is no more likely to change that than germany was; and germany at least had a plausible theory at the time.",
 'Unfortunately based on the dumpsters at every single complex and store, even people with houses have no respect for throwing away things properly or even getting it in the can. Not to mention the people who constantly throw trash out the car windows or any of the other scenarios where people that are more fortunate are total pieces of shit and contribute to the demise of our planet, like who is really making a bigger impact on climate change, the president of a growing fast food company that levels ecosystem’s paves them over, throws away thousands of pounds of food a year, constantly having running vehicles outside, contributing to light pollution etc. or the homeless guy that lives in a tent, has no car, walks ir bikes around, throws trash in their general vicinity and doesnt have the money to be buying the unnecessary amount of food the average person buys which all goes in a landfill anyways or into the sea via excretion’s? We are all just humans trying ti make it by and nothing really matters except doing what you can to stay happy and keep the people around you happy.',
 'For real, a lot of people here get very indignant at pod complexes, eating insects, and voluntary sterility…but keep validating all the climate change alarmism. Even if this is all the bourgeoisie’s fault, it’s still the necessary thing to do to save the biosphere.',
 'Bernie considers global warming the most essential change to national security. He opposed the building of the Dakota Access Pipeline on the grounds that it will have a noteworthy influence on the climate.\n___\nBernie Bot -Bernie_Bot_01',
 'please tell him that you can’t fix climate change under capitalism 💀',
 'Anyone that denies Climate change is a fool or have never read history. Creation? Religion is only good in small doses. We are very similar in a lot o our views my bro. Please give me a chance.',
 'Yea they just got a $1.2 billion dollar donation to Leonard Leo, who spearheaded the construction of a conservative supreme court supermajority to end abortion, block government regulations, stymie the fight against climate change and limit voting rights.\n\nIt was an anonymous donation transferred to a tax-exempt trust. There is no public record of it. ProPublica and the Times uncovered it. \n\nMy guess is Trumps legal fees have been draining them dry. He also does not share any of the money he receives from campaigning — which is hundreds of millions. Sounds like they’re tired of footing the bill for their rich master who regularly hangs them out to dry.',
 'Who leads america has a massive impact on climate change',
 'Bernie considers climate crisis the most valuable change to national security. He opposed the establishment of the Dakota Access Pipeline on the grounds that it will have a huge result on the climate.\n___\nBernie Bot -Bernie_Bot_03',
 "Close, he's dedicating himself to fighting **for** climate change and creeping fascism.",
 'Would Reagan have passed the biggest climate change legislation in our history? Would Regan have forgiven 10k of student loans (plus billions more from fraudulent universities)? Would Reagan have invested a trillion dollars in infrastructure?',
 'Life/planet hack: vote man made climate change deniers out of office',
 'I agree with Mighty_Lu_Bu. I agree with your logic re: the malthusians, but don\'t think they\'re a meaningful cause of the trans-movement because I think there are larger/more powerful explanations for it and because, IMO, the people most susceptible to getting sucked into the trans-movement are confused gay/lesbian kids who very likely wouldn\'t procreate anyways (so why push a movement that would mainly sterilize kids who weren\'t going to procreate anyways?).\n\nIn addition to a confluence of other causes, I think the trans-movement ("TM") is mostly a result of the susceptibility of humans to mass-delusions, the fact that supporting the TM became an easy way for shameless/delusional liberals to virtue signal about how immensely compassionate/progressive/open-minded they are, and the fact that tons of intentionally and unintentionally psychopathic institutions are pushing the TM (despite the fact that it\'s mutilating confused kids) because in some direct or indirect way it could (or will) yield them more power/money/influence.\n\n&gt;They make abortion into a human right because they want there to be less babies.\n\nMakes sense, but I think more importantly pro-lifers just want to be able to hedonistically have irresponsible sex without having to deal with the potential consequences of doing so (making abortion legal also helps alleviate their guilt of killing a thing that would turn into a human ["if legal, then not bad/immoral"]). I think it\'s pretty compelling from a fiscally conservative perspective too, and that is, "if we don\'t let these irresponsible people abort their potential future kids we\'re going to have to pay to help raise them."\n\n&gt;Then they go talk about the horrors of climate change and overpopulation. I wonder what "fixes" those "issues"? Oh yeah, making less babies.\n\n&gt;They convince healthy, straight, left-of-center people that it is selfish to marry and reproduce, and that it\'s something to be ashamed of, because the net result of that is that they make less babies.\n\nMakes sense, but I think that\'s better explained as an unintended consequence of extreme eco-anxiety caused by the green movement (with hyper-agreeable &amp; very low-self-esteemed people being the most likely to fall into that line of thinking).\n\n&gt;**They** keep testing the waters with "Minor attracted persons" because if they can normalize sex with children, then that is less likely to result in viable offspring, and the trauma inflicted upon their victims is more likely to end up falling for one of their other traps that makes it so they have less babies.\n\nNo clue what you\'re talking about here. Who is this they you\'re referring to? This seems very conspiratorial...\n\n&gt;It\'s blatant genetic warfare. They\'re attempting to get the lower castes to remove ourselves from the gene pool.\n\nNo. That\'s flawed logic/not knowing the actual statistics... Birth rate is negatively correlated with income. That is, statistically, the "upper castes" have less kids than the "lower castes" (when you\'re .\n\nSee data: https://www.statista.com/statistics/241530/birth-rate-by-family-income-in-the-us/\n\nhttps://en.wikipedia.org/wiki/Income_and_fertility',
 "Solve climate change but don't make me sacrifice anything!",
 "I can see where you're coming from. I just think it's a way to cope, you know? I'd rather ponder over a weird celebrity's dating life on the internet than worry over politics and war and climate change. Yeah, there are better ways to spend your time, but it helps to get your mind off of the events of the world.",
 "Yeah you don't seem to be getting the point of my post at all,\n\nThe point is that people who rush in with the message YOU MUST CUT OUT MEAT AND DAIRY COMPLETELY FROM YOUR DIET!!! Actually drastically hurt the cause of animal welfare and climate change by making people think of it as an All or nothing thing instead of something that they can participate in at whatever level they're comfortable",
 'Less than allowing climate change to continue unrestricted.',
 'It’s pretty 50/50. I just made a poll asking republicans if they believe the government should do something to stop climate change and 18 said yes and 16 said either climate change isn’t real or it is real, but nothing should be done.',
 'Minneapolis try not to change the political climate in our country and possibly the world challenge (damn near impossible!)',
 'I don’t believe it’s as simple as you make it out to be. \n\nThe US for example a country big on energy and consumption, could have a population of, let’s say, 10 billion, almost 30x more than what they have now. But if that consumption is entirely fuelled by renewable energy, then what? Are they still worse than a country with a growing 50m population that do not use any renewable energy sources? \n\nBtw, all this presumes human activity is the main source fuelling climate change in the first place.',
 "Yes they are.\n\nThen they'll blame climate change, collapse the economy bring in martial law and offer a social credit system as a solution to the problems.",
 "The first law of thermodynamics IS refutable. Like every other SCIENTIFIC statement/hypothesis/theory, it has to be falsifiable. That's the core of science.\n\nPeople seem to think that me saying climate change being based on models instead of the true definition of science means that I think the climate change models are wrong. I don't. I just don't recognize it as science.\n\nFact is, there are tons and tons of factors outside human control that also account for climate changes. Therefore it is hard to correctly measure the true influence of human influence on the climate, sinxe we can't measure the influence without these external factors.\n\nThat doesn't mean I think it's a bad idea to switch to different types of energy regardless. Just the amount of people near large factories getting cancer and other illnesses, along with the fact that our current usage of fossil fuels won't be sustainable makes green energy a good investment regardless of our influence on the climate. I also think we have enough data to create really, really acurate models on the climate. But that still doesn't make it science.",
 "Things that have largely replaced any grocery purchases:\n-- Apples, pears, quince. For whatever reason, cherries and plums are a lot flakier to pollinate, she is years better than me others\n-- Aronia and blackberries. Aronia isn't the tastiest berry, so naturally it's the most prolific\n-- Garlic, elephant garlic, leeks\n-- Kale and chard. These are essentially self-seeding at this point. Also a variety of perennial greens such as sorrel\n-- Squash, all types that we would ever want to eat\n-- potatoes. Mine originate from true botanic seed, so the cost-benefit is better than tuber-grown potatoes\n-- Green beans &amp; shelling peas\n-- dried favas and soup peas\n-- Paprika and hot sauce peppers\n-- Tomatillos and eggplant\n-- Fresh herbs, green onions, chives, basil frozen in olive oil\n\nNever enough of:\n-- Tomatoes. I can ALWAYS use more. They are often not prolific in my area so it's an annual battle\n-- Strawberries. Often the slugs get them\n-- Currents and gooseberries, plagued by diseases, sigh\n-- Lettuce, that fickle thing that bolts\n-- Most other root vegetables, often because I'm bad at succession sowing\n-- Dry beans, I grow lots of varieties but not in adequate quantity\n-- Bulb onions, somehow never sow and thin seedlings appropriately\n-- Most brassicas besides kale, again with timing/spacing/succession sowing\n-- Grains as a category, glad there are professional farmers near me to buy these from\n\nMaybe someday the plants will get big enough:\n-- Nuts &amp; blueberries (irrigation woes)\n-- Figs and pine nuts (slow growing)\n-- Olives ( the climate change bet)",
 'I remember griping about “eight-month winters” way back in the 80’s, when you could count on the first flakes falling in October and the only months guaranteed to be snow-free were June-September.  These days?  October is still a bike-riding month.  Got in a ride in November once. And every few years the extended deep freeze we usually get in January doesn’t really happen, replaced by shorter one or two-day freezes between thaws. You can’t convince me climate change isn’t real.  I’ve been watching it happen with my own eyes.',
 '&gt;you can’t set a standard of what you will believe and discount anything that doesn’t pass your bullshit meter\n\n\nYes you can? And you absolutely should, whether you’re discussing ghosts or climate change or whether a child stole a cookie after they were told no. That’s the *only* way to interact with other people.',
 'That is not how the world works,   everything is gone when a asteroid hits.  this also makes  the  pollution and climate change a non issue.. as everything will be quickly wiped away in just  a few hours',
 'No, no it is not. Consecutive 99°+ is for mid-July to mid-August. It’s way too late in the season for a hot streak like this. \n\nBut hey, climate change isn’t real, am I right?',
 'I\'d *love* to see some empirical data to support that statement. We\'re importing immigrants for the vote now? Puh-leaseeeee.\n\n"Majority of Americans"? We know that the majority of young voters--Millenials and Gen-Z-- are overwhelmingly liberal. Even if that doesn\'t constitute the majority now, it\'s the future.\nSources: https://www.dataforprogress.org/blog/6/14/progressives-control-the-future\n\nhttps://www.pewresearch.org/social-trends/2019/01/17/generation-z-looks-a-lot-like-millennials-on-key-social-and-political-issues/\n\nSocial issues and climate change are now at the forefront. The Republican party\'s unwillingness to recognize bodily autonomy of women and trans-men, climate change, systemic racism, and gun regulation is absolutely disgusting. Liberals aren\'t in need of outside help to make the Republican party unappealing.',
 "I don't know shit about anything and don't live anywhere near there. \n\nI figured insurance in Florida would be expensive and hard to get. Who would want to cover a house that is more likely every year, because of climate change, to get blasted by bigger and bigger hurricanes. I've heard there are a lot of places where sink holes can open up at any minute too not to mention how much of it all will be underwater when sea levels really start to rise.",
 'Bernie considers climate crisis the most valuable change to national security. He opposed the establishment of the Dakota Access Pipeline on the grounds that it will have a notable result on the climate.\n___\nBernie Bot -Bernie_Bot_02',
 'Artificially placed trees in such low density do not contribute to reducing the effects of climate change.',
 'C-c-c-c-climate change!!!!!',
 'Probably still the lowest of all energy creation methods when accounting for pollution, climate change etc.',
 'Humans have affected the planet for sure.  Is the planetary climate change out of the regular flux if you go back a few\nMillion years?  No. \n\nOk now you can vote me down for not following the alt left hive mind.',
 'Awwwwwww you really said it, man. I just want to buy a bumper sticker now that reads: "I prefer systemic change over climate change".\n\nWhat are the root causes? Testosterone?',
 "I think it's more a matter of making sure your core values align and the rest will flow naturally. If they're a climate change denier and think you're a lunatic for prepping for example, that's indicative of a misalignment that runs deeper than just chuckling at you for how many tins of beans you own.",
 "One of the 1st rules of understanding climate change is: Weather isn't climate.\n\nNo monsoon, snowfall, heatwave, or cold snap, etc. is relevant to climate change.",
 'You didn’t actually read the CPC policy declaration did you? \n“We believe that an effective international emissions reduction regime on climate change must be truly global and must include binding targets for all the world’s major emitters, including China and the United States.” \n\nMy guess is the other cons were opposed to adding that extra line because they opposed O’Toole’s carbon tax which does nothing to actually help the environment, but just contributes to inflation and a high cost of living. \n\nO’Toole lost because he was a spineless tool who betrayed conservative values. We tried “progressive” it didn’t work. Time to go back to a leader who is actually fiscally conservative to reduce the debt Trudeau has put us in. \n\nGive me some actual proof of CPC leaders trying to “take away peoples rights” instead of just repeating Trudeau’s virtue signaling and US based political accusations. \n\nCanadian inflation under Trudeau is significantly higher than every other first world country in the world last I checked. The “oh it’s happening everywhere” doesn’t excuse Trudeau irresponsibly putting us in more debt than every other prime minister in history put together and believing the budget will “balance itself”. Also, some of those supply shortages are due to the government’s strict regulations and covid policies. Have you taken a look at the increase in cost of fuel between south Western Ontario and Michigan? It’s massive! Know why? Carbon tax! One of the things you are saying is responsible for inflation other than government spending, is directly connected to government policy!!! Please open your eyes.',
 'Watch the HBomb vid on climate change, a lot of good clips of people saying obviously wrong things',
 'I love the narrative they take on children identifying as trans is abuse. Like trans people who are abandoned have higher rates of suicide, yet when their parents support and love them, that changed the narrative, illogically so and the party’s followers are so braindead they followed that theme. Vaccines that prevent disease and progressed the world medicinally is illogical toyed with and blindly agreed upon as ‘bad’, and they actually believe that shit without ANY facts, in fact, all the facts pointing out their idiocy. Climate change, I hope when the earth burns it GETS THEM BAD. I want earth to go all in these sad fs and get revenge. And teachers helping students feel comfortable about their identity, literally at times saving their lives and making them feel comfortable (saved mine) is a priority over the kids being spanked in southern states, the incest, the 200,000 kids sexually abused in conservative churches in the US just this decade.',
 'It is very much due to climate change. Hotter it is, more water the clouds hold, and the more water that comes down when it rains. + pakistan has the second most glaciers of any place in the world, they are melting',
 '“I’m not saying climate change isn’t real, I’m just saying you’re a sheep for believing it’s real”\n\nYou gave me examples of popular scientific ideas that later proved to not be true, and took their existence to mean that *no* scientific assertion is robust. This is disinformation.',
 "These mfers will always go ask college freshman randos to explain the intricacies of climate change or respond to dumb shit and then edit the videos when they can't do it and claim a W lmao.\n\nYou notice they're never approaching field experts. Wonder why? 🤔🤔",
 'They focus too heavily on climate change, but have some awareness of other huge factors: topsoil depletion/destruction, freshwater supply depletion/contamination (including aquifers), deforestation, biodiversity destruction (including beneficial insects), pollution of all kinds.\n\nThese reinforce each other and converge. The whole is greater than the sum of the parts.\n\nNever mind all the destructive ways people can respond to the consequences.',
 'So if it doesn’t rain it’s climate change but if it rains a lot it’s also climate change?\n\nMaybe the world has just always experienced extreme changes in precipitation on a regional basis..',
 'Bigger contribution to co2 is fuel from ships. The whole urban lifestyle is based on long distrance transport of goods. If you want to avoid climate change buy a car and shop locally!',
 'If climate change wipes of Pakistan then it is acceptable',
 'COVID linked to climate change',
 'Until the sun burns out, or more realistically, until climate change gets bad enough to completely upend civilization as we know it. So like 15-30 years. \n\nYou got time',
 "[**Blackfish City**](https://www.goodreads.com/book/show/35068768-blackfish-city)\n\n^(By: Sam J. Miller | 336 pages | Published: 2018 | Popular Shelves: science-fiction, sci-fi, fiction, fantasy, dystopian)\n\n&gt;After the climate wars, a floating city is constructed in the Arctic Circle, a remarkable feat of mechanical and social engineering, complete with geothermal heating and sustainable energy. The city’s denizens have become accustomed to a roughshod new way of living; however, the city is starting to fray along the edges—crime and corruption have set in, the contradictions of incredible wealth alongside direst poverty are spawning unrest, and a new disease called “the breaks” is ravaging the population.\n&gt;\n&gt;When a strange new visitor arrives—a woman riding an orca, with a polar bear at her side—the city is entranced. The “orcamancer,” as she’s known, very subtly brings together four people—each living on the periphery—to stage unprecedented acts of resistance. By banding together to save their city before it crumbles under the weight of its own decay, they will learn shocking truths about themselves.\n&gt;\n&gt;Blackfish City is a remarkably urgent—and ultimately very hopeful—novel about political corruption, organized crime, technology run amok, the consequences of climate change, gender identity, and the unifying power of human connection.\n\n^(This book has been suggested 2 times)\n\n***\n\n^(62822 books suggested | )[^(I don't feel so good.. )](https://debugger.medium.com/goodreads-is-retiring-its-current-api-and-book-loving-developers-arent-happy-11ed764dd95)^(| )[^(Source)](https://github.com/rodohanna/reddit-goodreads-bot)",
 'The answer isnt to ban future projects or exports. Its to tax the living shit out of it so that it will pay for our transition to of our grid and transport sector, so that it will help pay for the devastation climate change is and will cause our country. \nNorway is considered a very Green nation yet fossil fuels are by far their largest exports, however they use that revenue to invest in renewable energy as well as their pension fund.',
 'It’s not that it’s“not debatable” - I mean, we could debate the shape of the Earth if you want. \n\nIt’s more that there’s a vast and overwhelming amount of evidence pointing to manmade climate change being real, and much less evidence to support any other conclusions.',
 "Nah see you see, we need him there to sell real estate, since he can apparently sell homes under 10 feet of water from climate change to FUCKING AQUAMAN. \n\nContext if you don't get the joke: https://i.imgur.com/zFOSoKZ.mp4",
 'Use it on your phone for a few weeks and it will come easy to you. 20 is room temperature. 37 is human body temperature. 0 freezing, 10 is like cold weather during fall or something. 30 is the climate change hell we exist in right now.',
 "Don't let climate change catch you slippin",
 "So... what is the plenty of evidence to debunk climate change? If it's so abundant, and so convincing to you, I'd like to evaluate it too so I can have the full info please.\n\nYou keep saying it's wrong but never provide evidence of this.",
 'I too was disappointed in this episode. There were a number of proverbial elephants in the room. \n1. No mention of lockdown\n2. Completely skipped over any theories relating to why puberty is happening earlier (i.e. hormones in food or climate change, or simply saying "we don\'t know why")\n3. The use of C as a case study. For all we know C\'s experience could\'ve been driven by their specific gender dismorphia rather than a more common environmental shift that is applicable to the 90%+ of the pop that is cis. I get that TheDaily likes to interview non-cis young adults (see the Girl Scouts v Boy Scouts episode). But the fact that only C was highlighted, a presumably white non-binary person, was a huge miss. Including a cis boy or girl and/or gay or non-white person would\'ve done more to support The Daily\'s thesis that there *is* a generalized mental health crisis. \n\nTl;Dr: this episode was a huge disappointment',
 'This is an example of infrastructure not designed for the impacts of climate change.',
 '&gt;Can we please use this weekend for “climate change”\n\nI think there’s a typo in this sentence because I’m not really sure what this means.',
 'Yes, ideals that are further left than "climate change is major issue" are very dangerous. George Washington warned against political parties, and I think he would\'ve lobbied against extremist views as well. America fought Communism for a reason. It\'s not as bad as Nazism, but still dangerous',
 'Meh. Actually, most economists are well aware of climate change and comment on it frequently. They also are aware that capitalism is deeply flawed. You might want to read what actual economists have to say before passing judgment on an entire profession.',
 'Maybe stop polluting the valley and stop climate change denying… then God might magically “bless” you with Devine intervention.',
 'This is a flood plain.  Climate change or not; populations building there or not...it would have flooded!',
 'Yeah climate change is gonna wreck a lot of the world and we ain’t remotely prepared for it',
 'Real climate change',
 "Oh gosh, another victim of climate change. I'm betting she took a nap or something and that's what did her in.",
 'Limiting climate change',
 "I appreciate the humor, but there are people already obsessed with the economy and many don't give two fucks about how their economy is going to be completely fucked by climate change.\n\nIt's possible this perspective on it could help convince some otherwise adversarial folks to change their minds.",
 'No, I said the world needs to do something about the climate change, Pakistan alone building dams won’t do anything as the “volume” of floods and rain will only increase ever year.',
...]

def test_4():
    """
    Test evaluation for sentiment between -2 and -1
    """
    reddit = Reddit('the-reddit-climate-change-dataset-comments.csv')
    assert reddit.analized_sentiment(-2,-1) == None

def test_5():
    """
    Test evaluation for score 4
    """
    reddit = Reddit('the-reddit-climate-change-dataset-comments.csv')
    assert reddit.score(4) == ['Yeah but what the above commenter is saying is their base doesn’t want any of that. They detest all of those things, even the small gradual changes. Investing in nuclear energy is a tacit acknowledgement of man made climate change. Any acknowledgement or concession and they will be primaried out in a minute', 'Yeah but what the above commenter is saying is their base doesn’t want any of that. They detest all of those things, even the small gradual changes. Investing in nuclear energy is a tacit acknowledgement of man made climate change. Any acknowledgement or concession and they will be primaried out in a minute'] 

def test_6():
    """
    Test evaluation for score 0
    """
    reddit = Reddit('the-reddit-climate-change-dataset-comments.csv')
    assert reddit.score(0) == ['Yeah but what the above commenter is saying is their base doesn’t want any of that. They detest all of those things, even the small gradual changes. Investing in nuclear energy is a tacit acknowledgement of man made climate change. Any acknowledgement or concession and they will be primaried out in a minute', "You are correct. I have no concern for climate change. I leave that in God's hands to not man's.\n\nHow would you express Christ to the world?"] 


Overwriting test_data.py


In [ ]:
!pytest . -vv

Test session starts (platform: linux, Python 3.7.14, pytest 3.6.4, pytest-sugar 0.9.5)
cachedir: .pytest_cache
rootdir: /content, inifile:
plugins: typeguard-2.7.1, sugar-0.9.5

 test_data.py::test_1 ✓                                           17% █▋        
 test_data.py::test_2 ✓                                           33% ███▍      
 test_data.py::test_3 ✓                                           50% █████     
 test_data.py::test_4 ✓                                           67% ██████▋   
 test_data.py::test_5 ✓                                           83% ████████▍ 
 test_data.py::test_6 ✓                                          100% ██████████

Results (808.01s):
       6 passed
